In [ ]:
import os 
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules 
#from mlxtend.preprocessing import OnehotTransactions


import pandas as pd
import numpy as np

import re
import string

import spacy
import smart_open

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [ ]:
#importing modules
import os 
print('importing "os" operating system')
from io import BytesIO
#from urllib.request import urlopen
from zipfile import ZipFile
import pandas as pd
print('pandas')
import numpy as np
import geopandas as gpd
print('geopandas')
from shapely.geometry import Point
print('shapely')
import numpy as np
print('numpy')
from statsmodels.tsa.stattools import grangercausalitytests
print('statsmodels')
#import pysal as ps
#print('pysal')
import seaborn as sns
print('seaborn')
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
print('matplotlib')


%matplotlib inline

In [ ]:
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard
from sklearn.metrics import jaccard_score

#from sklearn.metrics import jaccard_similarity_score
from difflib import SequenceMatcher
import difflib as dfl

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
import warnings
#warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

## Approri part 2 and spatial auto

In [ ]:
#drugname = 'tysabri' #, , , , 
#drugnamepath = drugname[:6]
#temporal = 'year'
#drugnamepath
#topk = 2

drugname = 'xarelto'#eliquis, xarelto, pradaxa
print('drugname: ', drugname)

drugnamepath = drugname[:6]
temporal = str('year')
drugnamepath
topics=5
topk = 'top'+str(topics)
        
#topk = 'tp'+str(k)
print('topic', topics)
        #topk = 'tp10'
        #tpk
kk=topics

In [ ]:
drugnamepath

## Loading Data

In [ ]:
#data_dirpath1 = r"D:\Research\FAERS_Public\Data\Analysis_Ready_data"+str('\\')+str(topk)+str('\\')+str(drugnamepath)+"_itemset_joinedPT\\"+str(temporal)+"_nai"
data_dirpath1 = r"D:\Research\FAERS_Public_LDA\Data\Analysis_Ready_data\Extracted_data\df_xarelto_europe.csv"
#"D:\Research\FAERS_Public_LDA\Data\Analysis_Ready_data\Extracted_data\df_xarelto_europe_PT_joined.csv"
#os.chdir(data_dirpath1)
data_dirpath1
#print(os.getcwd())
Faers_comparision = pd.read_csv(data_dirpath1)
Faers_comparision.rename({'pt': 'FaersValue'}, axis=1, inplace=True)

Faers_comparision.head()


## Data Filtering

In [ ]:
#Faers_comparision['spatial'], Faers_comparision['temporal'] = Faers_comparision['spatiotemporal'].str.split('_', 1).str
Faers_comparision =Faers_comparision[(Faers_comparision['year']>=2014)&(Faers_comparision['year']<=2017)]
Faers_comparision = Faers_comparision.groupby(['occr_country', 'year'],as_index=False).agg({ 'drugname': lambda x: list(x),'FaersValue': lambda x: list(x)})
#Faers_comparision['drugname']=list(map(set,Faers_comparision['drugname']))

Faers_comparision.head(10)

In [ ]:
Faers_comparision.info()

## Data Cleanup

In [ ]:
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].astype(str)

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str[2:]
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str[:-2]
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].astype(str)
Faers_comparision['FaersValue']=Faers_comparision.FaersValue.str.split(',')

Faers_comparision.FaersValue = Faers_comparision.FaersValue.apply(str)



In [ ]:

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.strip()
#Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lower()
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(u"/",", ")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(u"\\\\", ", ")

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(u"' ","'")

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace('\n',"")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(' \n',"")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace('\n ',"")

Faers_comparision['FaersValue'] = [' '.join(c.split()) for c in Faers_comparision['FaersValue'].astype(str)]

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r"\n",  ' ', regex=True)
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(r"[\"\']", '')
#Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(r"[\"\]", '')

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(',,,',',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(',,',',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace('{','')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace('','')

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str[1:]
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str[:-1]
Faers_comparision['FaersValue'][0]

In [ ]:
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(r"[\"\']", '')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(', ')

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].apply(lambda x: ', '.join(sorted(x.split(','))))

Faers_comparision['FaersValue'][0]


In [ ]:
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(r"[\"\']}{}", '')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip('}')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip("''")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip('}')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].apply(lambda x: ', '.join(sorted(x.split(','))))

Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip("''")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip('}')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip("''")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip('}')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip('}')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip("''")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip('}')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip('}')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip("''")
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.replace(r"}", '')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.rstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(' ,')
Faers_comparision['FaersValue'] =Faers_comparision['FaersValue'].str.split(',').apply(set).str.join(',')
Faers_comparision['FaersValue'] =Faers_comparision['FaersValue'].str.lower()
Faers_comparision['FaersValue'][0]

In [ ]:
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].apply(lambda x: ', '.join(sorted(x.split(','))))
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(',')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(', ')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lstrip(' ,')
Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.strip(',')
Faers_comparision['FaersValue'] = [' '.join(c.split()) for c in Faers_comparision['FaersValue'].astype(str)]

Faers_comparision['FaersValue'][0]

In [ ]:
Faers_comparision['FaersValue'].head(1)

itemset_path = r"G:\Research\FAERS_Public\Data\Analysis_Ready_data\combined_Itemset\tp10"
os.chdir(itemset_path)
Faers_comparision.to_csv("itemset_FAERS_temporal_"+str(temporal)+"_drug_"+str(drugname)+"_top10_version_naive.csv")
Faers_comparision[Faers_comparision['spatial']=='dk']

In [ ]:
#similar(temporal_1_.FaersValue.loc[3], temporal_1_.FaersValue.loc[0])
#haematuria, pulmonary embolism, renal vein thrombosis, spinal pain, pre-existing condition improved, deep vein thrombosis, drug ineffective

# LDA

In [ ]:
#https://stackabuse.com/python-for-nlp-topic-modeling/

In [ ]:
#https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [ ]:
#https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

In [ ]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
Faers_comparision.head()
Faers_comparision.info()

In [ ]:
Faers_comparision['FaersValue'].head()

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=.5, min_df=50, stop_words='english')
doc_term_matrix = count_vect.fit_transform(Faers_comparision['FaersValue'].values.astype('U'))

from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=10, random_state=42)
LDA.fit(doc_term_matrix)

first_topic = LDA.components_[2]
first_topic
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

for i in top_topic_words:
    print(count_vect.get_feature_names()[i])
    pass
#count_vect.get_feature_names()

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

topic_values = LDA.transform(doc_term_matrix)
topic_values.shape
topic_values

Faers_comparision['Topic'] = topic_values.argmax(axis=1)
Faers_comparision.head()

Faers_comparision['FaersValue'][Faers_comparision['Topic']==5].head(10)


In [ ]:
topics

# Gensim

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

In [ ]:
import nltk 
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

Faers_comparision['FaersValue1'] = Faers_comparision['FaersValue'].dropna().apply(lambda x: remove_punct(x))



def tokenization(text):
    text = re.split('\W+', text)
    return text

Faers_comparision['FaersValue2'] = Faers_comparision['FaersValue1'].dropna().apply(lambda x: tokenization(x.lower()))

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
Faers_comparision['FaersValue3'] = Faers_comparision['FaersValue2'].dropna().apply(lambda x: remove_stopwords(x))

ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

Faers_comparision['FaersValue4'] = Faers_comparision['FaersValue3'].dropna().apply(lambda x: stemming(x))



wn = nltk.WordNetLemmatizer()


def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

Faers_comparision['FaersValue5'] = Faers_comparision['FaersValue4'].dropna().apply(lambda x: lemmatizer(x))



def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

Faers_comparision['FaersValue6'] = Faers_comparision['FaersValue5'].dropna().apply(lambda x: lemmatizer(x))



countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(Faers_comparision['FaersValue6'])
print('{} Number of rows has {} words'.format(countVector.shape[0], countVector.shape[1]))




print('Dataset size:',Faers_comparision.shape)
print('Columns are:',Faers_comparision.columns)



In [ ]:
text_list=Faers_comparision['FaersValue6'].tolist()
print(text_list[1])

In [ ]:
dictionary = corpora.Dictionary(text_list)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in text_list]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=topics, random_state=100,
                chunksize=1000, passes=50,iterations=50)


In [ ]:


print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=100))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=text_list, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
lda_model.print_topics()
ttopcs = []
for i,topic in lda_model.show_topics(formatted=True, num_topics=topics, num_words=10):
    #ttopcs.append(str(i)+": "+ topic)
    #with open(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugnamepath)+"\\tp"+str(topics)+"topic_table_summary_"+str(drugname)+"_Output.txt", "w") as text_file:
    print(str(i)+": "+ topic)
    
    
ll = []
#with open(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugnamepath)+"\\tp"+str(topics)+"coherence_table_values_summary_"+str(drugname)+"_Output.txt", "w") as text_files:

for i,topic in lda_model.show_topics(formatted=True, num_topics=topics, num_words=10):
    #print("Num Topics =", m, " has Coherence Value of", round(cv, 3))
    with open(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugnamepath)+"\\tp"+str(topics)+"topic_table_summary_"+str(drugname)+"_Output.txt", "w") as text_file:
        #print("Num Topics =", m, " has Coherence Value of", round(cv, 3))
        #ll.extend("Num Topics =", m, " has Coherence Value of", round(cv, 3))
        ll.append('{}+": "+ {}\n'.format(str(i), topic))
        #text_files.write('\n'.join("Num Topics =", m, " has Coherence Value of", round(cv, 3)))
        text_file.write('\n'.join(ll))

In [ ]:
lda_model[doc_term_matrix[0]] # corpus[0] means the first document.

In [ ]:
doc_term_matrix[0]
#text_list[0]

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=text_list, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:


model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=text_list, start=1, limit=50, step=1)



In [ ]:
# Show graph
limit=50; start=1; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
sns.set(font_scale=2, rc={'figure.figsize':(15,15)})
plt.xticks(fontsize=15, rotation=90) #font size for x ticks, #rotation=90 rotate labels 90 degrees
plt.yticks(fontsize=15, rotation=90)
plt.title("Coherence score vs Num of topics", fontsize=30)#Change here



plt.plot(x, coherence_values)



#plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
#plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
#plt.box('off')

        #plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"Top_"+str(kk)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.png", dpi=300)

        
plt.savefig(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugnamepath)+"\\tp"+str(topics)+"coherence_table_values_summary_"+str(drugname)+"_.png", dpi=500)        
plt.show()# Print the coherence scores


In [ ]:
#model_list
from scipy.interpolate import interp1d
from scipy import interpolate

tck = interpolate.splrep(x, coherence_values, k=3, s=1)

#xnew = np.arange(0, 2*np.pi, np.pi/50)

ynew = interpolate.splev(x, tck, der=0)

plt.plot(x, coherence_values, ynew)
plt.savefig(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugnamepath)+"\\tp"+str(topics)+"coherence_table_values_summary_spline"+str(drugname)+"_.png", dpi=500)        
plt.show()# Print the coherence scores

In [ ]:
# Show graph
limit=50; start=1; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
sns.set(font_scale=2, rc={'figure.figsize':(15,15)})
plt.xticks(fontsize=15, rotation=90) #font size for x ticks, #rotation=90 rotate labels 90 degrees
plt.yticks(fontsize=15, rotation=90)
plt.title("Coherence score vs Num of topics", fontsize=30)#Change here

#plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
#plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
#plt.box('off')

        #plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"Top_"+str(kk)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.png", dpi=300)

        
#plt.savefig(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\Summary_topics\\"+"coherence_table_values_summary_"+str(drugname)+"_10and20.png", dpi=500)        
plt.show()# Print the coherence scores


In [ ]:
# Print the coherence scores
#%%capture cap --no-stderr
ll = []
#with open(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugnamepath)+"\\tp"+str(topics)+"coherence_table_values_summary_"+str(drugname)+"_Output.txt", "w") as text_files:

for m, cv in zip(x, coherence_values):
    #print("Num Topics =", m, " has Coherence Value of", round(cv, 3))
    with open(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugnamepath)+"\\tp"+str(topics)+"coherence_table_values_summary_"+str(drugname)+"_Output.txt", "w") as text_files:
        #print("Num Topics =", m, " has Coherence Value of", round(cv, 3))
        #ll.extend("Num Topics =", m, " has Coherence Value of", round(cv, 3))
        ll.append('Num Topics =", {}, " has Coherence Value of"{}\n'.format(m, cv))
        #text_files.write('\n'.join("Num Topics =", m, " has Coherence Value of", round(cv, 3)))
        text_files.write('\n'.join(ll))

        #text_files.close()  
    #print("Num Topics =", m, " has Coherence Value of", round(cv, 3), file=text_file)

In [ ]:
topic=topics

## Re-reun model with optimized topic #

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=topic, random_state=100,
                chunksize=10000, passes=50,iterations=50)

print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=100))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=text_list, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## topics documents

In [ ]:
#lda_model
get_document_topics = lda_model.get_document_topics(doc_term_matrix[0])
get_document_topics = [lda_model.get_document_topics(item) for item in doc_term_matrix]

In [ ]:
#get_document_topics[0]

In [ ]:
#get_document_topics = lda_model.get_document_topics(doc_term_matrix)
#print(get_document_topics[0])

In [ ]:
get_document_topics = lda_model.get_topics()
get_document_topics

In [ ]:
# Print the Keyword in the 10 topics
#print(lda_model.print_topics())
#doc_lda = lda_model[corpus]
lda_model.print_topics()
sent_topics_df = pd.DataFrame()


def format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                ab = list(itertools.chain([i],[j],[row],[topic_keywords], [wp]))
                list(ab)
                #t =list(i,j,row,topic_num,prop_topic)
                wpp_tpk.append(ab)

            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
#df_dominant_topic.head(10)
sent_topics_df.head()

In [ ]:
##use this one
import itertools
sent_topics_df = pd.DataFrame()
wpp=[]
tpk = []
wpp_tpk =[]
topicss = []
for i, row in enumerate(lda_model[doc_term_matrix]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num)
                wpp.append(wp)
                topic_keywords = ", ".join([word for word, prop in wp])
                tpk.append(topic_keywords)
                #sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                ab = list(itertools.chain([i],[j],[row],[topic_keywords], [wp]))
                list(ab)
                rows = list(row)
                wpp_tpk.append(ab)
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords,ab,rows]), ignore_index=True)

                #wpp_tpk.append(i+j+row+topic_num+prop_topic)
                #print(i,j,row,topic_num,prop_topic)
                #print(row)
            else:
                break
                
sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Kitchen_Sink','matrix']

    # Add original text to the end of the output
#contents = pd.Series(texts)
#sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#return(sent_topics_df)
sent_topics_df.head()
#df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list)

# Format
#df_dominant_topic = df_topic_sents_keywords.reset_index()
#df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Kitchen_Sink', 'Keywords', 'Text']

def format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num)
                wpp.append(wp)
                topic_keywords = ", ".join([word for word, prop in wp])
                tpk.append(topic_keywords)
                #sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                ab = list(itertools.chain([i],[j],[row],[topic_keywords], [wp]))
                topics = [item[0] for item in row]
                topicss.append(topics)
                list(ab)
                rows = list(row)
                #t =list(i,j,row,topic_num,prop_topic)
                wpp_tpk.append(ab)
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords,ab, rows, topics]), ignore_index=True)

            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords','Kitchen_Sink','matrix','topics']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib','Keywords','Kitchen_Sink', 'matrix', 'topics', 'Topic-Text']

# Show
df_dominant_topic.head(2)


In [ ]:
#df_topic_sents_keywords
#[item[0] for item in row]
#topicss
#sent_topics_df.info()
#df_dominant_topic.info()

In [ ]:
#topicss
df_dominant_topic['topics'].head(40)


In [ ]:
topic_keywords[0]

##use this one
sent_topics_df = pd.DataFrame()
wpp=[]
tpk = []
wpp_tpk =[]
for i, row in enumerate(lda_model[doc_term_matrix]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num)
                wpp.append(wp)
                topic_keywords = ", ".join([word for word, prop in wp])
                tpk.append(topic_keywords)
                #sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                ab = list(itertools.chain([i],[j],[row],[topic_keywords], [wp]))
                list(ab)
                rows = list(row)
                wpp_tpk.append(ab)
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords,ab,rows]), ignore_index=True)

                #wpp_tpk.append(i+j+row+topic_num+prop_topic)
                #print(i,j,row,topic_num,prop_topic)
                #print(row)
            else:
                break
                
sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Kitchen_Sink','matrix']

    # Add original text to the end of the output
#contents = pd.Series(texts)
#sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#return(sent_topics_df)
sent_topics_df.head()
#df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list)

# Format
#df_dominant_topic = df_topic_sents_keywords.reset_index()
#df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Kitchen_Sink', 'Keywords', 'Text']



#df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib','Keywords','Kitchen_Sink', 'matrix' 'Text']

# Show
df_dominant_topic.head(2)


In [ ]:
#Faers_comparision= pd.DataFrame()
#df_dominant_topic=pd.DataFrame()

In [ ]:
Faers_comparision.reset_index(inplace=True)
Faers_comparision.rename( columns={'index':'Document_No'}, inplace=True )
#Faers_comparision.tail(1)

In [ ]:
Faers_Topics = pd.merge(Faers_comparision, df_dominant_topic, on=["Document_No"])
Faers_Topics.head()
Faers_Topics.shape
Faers_Topics.head(1)

In [ ]:
#drugname = 'tysabri' #, , , , 
#drugnamepath = drugname[:6]
#temporal = 'year'
#drugnamepath
#topk = 2

drugname = 'xarelto'#eliquis, xarelto, pradaxa
print('drugname: ', drugname)

drugnamepath = drugname[:6]
temporal = str('year')
drugnamepath
topk = 'top'+str(topic)
        
#topk = 'tp'+str(k)
print('topic', topic)
        #topk = 'tp10'
        #tpk
kk=topic

In [ ]:
#"D:\Research\FAERS_Public_LDA\Data\Analysis_Ready_data\tp10\eliqui_itemset_joinedPT\year_nai"

Faers_Topics.to_csv(r"D:\Research\FAERS_Public_LDA\Data\Analysis_Ready_data\tp"+str(topic)+"\\"+str(drugnamepath)+"_itemset_joinedPT\\"+str(temporal)+"_nai\Faers_Topics_countries_grouped_2014_2017tp"+str(topics)+".csv")

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics

#df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=text_list)

vis = pyLDAvis.gensim.prepare(topic_model=lda_model, 
                              corpus=doc_term_matrix, 
                              dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

pyLDAvis.save_html(vis, r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugname[:6])+
                   "\\ldamodelvis"+str(drugname[:6])+"tp"+str(topics)+".html")
pyLDAvis.save_json(vis, r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Summary_topics\\"+str(drugname[:6])+
                   "\\ldamodelvis"+str(drugname[:6])+"tp"+str(topics)+".json")
pyLDAvis.display(vis)


In [ ]:
#mat['topic0']
Faers_Topics.matrix.apply(pd.Series).add_prefix('code_')

test = Faers_Topics.matrix.apply(pd.Series).add_prefix('code_')
test.info()
Faers_Topics.info()


In [ ]:
#Faers_Topics.matrix.apply(pd.Series).add_prefix('code_')

#mat.matrix
#test.code_0[0][0]
for col_num in range(0,5):
    print(col_num)
    pass
    for i, row  in enumerate(test['code_'+str(col_num)]):
    #print(i)
        try:
            if row[0] == 0:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]
        except TypeError:
            pass
        try:
            if row[0] == 1:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]
        except TypeError:
            pass  
        try:
            if row[0] == 2:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]
        except TypeError:
            pass  
        try:
            if row[0] == 3:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]   
        except TypeError:
            pass 
        try:
            if row[0] == 4:
            ##pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1] 
        except TypeError:
            pass 
        try:
            if row[0] == 5:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]
        except TypeError:
            pass

Faers_Topics=Faers_Topics.fillna(0)

print('done')

In [ ]:
#Faers_Topics.matrix.apply(pd.Series).add_prefix('code_')

#mat.matrix
#test.code_0[0][0]
for col_num in range(0,5):
    print(col_num)
    pass
    for i, row  in enumerate(test['code_'+str(col_num)]):
    #print(i)
        try:
            if row[0] == 0:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]
        except TypeError:
            pass
        try:
            if row[0] == 1:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]
        except TypeError:
            pass  
        try:
            if row[0] == 2:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]
        except TypeError:
            pass  
        try:
            if row[0] == 3:
            #pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1]   
        except TypeError:
            pass 
        try:
            if row[0] == 4:
            ##pass
            #print(row[1])
            #print(row[0])
            #print(i)
                Faers_Topics.loc[Faers_Topics.index[i], 'topic'+(str(row[0]))] = row[1] 
        except TypeError:
            pass 
    else: 
        pass
       
Faers_Topics=Faers_Topics.fillna(0)

print('done')

In [ ]:
#pwd
Faers_Topics.info()

#Faers_Topics.head()


In [ ]:
Faers_Topics['year'].unique()

In [ ]:
#create eucliedean distance 
display(Faers_Topics[['occr_country','year','topic0','topic1','topic3','topic4']].head())
#mat.info()
#Faers_Topics[(Faers_Topics['year']==2016),(['CNTRY_NAME','year','topic0','topic1','topic3','topic4','topic5','topic6','topic7','topic8', 'topic9'])]
Faers_Topics2014=Faers_Topics[Faers_Topics['year']==2014]
Faers_Topics2015=Faers_Topics[Faers_Topics['year']==2015]
Faers_Topics2016=Faers_Topics[Faers_Topics['year']==2016]
Faers_Topics2017=Faers_Topics[Faers_Topics['year']==2017]


temporal_1_ = Faers_Topics.query("year == '2014'")
temporal_2_ = Faers_Topics.query("year == '2015'")
temporal_3_ = Faers_Topics.query("year == '2016'")
temporal_4_ = Faers_Topics.query("year == '2017'")


#A=np.array([Faers_Topics]);
#Faers_Topics.topic0
#Faers_Topics2016.head()
var = Faers_Topics.occr_country.tolist()
var =set(var)
len(var)
var =sorted(list(var))
len(var)
Faers_Topics.to_csv(r"D:\Research\FAERS_Public_LDA\Data\Analysis_Ready_data\tp"+str(topic)+"\\"+str(drugnamepath)+"_itemset_joinedPT\\"+str(temporal)+"_nai\Faers_Topics_countries_grouped_2014_2017_topics_separatedtp"+str(topics)+".csv")

In [ ]:
#create eucliedean distance 
display(Faers_Topics[['occr_country','year','topic0','topic1','topic3','topic4']].head())
#mat.info()
#Faers_Topics[(Faers_Topics['year']==2016),(['CNTRY_NAME','year','topic0','topic1','topic3','topic4','topic5','topic6','topic7','topic8', 'topic9'])]
Faers_Topics2014=Faers_Topics[Faers_Topics['year']==2014]
Faers_Topics2015=Faers_Topics[Faers_Topics['year']==2015]
Faers_Topics2016=Faers_Topics[Faers_Topics['year']==2016]
Faers_Topics2017=Faers_Topics[Faers_Topics['year']==2017]


temporal_1_ = Faers_Topics.query("year == '2014'")
temporal_2_ = Faers_Topics.query("year == '2015'")
temporal_3_ = Faers_Topics.query("year == '2016'")
temporal_4_ = Faers_Topics.query("year == '2017'")


#A=np.array([Faers_Topics]);
#Faers_Topics.topic0
#Faers_Topics2016.head()
var = Faers_Topics.occr_country.tolist()
var =set(var)
len(var)
var =sorted(list(var))
len(var)
Faers_Topics.to_csv(r"D:\Research\FAERS_Public_LDA\Data\Analysis_Ready_data\tp"+str(topic)+"\\"+str(drugnamepath)+"_itemset_joinedPT\\"+str(temporal)+"_nai\Faers_Topics_countries_grouped_2014_2017_topics_separatedtp"+str(topics)+".csv")

In [ ]:
display(Faers_Topics[['occr_country','year','topic0','topic1','topic2','topic3','topic4']].head(5))
#Faers_Topics[Faers_Topics['occr_country']=='de']

In [ ]:
true_list= []
temp_list= []

for x in Faers_Topics.occr_country:
    temp_list.append(x)

for x in temp_list:
    
    if temp_list.count(x) ==4:
        true_list.append(x)
var = true_list
var =set(var)
len(var)
var =sorted(list(var))
len(var)


In [ ]:
countofcountries = len(var)
countofcountries

## Spatial analysis and Output 

#Do not delete - this is used for generating variables in the next cell
year=2017
for x in range(0,len(var)):
    print(var[x]+str(year)+"=np.array([Faers_Topics"+str(year)+".iloc["+str(x)+"]['topic0'],Faers_Topics"+str(year)+".iloc["+str(x)+"]['topic1'],Faers_Topics"+str(year)+".iloc["+str(x)+"]['topic2'],Faers_Topics"+str(year)+".iloc["+str(x)+"]['topic3'],"
                          "Faers_Topics"+str(year)+".iloc["+str(x)+"]['topic4']])")

In [ ]:
#2014
at2014=np.array([Faers_Topics2014.iloc[0]['topic0'],Faers_Topics2014.iloc[0]['topic1'],Faers_Topics2014.iloc[0]['topic2'],Faers_Topics2014.iloc[0]['topic3'],Faers_Topics2014.iloc[0]['topic4']])
be2014=np.array([Faers_Topics2014.iloc[1]['topic0'],Faers_Topics2014.iloc[1]['topic1'],Faers_Topics2014.iloc[1]['topic2'],Faers_Topics2014.iloc[1]['topic3'],Faers_Topics2014.iloc[1]['topic4']])
bg2014=np.array([Faers_Topics2014.iloc[2]['topic0'],Faers_Topics2014.iloc[2]['topic1'],Faers_Topics2014.iloc[2]['topic2'],Faers_Topics2014.iloc[2]['topic3'],Faers_Topics2014.iloc[2]['topic4']])
by2014=np.array([Faers_Topics2014.iloc[3]['topic0'],Faers_Topics2014.iloc[3]['topic1'],Faers_Topics2014.iloc[3]['topic2'],Faers_Topics2014.iloc[3]['topic3'],Faers_Topics2014.iloc[3]['topic4']])
ch2014=np.array([Faers_Topics2014.iloc[4]['topic0'],Faers_Topics2014.iloc[4]['topic1'],Faers_Topics2014.iloc[4]['topic2'],Faers_Topics2014.iloc[4]['topic3'],Faers_Topics2014.iloc[4]['topic4']])
cz2014=np.array([Faers_Topics2014.iloc[5]['topic0'],Faers_Topics2014.iloc[5]['topic1'],Faers_Topics2014.iloc[5]['topic2'],Faers_Topics2014.iloc[5]['topic3'],Faers_Topics2014.iloc[5]['topic4']])
de2014=np.array([Faers_Topics2014.iloc[6]['topic0'],Faers_Topics2014.iloc[6]['topic1'],Faers_Topics2014.iloc[6]['topic2'],Faers_Topics2014.iloc[6]['topic3'],Faers_Topics2014.iloc[6]['topic4']])
dk2014=np.array([Faers_Topics2014.iloc[7]['topic0'],Faers_Topics2014.iloc[7]['topic1'],Faers_Topics2014.iloc[7]['topic2'],Faers_Topics2014.iloc[7]['topic3'],Faers_Topics2014.iloc[7]['topic4']])
ee2014=np.array([Faers_Topics2014.iloc[8]['topic0'],Faers_Topics2014.iloc[8]['topic1'],Faers_Topics2014.iloc[8]['topic2'],Faers_Topics2014.iloc[8]['topic3'],Faers_Topics2014.iloc[8]['topic4']])
es2014=np.array([Faers_Topics2014.iloc[9]['topic0'],Faers_Topics2014.iloc[9]['topic1'],Faers_Topics2014.iloc[9]['topic2'],Faers_Topics2014.iloc[9]['topic3'],Faers_Topics2014.iloc[9]['topic4']])
fi2014=np.array([Faers_Topics2014.iloc[10]['topic0'],Faers_Topics2014.iloc[10]['topic1'],Faers_Topics2014.iloc[10]['topic2'],Faers_Topics2014.iloc[10]['topic3'],Faers_Topics2014.iloc[10]['topic4']])
fr2014=np.array([Faers_Topics2014.iloc[11]['topic0'],Faers_Topics2014.iloc[11]['topic1'],Faers_Topics2014.iloc[11]['topic2'],Faers_Topics2014.iloc[11]['topic3'],Faers_Topics2014.iloc[11]['topic4']])
gb2014=np.array([Faers_Topics2014.iloc[12]['topic0'],Faers_Topics2014.iloc[12]['topic1'],Faers_Topics2014.iloc[12]['topic2'],Faers_Topics2014.iloc[12]['topic3'],Faers_Topics2014.iloc[12]['topic4']])
gr2014=np.array([Faers_Topics2014.iloc[13]['topic0'],Faers_Topics2014.iloc[13]['topic1'],Faers_Topics2014.iloc[13]['topic2'],Faers_Topics2014.iloc[13]['topic3'],Faers_Topics2014.iloc[13]['topic4']])
hr2014=np.array([Faers_Topics2014.iloc[14]['topic0'],Faers_Topics2014.iloc[14]['topic1'],Faers_Topics2014.iloc[14]['topic2'],Faers_Topics2014.iloc[14]['topic3'],Faers_Topics2014.iloc[14]['topic4']])
hu2014=np.array([Faers_Topics2014.iloc[15]['topic0'],Faers_Topics2014.iloc[15]['topic1'],Faers_Topics2014.iloc[15]['topic2'],Faers_Topics2014.iloc[15]['topic3'],Faers_Topics2014.iloc[15]['topic4']])
ie2014=np.array([Faers_Topics2014.iloc[16]['topic0'],Faers_Topics2014.iloc[16]['topic1'],Faers_Topics2014.iloc[16]['topic2'],Faers_Topics2014.iloc[16]['topic3'],Faers_Topics2014.iloc[16]['topic4']])
it2014=np.array([Faers_Topics2014.iloc[17]['topic0'],Faers_Topics2014.iloc[17]['topic1'],Faers_Topics2014.iloc[17]['topic2'],Faers_Topics2014.iloc[17]['topic3'],Faers_Topics2014.iloc[17]['topic4']])
nl2014=np.array([Faers_Topics2014.iloc[18]['topic0'],Faers_Topics2014.iloc[18]['topic1'],Faers_Topics2014.iloc[18]['topic2'],Faers_Topics2014.iloc[18]['topic3'],Faers_Topics2014.iloc[18]['topic4']])
no2014=np.array([Faers_Topics2014.iloc[19]['topic0'],Faers_Topics2014.iloc[19]['topic1'],Faers_Topics2014.iloc[19]['topic2'],Faers_Topics2014.iloc[19]['topic3'],Faers_Topics2014.iloc[19]['topic4']])
pl2014=np.array([Faers_Topics2014.iloc[20]['topic0'],Faers_Topics2014.iloc[20]['topic1'],Faers_Topics2014.iloc[20]['topic2'],Faers_Topics2014.iloc[20]['topic3'],Faers_Topics2014.iloc[20]['topic4']])
pt2014=np.array([Faers_Topics2014.iloc[21]['topic0'],Faers_Topics2014.iloc[21]['topic1'],Faers_Topics2014.iloc[21]['topic2'],Faers_Topics2014.iloc[21]['topic3'],Faers_Topics2014.iloc[21]['topic4']])
se2014=np.array([Faers_Topics2014.iloc[22]['topic0'],Faers_Topics2014.iloc[22]['topic1'],Faers_Topics2014.iloc[22]['topic2'],Faers_Topics2014.iloc[22]['topic3'],Faers_Topics2014.iloc[22]['topic4']])
si2014=np.array([Faers_Topics2014.iloc[23]['topic0'],Faers_Topics2014.iloc[23]['topic1'],Faers_Topics2014.iloc[23]['topic2'],Faers_Topics2014.iloc[23]['topic3'],Faers_Topics2014.iloc[23]['topic4']])
sk2014=np.array([Faers_Topics2014.iloc[24]['topic0'],Faers_Topics2014.iloc[24]['topic1'],Faers_Topics2014.iloc[24]['topic2'],Faers_Topics2014.iloc[24]['topic3'],Faers_Topics2014.iloc[24]['topic4']])
tr2014=np.array([Faers_Topics2014.iloc[25]['topic0'],Faers_Topics2014.iloc[25]['topic1'],Faers_Topics2014.iloc[25]['topic2'],Faers_Topics2014.iloc[25]['topic3'],Faers_Topics2014.iloc[25]['topic4']])

In [ ]:
#2015
at2015=np.array([Faers_Topics2015.iloc[0]['topic0'],Faers_Topics2015.iloc[0]['topic1'],Faers_Topics2015.iloc[0]['topic2'],Faers_Topics2015.iloc[0]['topic3'],Faers_Topics2015.iloc[0]['topic4']])
be2015=np.array([Faers_Topics2015.iloc[1]['topic0'],Faers_Topics2015.iloc[1]['topic1'],Faers_Topics2015.iloc[1]['topic2'],Faers_Topics2015.iloc[1]['topic3'],Faers_Topics2015.iloc[1]['topic4']])
bg2015=np.array([Faers_Topics2015.iloc[2]['topic0'],Faers_Topics2015.iloc[2]['topic1'],Faers_Topics2015.iloc[2]['topic2'],Faers_Topics2015.iloc[2]['topic3'],Faers_Topics2015.iloc[2]['topic4']])
by2015=np.array([Faers_Topics2015.iloc[3]['topic0'],Faers_Topics2015.iloc[3]['topic1'],Faers_Topics2015.iloc[3]['topic2'],Faers_Topics2015.iloc[3]['topic3'],Faers_Topics2015.iloc[3]['topic4']])
ch2015=np.array([Faers_Topics2015.iloc[4]['topic0'],Faers_Topics2015.iloc[4]['topic1'],Faers_Topics2015.iloc[4]['topic2'],Faers_Topics2015.iloc[4]['topic3'],Faers_Topics2015.iloc[4]['topic4']])
cz2015=np.array([Faers_Topics2015.iloc[5]['topic0'],Faers_Topics2015.iloc[5]['topic1'],Faers_Topics2015.iloc[5]['topic2'],Faers_Topics2015.iloc[5]['topic3'],Faers_Topics2015.iloc[5]['topic4']])
de2015=np.array([Faers_Topics2015.iloc[6]['topic0'],Faers_Topics2015.iloc[6]['topic1'],Faers_Topics2015.iloc[6]['topic2'],Faers_Topics2015.iloc[6]['topic3'],Faers_Topics2015.iloc[6]['topic4']])
dk2015=np.array([Faers_Topics2015.iloc[7]['topic0'],Faers_Topics2015.iloc[7]['topic1'],Faers_Topics2015.iloc[7]['topic2'],Faers_Topics2015.iloc[7]['topic3'],Faers_Topics2015.iloc[7]['topic4']])
ee2015=np.array([Faers_Topics2015.iloc[8]['topic0'],Faers_Topics2015.iloc[8]['topic1'],Faers_Topics2015.iloc[8]['topic2'],Faers_Topics2015.iloc[8]['topic3'],Faers_Topics2015.iloc[8]['topic4']])
es2015=np.array([Faers_Topics2015.iloc[9]['topic0'],Faers_Topics2015.iloc[9]['topic1'],Faers_Topics2015.iloc[9]['topic2'],Faers_Topics2015.iloc[9]['topic3'],Faers_Topics2015.iloc[9]['topic4']])
fi2015=np.array([Faers_Topics2015.iloc[10]['topic0'],Faers_Topics2015.iloc[10]['topic1'],Faers_Topics2015.iloc[10]['topic2'],Faers_Topics2015.iloc[10]['topic3'],Faers_Topics2015.iloc[10]['topic4']])
fr2015=np.array([Faers_Topics2015.iloc[11]['topic0'],Faers_Topics2015.iloc[11]['topic1'],Faers_Topics2015.iloc[11]['topic2'],Faers_Topics2015.iloc[11]['topic3'],Faers_Topics2015.iloc[11]['topic4']])
gb2015=np.array([Faers_Topics2015.iloc[12]['topic0'],Faers_Topics2015.iloc[12]['topic1'],Faers_Topics2015.iloc[12]['topic2'],Faers_Topics2015.iloc[12]['topic3'],Faers_Topics2015.iloc[12]['topic4']])
gr2015=np.array([Faers_Topics2015.iloc[13]['topic0'],Faers_Topics2015.iloc[13]['topic1'],Faers_Topics2015.iloc[13]['topic2'],Faers_Topics2015.iloc[13]['topic3'],Faers_Topics2015.iloc[13]['topic4']])
hr2015=np.array([Faers_Topics2015.iloc[14]['topic0'],Faers_Topics2015.iloc[14]['topic1'],Faers_Topics2015.iloc[14]['topic2'],Faers_Topics2015.iloc[14]['topic3'],Faers_Topics2015.iloc[14]['topic4']])
hu2015=np.array([Faers_Topics2015.iloc[15]['topic0'],Faers_Topics2015.iloc[15]['topic1'],Faers_Topics2015.iloc[15]['topic2'],Faers_Topics2015.iloc[15]['topic3'],Faers_Topics2015.iloc[15]['topic4']])
ie2015=np.array([Faers_Topics2015.iloc[16]['topic0'],Faers_Topics2015.iloc[16]['topic1'],Faers_Topics2015.iloc[16]['topic2'],Faers_Topics2015.iloc[16]['topic3'],Faers_Topics2015.iloc[16]['topic4']])
it2015=np.array([Faers_Topics2015.iloc[17]['topic0'],Faers_Topics2015.iloc[17]['topic1'],Faers_Topics2015.iloc[17]['topic2'],Faers_Topics2015.iloc[17]['topic3'],Faers_Topics2015.iloc[17]['topic4']])
nl2015=np.array([Faers_Topics2015.iloc[18]['topic0'],Faers_Topics2015.iloc[18]['topic1'],Faers_Topics2015.iloc[18]['topic2'],Faers_Topics2015.iloc[18]['topic3'],Faers_Topics2015.iloc[18]['topic4']])
no2015=np.array([Faers_Topics2015.iloc[19]['topic0'],Faers_Topics2015.iloc[19]['topic1'],Faers_Topics2015.iloc[19]['topic2'],Faers_Topics2015.iloc[19]['topic3'],Faers_Topics2015.iloc[19]['topic4']])
pl2015=np.array([Faers_Topics2015.iloc[20]['topic0'],Faers_Topics2015.iloc[20]['topic1'],Faers_Topics2015.iloc[20]['topic2'],Faers_Topics2015.iloc[20]['topic3'],Faers_Topics2015.iloc[20]['topic4']])
pt2015=np.array([Faers_Topics2015.iloc[21]['topic0'],Faers_Topics2015.iloc[21]['topic1'],Faers_Topics2015.iloc[21]['topic2'],Faers_Topics2015.iloc[21]['topic3'],Faers_Topics2015.iloc[21]['topic4']])
se2015=np.array([Faers_Topics2015.iloc[22]['topic0'],Faers_Topics2015.iloc[22]['topic1'],Faers_Topics2015.iloc[22]['topic2'],Faers_Topics2015.iloc[22]['topic3'],Faers_Topics2015.iloc[22]['topic4']])
si2015=np.array([Faers_Topics2015.iloc[23]['topic0'],Faers_Topics2015.iloc[23]['topic1'],Faers_Topics2015.iloc[23]['topic2'],Faers_Topics2015.iloc[23]['topic3'],Faers_Topics2015.iloc[23]['topic4']])
sk2015=np.array([Faers_Topics2015.iloc[24]['topic0'],Faers_Topics2015.iloc[24]['topic1'],Faers_Topics2015.iloc[24]['topic2'],Faers_Topics2015.iloc[24]['topic3'],Faers_Topics2015.iloc[24]['topic4']])
tr2015=np.array([Faers_Topics2015.iloc[25]['topic0'],Faers_Topics2015.iloc[25]['topic1'],Faers_Topics2015.iloc[25]['topic2'],Faers_Topics2015.iloc[25]['topic3'],Faers_Topics2015.iloc[25]['topic4']])

In [ ]:
#2016
at2016=np.array([Faers_Topics2016.iloc[0]['topic0'],Faers_Topics2016.iloc[0]['topic1'],Faers_Topics2016.iloc[0]['topic2'],Faers_Topics2016.iloc[0]['topic3'],Faers_Topics2016.iloc[0]['topic4']])
be2016=np.array([Faers_Topics2016.iloc[1]['topic0'],Faers_Topics2016.iloc[1]['topic1'],Faers_Topics2016.iloc[1]['topic2'],Faers_Topics2016.iloc[1]['topic3'],Faers_Topics2016.iloc[1]['topic4']])
bg2016=np.array([Faers_Topics2016.iloc[2]['topic0'],Faers_Topics2016.iloc[2]['topic1'],Faers_Topics2016.iloc[2]['topic2'],Faers_Topics2016.iloc[2]['topic3'],Faers_Topics2016.iloc[2]['topic4']])
by2016=np.array([Faers_Topics2016.iloc[3]['topic0'],Faers_Topics2016.iloc[3]['topic1'],Faers_Topics2016.iloc[3]['topic2'],Faers_Topics2016.iloc[3]['topic3'],Faers_Topics2016.iloc[3]['topic4']])
ch2016=np.array([Faers_Topics2016.iloc[4]['topic0'],Faers_Topics2016.iloc[4]['topic1'],Faers_Topics2016.iloc[4]['topic2'],Faers_Topics2016.iloc[4]['topic3'],Faers_Topics2016.iloc[4]['topic4']])
cz2016=np.array([Faers_Topics2016.iloc[5]['topic0'],Faers_Topics2016.iloc[5]['topic1'],Faers_Topics2016.iloc[5]['topic2'],Faers_Topics2016.iloc[5]['topic3'],Faers_Topics2016.iloc[5]['topic4']])
de2016=np.array([Faers_Topics2016.iloc[6]['topic0'],Faers_Topics2016.iloc[6]['topic1'],Faers_Topics2016.iloc[6]['topic2'],Faers_Topics2016.iloc[6]['topic3'],Faers_Topics2016.iloc[6]['topic4']])
dk2016=np.array([Faers_Topics2016.iloc[7]['topic0'],Faers_Topics2016.iloc[7]['topic1'],Faers_Topics2016.iloc[7]['topic2'],Faers_Topics2016.iloc[7]['topic3'],Faers_Topics2016.iloc[7]['topic4']])
ee2016=np.array([Faers_Topics2016.iloc[8]['topic0'],Faers_Topics2016.iloc[8]['topic1'],Faers_Topics2016.iloc[8]['topic2'],Faers_Topics2016.iloc[8]['topic3'],Faers_Topics2016.iloc[8]['topic4']])
es2016=np.array([Faers_Topics2016.iloc[9]['topic0'],Faers_Topics2016.iloc[9]['topic1'],Faers_Topics2016.iloc[9]['topic2'],Faers_Topics2016.iloc[9]['topic3'],Faers_Topics2016.iloc[9]['topic4']])
fi2016=np.array([Faers_Topics2016.iloc[10]['topic0'],Faers_Topics2016.iloc[10]['topic1'],Faers_Topics2016.iloc[10]['topic2'],Faers_Topics2016.iloc[10]['topic3'],Faers_Topics2016.iloc[10]['topic4']])
fr2016=np.array([Faers_Topics2016.iloc[11]['topic0'],Faers_Topics2016.iloc[11]['topic1'],Faers_Topics2016.iloc[11]['topic2'],Faers_Topics2016.iloc[11]['topic3'],Faers_Topics2016.iloc[11]['topic4']])
gb2016=np.array([Faers_Topics2016.iloc[12]['topic0'],Faers_Topics2016.iloc[12]['topic1'],Faers_Topics2016.iloc[12]['topic2'],Faers_Topics2016.iloc[12]['topic3'],Faers_Topics2016.iloc[12]['topic4']])
gr2016=np.array([Faers_Topics2016.iloc[13]['topic0'],Faers_Topics2016.iloc[13]['topic1'],Faers_Topics2016.iloc[13]['topic2'],Faers_Topics2016.iloc[13]['topic3'],Faers_Topics2016.iloc[13]['topic4']])
hr2016=np.array([Faers_Topics2016.iloc[14]['topic0'],Faers_Topics2016.iloc[14]['topic1'],Faers_Topics2016.iloc[14]['topic2'],Faers_Topics2016.iloc[14]['topic3'],Faers_Topics2016.iloc[14]['topic4']])
hu2016=np.array([Faers_Topics2016.iloc[15]['topic0'],Faers_Topics2016.iloc[15]['topic1'],Faers_Topics2016.iloc[15]['topic2'],Faers_Topics2016.iloc[15]['topic3'],Faers_Topics2016.iloc[15]['topic4']])
ie2016=np.array([Faers_Topics2016.iloc[16]['topic0'],Faers_Topics2016.iloc[16]['topic1'],Faers_Topics2016.iloc[16]['topic2'],Faers_Topics2016.iloc[16]['topic3'],Faers_Topics2016.iloc[16]['topic4']])
it2016=np.array([Faers_Topics2016.iloc[17]['topic0'],Faers_Topics2016.iloc[17]['topic1'],Faers_Topics2016.iloc[17]['topic2'],Faers_Topics2016.iloc[17]['topic3'],Faers_Topics2016.iloc[17]['topic4']])
nl2016=np.array([Faers_Topics2016.iloc[18]['topic0'],Faers_Topics2016.iloc[18]['topic1'],Faers_Topics2016.iloc[18]['topic2'],Faers_Topics2016.iloc[18]['topic3'],Faers_Topics2016.iloc[18]['topic4']])
no2016=np.array([Faers_Topics2016.iloc[19]['topic0'],Faers_Topics2016.iloc[19]['topic1'],Faers_Topics2016.iloc[19]['topic2'],Faers_Topics2016.iloc[19]['topic3'],Faers_Topics2016.iloc[19]['topic4']])
pl2016=np.array([Faers_Topics2016.iloc[20]['topic0'],Faers_Topics2016.iloc[20]['topic1'],Faers_Topics2016.iloc[20]['topic2'],Faers_Topics2016.iloc[20]['topic3'],Faers_Topics2016.iloc[20]['topic4']])
pt2016=np.array([Faers_Topics2016.iloc[21]['topic0'],Faers_Topics2016.iloc[21]['topic1'],Faers_Topics2016.iloc[21]['topic2'],Faers_Topics2016.iloc[21]['topic3'],Faers_Topics2016.iloc[21]['topic4']])
se2016=np.array([Faers_Topics2016.iloc[22]['topic0'],Faers_Topics2016.iloc[22]['topic1'],Faers_Topics2016.iloc[22]['topic2'],Faers_Topics2016.iloc[22]['topic3'],Faers_Topics2016.iloc[22]['topic4']])
si2016=np.array([Faers_Topics2016.iloc[23]['topic0'],Faers_Topics2016.iloc[23]['topic1'],Faers_Topics2016.iloc[23]['topic2'],Faers_Topics2016.iloc[23]['topic3'],Faers_Topics2016.iloc[23]['topic4']])
sk2016=np.array([Faers_Topics2016.iloc[24]['topic0'],Faers_Topics2016.iloc[24]['topic1'],Faers_Topics2016.iloc[24]['topic2'],Faers_Topics2016.iloc[24]['topic3'],Faers_Topics2016.iloc[24]['topic4']])
tr2016=np.array([Faers_Topics2016.iloc[25]['topic0'],Faers_Topics2016.iloc[25]['topic1'],Faers_Topics2016.iloc[25]['topic2'],Faers_Topics2016.iloc[25]['topic3'],Faers_Topics2016.iloc[25]['topic4']])

In [ ]:
#2017
at2017=np.array([Faers_Topics2017.iloc[0]['topic0'],Faers_Topics2017.iloc[0]['topic1'],Faers_Topics2017.iloc[0]['topic2'],Faers_Topics2017.iloc[0]['topic3'],Faers_Topics2017.iloc[0]['topic4']])
be2017=np.array([Faers_Topics2017.iloc[1]['topic0'],Faers_Topics2017.iloc[1]['topic1'],Faers_Topics2017.iloc[1]['topic2'],Faers_Topics2017.iloc[1]['topic3'],Faers_Topics2017.iloc[1]['topic4']])
bg2017=np.array([Faers_Topics2017.iloc[2]['topic0'],Faers_Topics2017.iloc[2]['topic1'],Faers_Topics2017.iloc[2]['topic2'],Faers_Topics2017.iloc[2]['topic3'],Faers_Topics2017.iloc[2]['topic4']])
by2017=np.array([Faers_Topics2017.iloc[3]['topic0'],Faers_Topics2017.iloc[3]['topic1'],Faers_Topics2017.iloc[3]['topic2'],Faers_Topics2017.iloc[3]['topic3'],Faers_Topics2017.iloc[3]['topic4']])
ch2017=np.array([Faers_Topics2017.iloc[4]['topic0'],Faers_Topics2017.iloc[4]['topic1'],Faers_Topics2017.iloc[4]['topic2'],Faers_Topics2017.iloc[4]['topic3'],Faers_Topics2017.iloc[4]['topic4']])
cz2017=np.array([Faers_Topics2017.iloc[5]['topic0'],Faers_Topics2017.iloc[5]['topic1'],Faers_Topics2017.iloc[5]['topic2'],Faers_Topics2017.iloc[5]['topic3'],Faers_Topics2017.iloc[5]['topic4']])
de2017=np.array([Faers_Topics2017.iloc[6]['topic0'],Faers_Topics2017.iloc[6]['topic1'],Faers_Topics2017.iloc[6]['topic2'],Faers_Topics2017.iloc[6]['topic3'],Faers_Topics2017.iloc[6]['topic4']])
dk2017=np.array([Faers_Topics2017.iloc[7]['topic0'],Faers_Topics2017.iloc[7]['topic1'],Faers_Topics2017.iloc[7]['topic2'],Faers_Topics2017.iloc[7]['topic3'],Faers_Topics2017.iloc[7]['topic4']])
ee2017=np.array([Faers_Topics2017.iloc[8]['topic0'],Faers_Topics2017.iloc[8]['topic1'],Faers_Topics2017.iloc[8]['topic2'],Faers_Topics2017.iloc[8]['topic3'],Faers_Topics2017.iloc[8]['topic4']])
es2017=np.array([Faers_Topics2017.iloc[9]['topic0'],Faers_Topics2017.iloc[9]['topic1'],Faers_Topics2017.iloc[9]['topic2'],Faers_Topics2017.iloc[9]['topic3'],Faers_Topics2017.iloc[9]['topic4']])
fi2017=np.array([Faers_Topics2017.iloc[10]['topic0'],Faers_Topics2017.iloc[10]['topic1'],Faers_Topics2017.iloc[10]['topic2'],Faers_Topics2017.iloc[10]['topic3'],Faers_Topics2017.iloc[10]['topic4']])
fr2017=np.array([Faers_Topics2017.iloc[11]['topic0'],Faers_Topics2017.iloc[11]['topic1'],Faers_Topics2017.iloc[11]['topic2'],Faers_Topics2017.iloc[11]['topic3'],Faers_Topics2017.iloc[11]['topic4']])
gb2017=np.array([Faers_Topics2017.iloc[12]['topic0'],Faers_Topics2017.iloc[12]['topic1'],Faers_Topics2017.iloc[12]['topic2'],Faers_Topics2017.iloc[12]['topic3'],Faers_Topics2017.iloc[12]['topic4']])
gr2017=np.array([Faers_Topics2017.iloc[13]['topic0'],Faers_Topics2017.iloc[13]['topic1'],Faers_Topics2017.iloc[13]['topic2'],Faers_Topics2017.iloc[13]['topic3'],Faers_Topics2017.iloc[13]['topic4']])
hr2017=np.array([Faers_Topics2017.iloc[14]['topic0'],Faers_Topics2017.iloc[14]['topic1'],Faers_Topics2017.iloc[14]['topic2'],Faers_Topics2017.iloc[14]['topic3'],Faers_Topics2017.iloc[14]['topic4']])
hu2017=np.array([Faers_Topics2017.iloc[15]['topic0'],Faers_Topics2017.iloc[15]['topic1'],Faers_Topics2017.iloc[15]['topic2'],Faers_Topics2017.iloc[15]['topic3'],Faers_Topics2017.iloc[15]['topic4']])
ie2017=np.array([Faers_Topics2017.iloc[16]['topic0'],Faers_Topics2017.iloc[16]['topic1'],Faers_Topics2017.iloc[16]['topic2'],Faers_Topics2017.iloc[16]['topic3'],Faers_Topics2017.iloc[16]['topic4']])
it2017=np.array([Faers_Topics2017.iloc[17]['topic0'],Faers_Topics2017.iloc[17]['topic1'],Faers_Topics2017.iloc[17]['topic2'],Faers_Topics2017.iloc[17]['topic3'],Faers_Topics2017.iloc[17]['topic4']])
nl2017=np.array([Faers_Topics2017.iloc[18]['topic0'],Faers_Topics2017.iloc[18]['topic1'],Faers_Topics2017.iloc[18]['topic2'],Faers_Topics2017.iloc[18]['topic3'],Faers_Topics2017.iloc[18]['topic4']])
no2017=np.array([Faers_Topics2017.iloc[19]['topic0'],Faers_Topics2017.iloc[19]['topic1'],Faers_Topics2017.iloc[19]['topic2'],Faers_Topics2017.iloc[19]['topic3'],Faers_Topics2017.iloc[19]['topic4']])
pl2017=np.array([Faers_Topics2017.iloc[20]['topic0'],Faers_Topics2017.iloc[20]['topic1'],Faers_Topics2017.iloc[20]['topic2'],Faers_Topics2017.iloc[20]['topic3'],Faers_Topics2017.iloc[20]['topic4']])
pt2017=np.array([Faers_Topics2017.iloc[21]['topic0'],Faers_Topics2017.iloc[21]['topic1'],Faers_Topics2017.iloc[21]['topic2'],Faers_Topics2017.iloc[21]['topic3'],Faers_Topics2017.iloc[21]['topic4']])
se2017=np.array([Faers_Topics2017.iloc[22]['topic0'],Faers_Topics2017.iloc[22]['topic1'],Faers_Topics2017.iloc[22]['topic2'],Faers_Topics2017.iloc[22]['topic3'],Faers_Topics2017.iloc[22]['topic4']])
si2017=np.array([Faers_Topics2017.iloc[23]['topic0'],Faers_Topics2017.iloc[23]['topic1'],Faers_Topics2017.iloc[23]['topic2'],Faers_Topics2017.iloc[23]['topic3'],Faers_Topics2017.iloc[23]['topic4']])
sk2017=np.array([Faers_Topics2017.iloc[24]['topic0'],Faers_Topics2017.iloc[24]['topic1'],Faers_Topics2017.iloc[24]['topic2'],Faers_Topics2017.iloc[24]['topic3'],Faers_Topics2017.iloc[24]['topic4']])
tr2017=np.array([Faers_Topics2017.iloc[25]['topic0'],Faers_Topics2017.iloc[25]['topic1'],Faers_Topics2017.iloc[25]['topic2'],Faers_Topics2017.iloc[25]['topic3'],Faers_Topics2017.iloc[25]['topic4']])

#Do not delete - this is used for generating variables in the next cell
year = 2017
for x in range(0,len(var)):
    print(str(var[x])+str(year)+'_all =  np.array([')
    for y in range(0,len(var)):
        print('np.sqrt(np.sum(('+str(var[x])+str(year)+' - '+str(var[y])+str(year)+")**2)),")
        #print(var[x], var[y])
    print('])')

In [ ]:
#2014
at2014_all =  np.array([
np.sqrt(np.sum((at2014 - at2014)**2)),
np.sqrt(np.sum((at2014 - be2014)**2)),
np.sqrt(np.sum((at2014 - bg2014)**2)),
np.sqrt(np.sum((at2014 - by2014)**2)),
np.sqrt(np.sum((at2014 - ch2014)**2)),
np.sqrt(np.sum((at2014 - cz2014)**2)),
np.sqrt(np.sum((at2014 - de2014)**2)),
np.sqrt(np.sum((at2014 - dk2014)**2)),
np.sqrt(np.sum((at2014 - ee2014)**2)),
np.sqrt(np.sum((at2014 - es2014)**2)),
np.sqrt(np.sum((at2014 - fi2014)**2)),
np.sqrt(np.sum((at2014 - fr2014)**2)),
np.sqrt(np.sum((at2014 - gb2014)**2)),
np.sqrt(np.sum((at2014 - gr2014)**2)),
np.sqrt(np.sum((at2014 - hr2014)**2)),
np.sqrt(np.sum((at2014 - hu2014)**2)),
np.sqrt(np.sum((at2014 - ie2014)**2)),
np.sqrt(np.sum((at2014 - it2014)**2)),
np.sqrt(np.sum((at2014 - nl2014)**2)),
np.sqrt(np.sum((at2014 - no2014)**2)),
np.sqrt(np.sum((at2014 - pl2014)**2)),
np.sqrt(np.sum((at2014 - pt2014)**2)),
np.sqrt(np.sum((at2014 - se2014)**2)),
np.sqrt(np.sum((at2014 - si2014)**2)),
np.sqrt(np.sum((at2014 - sk2014)**2)),
np.sqrt(np.sum((at2014 - tr2014)**2)),
])
be2014_all =  np.array([
np.sqrt(np.sum((be2014 - at2014)**2)),
np.sqrt(np.sum((be2014 - be2014)**2)),
np.sqrt(np.sum((be2014 - bg2014)**2)),
np.sqrt(np.sum((be2014 - by2014)**2)),
np.sqrt(np.sum((be2014 - ch2014)**2)),
np.sqrt(np.sum((be2014 - cz2014)**2)),
np.sqrt(np.sum((be2014 - de2014)**2)),
np.sqrt(np.sum((be2014 - dk2014)**2)),
np.sqrt(np.sum((be2014 - ee2014)**2)),
np.sqrt(np.sum((be2014 - es2014)**2)),
np.sqrt(np.sum((be2014 - fi2014)**2)),
np.sqrt(np.sum((be2014 - fr2014)**2)),
np.sqrt(np.sum((be2014 - gb2014)**2)),
np.sqrt(np.sum((be2014 - gr2014)**2)),
np.sqrt(np.sum((be2014 - hr2014)**2)),
np.sqrt(np.sum((be2014 - hu2014)**2)),
np.sqrt(np.sum((be2014 - ie2014)**2)),
np.sqrt(np.sum((be2014 - it2014)**2)),
np.sqrt(np.sum((be2014 - nl2014)**2)),
np.sqrt(np.sum((be2014 - no2014)**2)),
np.sqrt(np.sum((be2014 - pl2014)**2)),
np.sqrt(np.sum((be2014 - pt2014)**2)),
np.sqrt(np.sum((be2014 - se2014)**2)),
np.sqrt(np.sum((be2014 - si2014)**2)),
np.sqrt(np.sum((be2014 - sk2014)**2)),
np.sqrt(np.sum((be2014 - tr2014)**2)),
])
bg2014_all =  np.array([
np.sqrt(np.sum((bg2014 - at2014)**2)),
np.sqrt(np.sum((bg2014 - be2014)**2)),
np.sqrt(np.sum((bg2014 - bg2014)**2)),
np.sqrt(np.sum((bg2014 - by2014)**2)),
np.sqrt(np.sum((bg2014 - ch2014)**2)),
np.sqrt(np.sum((bg2014 - cz2014)**2)),
np.sqrt(np.sum((bg2014 - de2014)**2)),
np.sqrt(np.sum((bg2014 - dk2014)**2)),
np.sqrt(np.sum((bg2014 - ee2014)**2)),
np.sqrt(np.sum((bg2014 - es2014)**2)),
np.sqrt(np.sum((bg2014 - fi2014)**2)),
np.sqrt(np.sum((bg2014 - fr2014)**2)),
np.sqrt(np.sum((bg2014 - gb2014)**2)),
np.sqrt(np.sum((bg2014 - gr2014)**2)),
np.sqrt(np.sum((bg2014 - hr2014)**2)),
np.sqrt(np.sum((bg2014 - hu2014)**2)),
np.sqrt(np.sum((bg2014 - ie2014)**2)),
np.sqrt(np.sum((bg2014 - it2014)**2)),
np.sqrt(np.sum((bg2014 - nl2014)**2)),
np.sqrt(np.sum((bg2014 - no2014)**2)),
np.sqrt(np.sum((bg2014 - pl2014)**2)),
np.sqrt(np.sum((bg2014 - pt2014)**2)),
np.sqrt(np.sum((bg2014 - se2014)**2)),
np.sqrt(np.sum((bg2014 - si2014)**2)),
np.sqrt(np.sum((bg2014 - sk2014)**2)),
np.sqrt(np.sum((bg2014 - tr2014)**2)),
])
by2014_all =  np.array([
np.sqrt(np.sum((by2014 - at2014)**2)),
np.sqrt(np.sum((by2014 - be2014)**2)),
np.sqrt(np.sum((by2014 - bg2014)**2)),
np.sqrt(np.sum((by2014 - by2014)**2)),
np.sqrt(np.sum((by2014 - ch2014)**2)),
np.sqrt(np.sum((by2014 - cz2014)**2)),
np.sqrt(np.sum((by2014 - de2014)**2)),
np.sqrt(np.sum((by2014 - dk2014)**2)),
np.sqrt(np.sum((by2014 - ee2014)**2)),
np.sqrt(np.sum((by2014 - es2014)**2)),
np.sqrt(np.sum((by2014 - fi2014)**2)),
np.sqrt(np.sum((by2014 - fr2014)**2)),
np.sqrt(np.sum((by2014 - gb2014)**2)),
np.sqrt(np.sum((by2014 - gr2014)**2)),
np.sqrt(np.sum((by2014 - hr2014)**2)),
np.sqrt(np.sum((by2014 - hu2014)**2)),
np.sqrt(np.sum((by2014 - ie2014)**2)),
np.sqrt(np.sum((by2014 - it2014)**2)),
np.sqrt(np.sum((by2014 - nl2014)**2)),
np.sqrt(np.sum((by2014 - no2014)**2)),
np.sqrt(np.sum((by2014 - pl2014)**2)),
np.sqrt(np.sum((by2014 - pt2014)**2)),
np.sqrt(np.sum((by2014 - se2014)**2)),
np.sqrt(np.sum((by2014 - si2014)**2)),
np.sqrt(np.sum((by2014 - sk2014)**2)),
np.sqrt(np.sum((by2014 - tr2014)**2)),
])
ch2014_all =  np.array([
np.sqrt(np.sum((ch2014 - at2014)**2)),
np.sqrt(np.sum((ch2014 - be2014)**2)),
np.sqrt(np.sum((ch2014 - bg2014)**2)),
np.sqrt(np.sum((ch2014 - by2014)**2)),
np.sqrt(np.sum((ch2014 - ch2014)**2)),
np.sqrt(np.sum((ch2014 - cz2014)**2)),
np.sqrt(np.sum((ch2014 - de2014)**2)),
np.sqrt(np.sum((ch2014 - dk2014)**2)),
np.sqrt(np.sum((ch2014 - ee2014)**2)),
np.sqrt(np.sum((ch2014 - es2014)**2)),
np.sqrt(np.sum((ch2014 - fi2014)**2)),
np.sqrt(np.sum((ch2014 - fr2014)**2)),
np.sqrt(np.sum((ch2014 - gb2014)**2)),
np.sqrt(np.sum((ch2014 - gr2014)**2)),
np.sqrt(np.sum((ch2014 - hr2014)**2)),
np.sqrt(np.sum((ch2014 - hu2014)**2)),
np.sqrt(np.sum((ch2014 - ie2014)**2)),
np.sqrt(np.sum((ch2014 - it2014)**2)),
np.sqrt(np.sum((ch2014 - nl2014)**2)),
np.sqrt(np.sum((ch2014 - no2014)**2)),
np.sqrt(np.sum((ch2014 - pl2014)**2)),
np.sqrt(np.sum((ch2014 - pt2014)**2)),
np.sqrt(np.sum((ch2014 - se2014)**2)),
np.sqrt(np.sum((ch2014 - si2014)**2)),
np.sqrt(np.sum((ch2014 - sk2014)**2)),
np.sqrt(np.sum((ch2014 - tr2014)**2)),
])
cz2014_all =  np.array([
np.sqrt(np.sum((cz2014 - at2014)**2)),
np.sqrt(np.sum((cz2014 - be2014)**2)),
np.sqrt(np.sum((cz2014 - bg2014)**2)),
np.sqrt(np.sum((cz2014 - by2014)**2)),
np.sqrt(np.sum((cz2014 - ch2014)**2)),
np.sqrt(np.sum((cz2014 - cz2014)**2)),
np.sqrt(np.sum((cz2014 - de2014)**2)),
np.sqrt(np.sum((cz2014 - dk2014)**2)),
np.sqrt(np.sum((cz2014 - ee2014)**2)),
np.sqrt(np.sum((cz2014 - es2014)**2)),
np.sqrt(np.sum((cz2014 - fi2014)**2)),
np.sqrt(np.sum((cz2014 - fr2014)**2)),
np.sqrt(np.sum((cz2014 - gb2014)**2)),
np.sqrt(np.sum((cz2014 - gr2014)**2)),
np.sqrt(np.sum((cz2014 - hr2014)**2)),
np.sqrt(np.sum((cz2014 - hu2014)**2)),
np.sqrt(np.sum((cz2014 - ie2014)**2)),
np.sqrt(np.sum((cz2014 - it2014)**2)),
np.sqrt(np.sum((cz2014 - nl2014)**2)),
np.sqrt(np.sum((cz2014 - no2014)**2)),
np.sqrt(np.sum((cz2014 - pl2014)**2)),
np.sqrt(np.sum((cz2014 - pt2014)**2)),
np.sqrt(np.sum((cz2014 - se2014)**2)),
np.sqrt(np.sum((cz2014 - si2014)**2)),
np.sqrt(np.sum((cz2014 - sk2014)**2)),
np.sqrt(np.sum((cz2014 - tr2014)**2)),
])
de2014_all =  np.array([
np.sqrt(np.sum((de2014 - at2014)**2)),
np.sqrt(np.sum((de2014 - be2014)**2)),
np.sqrt(np.sum((de2014 - bg2014)**2)),
np.sqrt(np.sum((de2014 - by2014)**2)),
np.sqrt(np.sum((de2014 - ch2014)**2)),
np.sqrt(np.sum((de2014 - cz2014)**2)),
np.sqrt(np.sum((de2014 - de2014)**2)),
np.sqrt(np.sum((de2014 - dk2014)**2)),
np.sqrt(np.sum((de2014 - ee2014)**2)),
np.sqrt(np.sum((de2014 - es2014)**2)),
np.sqrt(np.sum((de2014 - fi2014)**2)),
np.sqrt(np.sum((de2014 - fr2014)**2)),
np.sqrt(np.sum((de2014 - gb2014)**2)),
np.sqrt(np.sum((de2014 - gr2014)**2)),
np.sqrt(np.sum((de2014 - hr2014)**2)),
np.sqrt(np.sum((de2014 - hu2014)**2)),
np.sqrt(np.sum((de2014 - ie2014)**2)),
np.sqrt(np.sum((de2014 - it2014)**2)),
np.sqrt(np.sum((de2014 - nl2014)**2)),
np.sqrt(np.sum((de2014 - no2014)**2)),
np.sqrt(np.sum((de2014 - pl2014)**2)),
np.sqrt(np.sum((de2014 - pt2014)**2)),
np.sqrt(np.sum((de2014 - se2014)**2)),
np.sqrt(np.sum((de2014 - si2014)**2)),
np.sqrt(np.sum((de2014 - sk2014)**2)),
np.sqrt(np.sum((de2014 - tr2014)**2)),
])
dk2014_all =  np.array([
np.sqrt(np.sum((dk2014 - at2014)**2)),
np.sqrt(np.sum((dk2014 - be2014)**2)),
np.sqrt(np.sum((dk2014 - bg2014)**2)),
np.sqrt(np.sum((dk2014 - by2014)**2)),
np.sqrt(np.sum((dk2014 - ch2014)**2)),
np.sqrt(np.sum((dk2014 - cz2014)**2)),
np.sqrt(np.sum((dk2014 - de2014)**2)),
np.sqrt(np.sum((dk2014 - dk2014)**2)),
np.sqrt(np.sum((dk2014 - ee2014)**2)),
np.sqrt(np.sum((dk2014 - es2014)**2)),
np.sqrt(np.sum((dk2014 - fi2014)**2)),
np.sqrt(np.sum((dk2014 - fr2014)**2)),
np.sqrt(np.sum((dk2014 - gb2014)**2)),
np.sqrt(np.sum((dk2014 - gr2014)**2)),
np.sqrt(np.sum((dk2014 - hr2014)**2)),
np.sqrt(np.sum((dk2014 - hu2014)**2)),
np.sqrt(np.sum((dk2014 - ie2014)**2)),
np.sqrt(np.sum((dk2014 - it2014)**2)),
np.sqrt(np.sum((dk2014 - nl2014)**2)),
np.sqrt(np.sum((dk2014 - no2014)**2)),
np.sqrt(np.sum((dk2014 - pl2014)**2)),
np.sqrt(np.sum((dk2014 - pt2014)**2)),
np.sqrt(np.sum((dk2014 - se2014)**2)),
np.sqrt(np.sum((dk2014 - si2014)**2)),
np.sqrt(np.sum((dk2014 - sk2014)**2)),
np.sqrt(np.sum((dk2014 - tr2014)**2)),
])
ee2014_all =  np.array([
np.sqrt(np.sum((ee2014 - at2014)**2)),
np.sqrt(np.sum((ee2014 - be2014)**2)),
np.sqrt(np.sum((ee2014 - bg2014)**2)),
np.sqrt(np.sum((ee2014 - by2014)**2)),
np.sqrt(np.sum((ee2014 - ch2014)**2)),
np.sqrt(np.sum((ee2014 - cz2014)**2)),
np.sqrt(np.sum((ee2014 - de2014)**2)),
np.sqrt(np.sum((ee2014 - dk2014)**2)),
np.sqrt(np.sum((ee2014 - ee2014)**2)),
np.sqrt(np.sum((ee2014 - es2014)**2)),
np.sqrt(np.sum((ee2014 - fi2014)**2)),
np.sqrt(np.sum((ee2014 - fr2014)**2)),
np.sqrt(np.sum((ee2014 - gb2014)**2)),
np.sqrt(np.sum((ee2014 - gr2014)**2)),
np.sqrt(np.sum((ee2014 - hr2014)**2)),
np.sqrt(np.sum((ee2014 - hu2014)**2)),
np.sqrt(np.sum((ee2014 - ie2014)**2)),
np.sqrt(np.sum((ee2014 - it2014)**2)),
np.sqrt(np.sum((ee2014 - nl2014)**2)),
np.sqrt(np.sum((ee2014 - no2014)**2)),
np.sqrt(np.sum((ee2014 - pl2014)**2)),
np.sqrt(np.sum((ee2014 - pt2014)**2)),
np.sqrt(np.sum((ee2014 - se2014)**2)),
np.sqrt(np.sum((ee2014 - si2014)**2)),
np.sqrt(np.sum((ee2014 - sk2014)**2)),
np.sqrt(np.sum((ee2014 - tr2014)**2)),
])
es2014_all =  np.array([
np.sqrt(np.sum((es2014 - at2014)**2)),
np.sqrt(np.sum((es2014 - be2014)**2)),
np.sqrt(np.sum((es2014 - bg2014)**2)),
np.sqrt(np.sum((es2014 - by2014)**2)),
np.sqrt(np.sum((es2014 - ch2014)**2)),
np.sqrt(np.sum((es2014 - cz2014)**2)),
np.sqrt(np.sum((es2014 - de2014)**2)),
np.sqrt(np.sum((es2014 - dk2014)**2)),
np.sqrt(np.sum((es2014 - ee2014)**2)),
np.sqrt(np.sum((es2014 - es2014)**2)),
np.sqrt(np.sum((es2014 - fi2014)**2)),
np.sqrt(np.sum((es2014 - fr2014)**2)),
np.sqrt(np.sum((es2014 - gb2014)**2)),
np.sqrt(np.sum((es2014 - gr2014)**2)),
np.sqrt(np.sum((es2014 - hr2014)**2)),
np.sqrt(np.sum((es2014 - hu2014)**2)),
np.sqrt(np.sum((es2014 - ie2014)**2)),
np.sqrt(np.sum((es2014 - it2014)**2)),
np.sqrt(np.sum((es2014 - nl2014)**2)),
np.sqrt(np.sum((es2014 - no2014)**2)),
np.sqrt(np.sum((es2014 - pl2014)**2)),
np.sqrt(np.sum((es2014 - pt2014)**2)),
np.sqrt(np.sum((es2014 - se2014)**2)),
np.sqrt(np.sum((es2014 - si2014)**2)),
np.sqrt(np.sum((es2014 - sk2014)**2)),
np.sqrt(np.sum((es2014 - tr2014)**2)),
])
fi2014_all =  np.array([
np.sqrt(np.sum((fi2014 - at2014)**2)),
np.sqrt(np.sum((fi2014 - be2014)**2)),
np.sqrt(np.sum((fi2014 - bg2014)**2)),
np.sqrt(np.sum((fi2014 - by2014)**2)),
np.sqrt(np.sum((fi2014 - ch2014)**2)),
np.sqrt(np.sum((fi2014 - cz2014)**2)),
np.sqrt(np.sum((fi2014 - de2014)**2)),
np.sqrt(np.sum((fi2014 - dk2014)**2)),
np.sqrt(np.sum((fi2014 - ee2014)**2)),
np.sqrt(np.sum((fi2014 - es2014)**2)),
np.sqrt(np.sum((fi2014 - fi2014)**2)),
np.sqrt(np.sum((fi2014 - fr2014)**2)),
np.sqrt(np.sum((fi2014 - gb2014)**2)),
np.sqrt(np.sum((fi2014 - gr2014)**2)),
np.sqrt(np.sum((fi2014 - hr2014)**2)),
np.sqrt(np.sum((fi2014 - hu2014)**2)),
np.sqrt(np.sum((fi2014 - ie2014)**2)),
np.sqrt(np.sum((fi2014 - it2014)**2)),
np.sqrt(np.sum((fi2014 - nl2014)**2)),
np.sqrt(np.sum((fi2014 - no2014)**2)),
np.sqrt(np.sum((fi2014 - pl2014)**2)),
np.sqrt(np.sum((fi2014 - pt2014)**2)),
np.sqrt(np.sum((fi2014 - se2014)**2)),
np.sqrt(np.sum((fi2014 - si2014)**2)),
np.sqrt(np.sum((fi2014 - sk2014)**2)),
np.sqrt(np.sum((fi2014 - tr2014)**2)),
])
fr2014_all =  np.array([
np.sqrt(np.sum((fr2014 - at2014)**2)),
np.sqrt(np.sum((fr2014 - be2014)**2)),
np.sqrt(np.sum((fr2014 - bg2014)**2)),
np.sqrt(np.sum((fr2014 - by2014)**2)),
np.sqrt(np.sum((fr2014 - ch2014)**2)),
np.sqrt(np.sum((fr2014 - cz2014)**2)),
np.sqrt(np.sum((fr2014 - de2014)**2)),
np.sqrt(np.sum((fr2014 - dk2014)**2)),
np.sqrt(np.sum((fr2014 - ee2014)**2)),
np.sqrt(np.sum((fr2014 - es2014)**2)),
np.sqrt(np.sum((fr2014 - fi2014)**2)),
np.sqrt(np.sum((fr2014 - fr2014)**2)),
np.sqrt(np.sum((fr2014 - gb2014)**2)),
np.sqrt(np.sum((fr2014 - gr2014)**2)),
np.sqrt(np.sum((fr2014 - hr2014)**2)),
np.sqrt(np.sum((fr2014 - hu2014)**2)),
np.sqrt(np.sum((fr2014 - ie2014)**2)),
np.sqrt(np.sum((fr2014 - it2014)**2)),
np.sqrt(np.sum((fr2014 - nl2014)**2)),
np.sqrt(np.sum((fr2014 - no2014)**2)),
np.sqrt(np.sum((fr2014 - pl2014)**2)),
np.sqrt(np.sum((fr2014 - pt2014)**2)),
np.sqrt(np.sum((fr2014 - se2014)**2)),
np.sqrt(np.sum((fr2014 - si2014)**2)),
np.sqrt(np.sum((fr2014 - sk2014)**2)),
np.sqrt(np.sum((fr2014 - tr2014)**2)),
])
gb2014_all =  np.array([
np.sqrt(np.sum((gb2014 - at2014)**2)),
np.sqrt(np.sum((gb2014 - be2014)**2)),
np.sqrt(np.sum((gb2014 - bg2014)**2)),
np.sqrt(np.sum((gb2014 - by2014)**2)),
np.sqrt(np.sum((gb2014 - ch2014)**2)),
np.sqrt(np.sum((gb2014 - cz2014)**2)),
np.sqrt(np.sum((gb2014 - de2014)**2)),
np.sqrt(np.sum((gb2014 - dk2014)**2)),
np.sqrt(np.sum((gb2014 - ee2014)**2)),
np.sqrt(np.sum((gb2014 - es2014)**2)),
np.sqrt(np.sum((gb2014 - fi2014)**2)),
np.sqrt(np.sum((gb2014 - fr2014)**2)),
np.sqrt(np.sum((gb2014 - gb2014)**2)),
np.sqrt(np.sum((gb2014 - gr2014)**2)),
np.sqrt(np.sum((gb2014 - hr2014)**2)),
np.sqrt(np.sum((gb2014 - hu2014)**2)),
np.sqrt(np.sum((gb2014 - ie2014)**2)),
np.sqrt(np.sum((gb2014 - it2014)**2)),
np.sqrt(np.sum((gb2014 - nl2014)**2)),
np.sqrt(np.sum((gb2014 - no2014)**2)),
np.sqrt(np.sum((gb2014 - pl2014)**2)),
np.sqrt(np.sum((gb2014 - pt2014)**2)),
np.sqrt(np.sum((gb2014 - se2014)**2)),
np.sqrt(np.sum((gb2014 - si2014)**2)),
np.sqrt(np.sum((gb2014 - sk2014)**2)),
np.sqrt(np.sum((gb2014 - tr2014)**2)),
])
gr2014_all =  np.array([
np.sqrt(np.sum((gr2014 - at2014)**2)),
np.sqrt(np.sum((gr2014 - be2014)**2)),
np.sqrt(np.sum((gr2014 - bg2014)**2)),
np.sqrt(np.sum((gr2014 - by2014)**2)),
np.sqrt(np.sum((gr2014 - ch2014)**2)),
np.sqrt(np.sum((gr2014 - cz2014)**2)),
np.sqrt(np.sum((gr2014 - de2014)**2)),
np.sqrt(np.sum((gr2014 - dk2014)**2)),
np.sqrt(np.sum((gr2014 - ee2014)**2)),
np.sqrt(np.sum((gr2014 - es2014)**2)),
np.sqrt(np.sum((gr2014 - fi2014)**2)),
np.sqrt(np.sum((gr2014 - fr2014)**2)),
np.sqrt(np.sum((gr2014 - gb2014)**2)),
np.sqrt(np.sum((gr2014 - gr2014)**2)),
np.sqrt(np.sum((gr2014 - hr2014)**2)),
np.sqrt(np.sum((gr2014 - hu2014)**2)),
np.sqrt(np.sum((gr2014 - ie2014)**2)),
np.sqrt(np.sum((gr2014 - it2014)**2)),
np.sqrt(np.sum((gr2014 - nl2014)**2)),
np.sqrt(np.sum((gr2014 - no2014)**2)),
np.sqrt(np.sum((gr2014 - pl2014)**2)),
np.sqrt(np.sum((gr2014 - pt2014)**2)),
np.sqrt(np.sum((gr2014 - se2014)**2)),
np.sqrt(np.sum((gr2014 - si2014)**2)),
np.sqrt(np.sum((gr2014 - sk2014)**2)),
np.sqrt(np.sum((gr2014 - tr2014)**2)),
])
hr2014_all =  np.array([
np.sqrt(np.sum((hr2014 - at2014)**2)),
np.sqrt(np.sum((hr2014 - be2014)**2)),
np.sqrt(np.sum((hr2014 - bg2014)**2)),
np.sqrt(np.sum((hr2014 - by2014)**2)),
np.sqrt(np.sum((hr2014 - ch2014)**2)),
np.sqrt(np.sum((hr2014 - cz2014)**2)),
np.sqrt(np.sum((hr2014 - de2014)**2)),
np.sqrt(np.sum((hr2014 - dk2014)**2)),
np.sqrt(np.sum((hr2014 - ee2014)**2)),
np.sqrt(np.sum((hr2014 - es2014)**2)),
np.sqrt(np.sum((hr2014 - fi2014)**2)),
np.sqrt(np.sum((hr2014 - fr2014)**2)),
np.sqrt(np.sum((hr2014 - gb2014)**2)),
np.sqrt(np.sum((hr2014 - gr2014)**2)),
np.sqrt(np.sum((hr2014 - hr2014)**2)),
np.sqrt(np.sum((hr2014 - hu2014)**2)),
np.sqrt(np.sum((hr2014 - ie2014)**2)),
np.sqrt(np.sum((hr2014 - it2014)**2)),
np.sqrt(np.sum((hr2014 - nl2014)**2)),
np.sqrt(np.sum((hr2014 - no2014)**2)),
np.sqrt(np.sum((hr2014 - pl2014)**2)),
np.sqrt(np.sum((hr2014 - pt2014)**2)),
np.sqrt(np.sum((hr2014 - se2014)**2)),
np.sqrt(np.sum((hr2014 - si2014)**2)),
np.sqrt(np.sum((hr2014 - sk2014)**2)),
np.sqrt(np.sum((hr2014 - tr2014)**2)),
])
hu2014_all =  np.array([
np.sqrt(np.sum((hu2014 - at2014)**2)),
np.sqrt(np.sum((hu2014 - be2014)**2)),
np.sqrt(np.sum((hu2014 - bg2014)**2)),
np.sqrt(np.sum((hu2014 - by2014)**2)),
np.sqrt(np.sum((hu2014 - ch2014)**2)),
np.sqrt(np.sum((hu2014 - cz2014)**2)),
np.sqrt(np.sum((hu2014 - de2014)**2)),
np.sqrt(np.sum((hu2014 - dk2014)**2)),
np.sqrt(np.sum((hu2014 - ee2014)**2)),
np.sqrt(np.sum((hu2014 - es2014)**2)),
np.sqrt(np.sum((hu2014 - fi2014)**2)),
np.sqrt(np.sum((hu2014 - fr2014)**2)),
np.sqrt(np.sum((hu2014 - gb2014)**2)),
np.sqrt(np.sum((hu2014 - gr2014)**2)),
np.sqrt(np.sum((hu2014 - hr2014)**2)),
np.sqrt(np.sum((hu2014 - hu2014)**2)),
np.sqrt(np.sum((hu2014 - ie2014)**2)),
np.sqrt(np.sum((hu2014 - it2014)**2)),
np.sqrt(np.sum((hu2014 - nl2014)**2)),
np.sqrt(np.sum((hu2014 - no2014)**2)),
np.sqrt(np.sum((hu2014 - pl2014)**2)),
np.sqrt(np.sum((hu2014 - pt2014)**2)),
np.sqrt(np.sum((hu2014 - se2014)**2)),
np.sqrt(np.sum((hu2014 - si2014)**2)),
np.sqrt(np.sum((hu2014 - sk2014)**2)),
np.sqrt(np.sum((hu2014 - tr2014)**2)),
])
ie2014_all =  np.array([
np.sqrt(np.sum((ie2014 - at2014)**2)),
np.sqrt(np.sum((ie2014 - be2014)**2)),
np.sqrt(np.sum((ie2014 - bg2014)**2)),
np.sqrt(np.sum((ie2014 - by2014)**2)),
np.sqrt(np.sum((ie2014 - ch2014)**2)),
np.sqrt(np.sum((ie2014 - cz2014)**2)),
np.sqrt(np.sum((ie2014 - de2014)**2)),
np.sqrt(np.sum((ie2014 - dk2014)**2)),
np.sqrt(np.sum((ie2014 - ee2014)**2)),
np.sqrt(np.sum((ie2014 - es2014)**2)),
np.sqrt(np.sum((ie2014 - fi2014)**2)),
np.sqrt(np.sum((ie2014 - fr2014)**2)),
np.sqrt(np.sum((ie2014 - gb2014)**2)),
np.sqrt(np.sum((ie2014 - gr2014)**2)),
np.sqrt(np.sum((ie2014 - hr2014)**2)),
np.sqrt(np.sum((ie2014 - hu2014)**2)),
np.sqrt(np.sum((ie2014 - ie2014)**2)),
np.sqrt(np.sum((ie2014 - it2014)**2)),
np.sqrt(np.sum((ie2014 - nl2014)**2)),
np.sqrt(np.sum((ie2014 - no2014)**2)),
np.sqrt(np.sum((ie2014 - pl2014)**2)),
np.sqrt(np.sum((ie2014 - pt2014)**2)),
np.sqrt(np.sum((ie2014 - se2014)**2)),
np.sqrt(np.sum((ie2014 - si2014)**2)),
np.sqrt(np.sum((ie2014 - sk2014)**2)),
np.sqrt(np.sum((ie2014 - tr2014)**2)),
])
it2014_all =  np.array([
np.sqrt(np.sum((it2014 - at2014)**2)),
np.sqrt(np.sum((it2014 - be2014)**2)),
np.sqrt(np.sum((it2014 - bg2014)**2)),
np.sqrt(np.sum((it2014 - by2014)**2)),
np.sqrt(np.sum((it2014 - ch2014)**2)),
np.sqrt(np.sum((it2014 - cz2014)**2)),
np.sqrt(np.sum((it2014 - de2014)**2)),
np.sqrt(np.sum((it2014 - dk2014)**2)),
np.sqrt(np.sum((it2014 - ee2014)**2)),
np.sqrt(np.sum((it2014 - es2014)**2)),
np.sqrt(np.sum((it2014 - fi2014)**2)),
np.sqrt(np.sum((it2014 - fr2014)**2)),
np.sqrt(np.sum((it2014 - gb2014)**2)),
np.sqrt(np.sum((it2014 - gr2014)**2)),
np.sqrt(np.sum((it2014 - hr2014)**2)),
np.sqrt(np.sum((it2014 - hu2014)**2)),
np.sqrt(np.sum((it2014 - ie2014)**2)),
np.sqrt(np.sum((it2014 - it2014)**2)),
np.sqrt(np.sum((it2014 - nl2014)**2)),
np.sqrt(np.sum((it2014 - no2014)**2)),
np.sqrt(np.sum((it2014 - pl2014)**2)),
np.sqrt(np.sum((it2014 - pt2014)**2)),
np.sqrt(np.sum((it2014 - se2014)**2)),
np.sqrt(np.sum((it2014 - si2014)**2)),
np.sqrt(np.sum((it2014 - sk2014)**2)),
np.sqrt(np.sum((it2014 - tr2014)**2)),
])
nl2014_all =  np.array([
np.sqrt(np.sum((nl2014 - at2014)**2)),
np.sqrt(np.sum((nl2014 - be2014)**2)),
np.sqrt(np.sum((nl2014 - bg2014)**2)),
np.sqrt(np.sum((nl2014 - by2014)**2)),
np.sqrt(np.sum((nl2014 - ch2014)**2)),
np.sqrt(np.sum((nl2014 - cz2014)**2)),
np.sqrt(np.sum((nl2014 - de2014)**2)),
np.sqrt(np.sum((nl2014 - dk2014)**2)),
np.sqrt(np.sum((nl2014 - ee2014)**2)),
np.sqrt(np.sum((nl2014 - es2014)**2)),
np.sqrt(np.sum((nl2014 - fi2014)**2)),
np.sqrt(np.sum((nl2014 - fr2014)**2)),
np.sqrt(np.sum((nl2014 - gb2014)**2)),
np.sqrt(np.sum((nl2014 - gr2014)**2)),
np.sqrt(np.sum((nl2014 - hr2014)**2)),
np.sqrt(np.sum((nl2014 - hu2014)**2)),
np.sqrt(np.sum((nl2014 - ie2014)**2)),
np.sqrt(np.sum((nl2014 - it2014)**2)),
np.sqrt(np.sum((nl2014 - nl2014)**2)),
np.sqrt(np.sum((nl2014 - no2014)**2)),
np.sqrt(np.sum((nl2014 - pl2014)**2)),
np.sqrt(np.sum((nl2014 - pt2014)**2)),
np.sqrt(np.sum((nl2014 - se2014)**2)),
np.sqrt(np.sum((nl2014 - si2014)**2)),
np.sqrt(np.sum((nl2014 - sk2014)**2)),
np.sqrt(np.sum((nl2014 - tr2014)**2)),
])
no2014_all =  np.array([
np.sqrt(np.sum((no2014 - at2014)**2)),
np.sqrt(np.sum((no2014 - be2014)**2)),
np.sqrt(np.sum((no2014 - bg2014)**2)),
np.sqrt(np.sum((no2014 - by2014)**2)),
np.sqrt(np.sum((no2014 - ch2014)**2)),
np.sqrt(np.sum((no2014 - cz2014)**2)),
np.sqrt(np.sum((no2014 - de2014)**2)),
np.sqrt(np.sum((no2014 - dk2014)**2)),
np.sqrt(np.sum((no2014 - ee2014)**2)),
np.sqrt(np.sum((no2014 - es2014)**2)),
np.sqrt(np.sum((no2014 - fi2014)**2)),
np.sqrt(np.sum((no2014 - fr2014)**2)),
np.sqrt(np.sum((no2014 - gb2014)**2)),
np.sqrt(np.sum((no2014 - gr2014)**2)),
np.sqrt(np.sum((no2014 - hr2014)**2)),
np.sqrt(np.sum((no2014 - hu2014)**2)),
np.sqrt(np.sum((no2014 - ie2014)**2)),
np.sqrt(np.sum((no2014 - it2014)**2)),
np.sqrt(np.sum((no2014 - nl2014)**2)),
np.sqrt(np.sum((no2014 - no2014)**2)),
np.sqrt(np.sum((no2014 - pl2014)**2)),
np.sqrt(np.sum((no2014 - pt2014)**2)),
np.sqrt(np.sum((no2014 - se2014)**2)),
np.sqrt(np.sum((no2014 - si2014)**2)),
np.sqrt(np.sum((no2014 - sk2014)**2)),
np.sqrt(np.sum((no2014 - tr2014)**2)),
])
pl2014_all =  np.array([
np.sqrt(np.sum((pl2014 - at2014)**2)),
np.sqrt(np.sum((pl2014 - be2014)**2)),
np.sqrt(np.sum((pl2014 - bg2014)**2)),
np.sqrt(np.sum((pl2014 - by2014)**2)),
np.sqrt(np.sum((pl2014 - ch2014)**2)),
np.sqrt(np.sum((pl2014 - cz2014)**2)),
np.sqrt(np.sum((pl2014 - de2014)**2)),
np.sqrt(np.sum((pl2014 - dk2014)**2)),
np.sqrt(np.sum((pl2014 - ee2014)**2)),
np.sqrt(np.sum((pl2014 - es2014)**2)),
np.sqrt(np.sum((pl2014 - fi2014)**2)),
np.sqrt(np.sum((pl2014 - fr2014)**2)),
np.sqrt(np.sum((pl2014 - gb2014)**2)),
np.sqrt(np.sum((pl2014 - gr2014)**2)),
np.sqrt(np.sum((pl2014 - hr2014)**2)),
np.sqrt(np.sum((pl2014 - hu2014)**2)),
np.sqrt(np.sum((pl2014 - ie2014)**2)),
np.sqrt(np.sum((pl2014 - it2014)**2)),
np.sqrt(np.sum((pl2014 - nl2014)**2)),
np.sqrt(np.sum((pl2014 - no2014)**2)),
np.sqrt(np.sum((pl2014 - pl2014)**2)),
np.sqrt(np.sum((pl2014 - pt2014)**2)),
np.sqrt(np.sum((pl2014 - se2014)**2)),
np.sqrt(np.sum((pl2014 - si2014)**2)),
np.sqrt(np.sum((pl2014 - sk2014)**2)),
np.sqrt(np.sum((pl2014 - tr2014)**2)),
])
pt2014_all =  np.array([
np.sqrt(np.sum((pt2014 - at2014)**2)),
np.sqrt(np.sum((pt2014 - be2014)**2)),
np.sqrt(np.sum((pt2014 - bg2014)**2)),
np.sqrt(np.sum((pt2014 - by2014)**2)),
np.sqrt(np.sum((pt2014 - ch2014)**2)),
np.sqrt(np.sum((pt2014 - cz2014)**2)),
np.sqrt(np.sum((pt2014 - de2014)**2)),
np.sqrt(np.sum((pt2014 - dk2014)**2)),
np.sqrt(np.sum((pt2014 - ee2014)**2)),
np.sqrt(np.sum((pt2014 - es2014)**2)),
np.sqrt(np.sum((pt2014 - fi2014)**2)),
np.sqrt(np.sum((pt2014 - fr2014)**2)),
np.sqrt(np.sum((pt2014 - gb2014)**2)),
np.sqrt(np.sum((pt2014 - gr2014)**2)),
np.sqrt(np.sum((pt2014 - hr2014)**2)),
np.sqrt(np.sum((pt2014 - hu2014)**2)),
np.sqrt(np.sum((pt2014 - ie2014)**2)),
np.sqrt(np.sum((pt2014 - it2014)**2)),
np.sqrt(np.sum((pt2014 - nl2014)**2)),
np.sqrt(np.sum((pt2014 - no2014)**2)),
np.sqrt(np.sum((pt2014 - pl2014)**2)),
np.sqrt(np.sum((pt2014 - pt2014)**2)),
np.sqrt(np.sum((pt2014 - se2014)**2)),
np.sqrt(np.sum((pt2014 - si2014)**2)),
np.sqrt(np.sum((pt2014 - sk2014)**2)),
np.sqrt(np.sum((pt2014 - tr2014)**2)),
])
se2014_all =  np.array([
np.sqrt(np.sum((se2014 - at2014)**2)),
np.sqrt(np.sum((se2014 - be2014)**2)),
np.sqrt(np.sum((se2014 - bg2014)**2)),
np.sqrt(np.sum((se2014 - by2014)**2)),
np.sqrt(np.sum((se2014 - ch2014)**2)),
np.sqrt(np.sum((se2014 - cz2014)**2)),
np.sqrt(np.sum((se2014 - de2014)**2)),
np.sqrt(np.sum((se2014 - dk2014)**2)),
np.sqrt(np.sum((se2014 - ee2014)**2)),
np.sqrt(np.sum((se2014 - es2014)**2)),
np.sqrt(np.sum((se2014 - fi2014)**2)),
np.sqrt(np.sum((se2014 - fr2014)**2)),
np.sqrt(np.sum((se2014 - gb2014)**2)),
np.sqrt(np.sum((se2014 - gr2014)**2)),
np.sqrt(np.sum((se2014 - hr2014)**2)),
np.sqrt(np.sum((se2014 - hu2014)**2)),
np.sqrt(np.sum((se2014 - ie2014)**2)),
np.sqrt(np.sum((se2014 - it2014)**2)),
np.sqrt(np.sum((se2014 - nl2014)**2)),
np.sqrt(np.sum((se2014 - no2014)**2)),
np.sqrt(np.sum((se2014 - pl2014)**2)),
np.sqrt(np.sum((se2014 - pt2014)**2)),
np.sqrt(np.sum((se2014 - se2014)**2)),
np.sqrt(np.sum((se2014 - si2014)**2)),
np.sqrt(np.sum((se2014 - sk2014)**2)),
np.sqrt(np.sum((se2014 - tr2014)**2)),
])
si2014_all =  np.array([
np.sqrt(np.sum((si2014 - at2014)**2)),
np.sqrt(np.sum((si2014 - be2014)**2)),
np.sqrt(np.sum((si2014 - bg2014)**2)),
np.sqrt(np.sum((si2014 - by2014)**2)),
np.sqrt(np.sum((si2014 - ch2014)**2)),
np.sqrt(np.sum((si2014 - cz2014)**2)),
np.sqrt(np.sum((si2014 - de2014)**2)),
np.sqrt(np.sum((si2014 - dk2014)**2)),
np.sqrt(np.sum((si2014 - ee2014)**2)),
np.sqrt(np.sum((si2014 - es2014)**2)),
np.sqrt(np.sum((si2014 - fi2014)**2)),
np.sqrt(np.sum((si2014 - fr2014)**2)),
np.sqrt(np.sum((si2014 - gb2014)**2)),
np.sqrt(np.sum((si2014 - gr2014)**2)),
np.sqrt(np.sum((si2014 - hr2014)**2)),
np.sqrt(np.sum((si2014 - hu2014)**2)),
np.sqrt(np.sum((si2014 - ie2014)**2)),
np.sqrt(np.sum((si2014 - it2014)**2)),
np.sqrt(np.sum((si2014 - nl2014)**2)),
np.sqrt(np.sum((si2014 - no2014)**2)),
np.sqrt(np.sum((si2014 - pl2014)**2)),
np.sqrt(np.sum((si2014 - pt2014)**2)),
np.sqrt(np.sum((si2014 - se2014)**2)),
np.sqrt(np.sum((si2014 - si2014)**2)),
np.sqrt(np.sum((si2014 - sk2014)**2)),
np.sqrt(np.sum((si2014 - tr2014)**2)),
])
sk2014_all =  np.array([
np.sqrt(np.sum((sk2014 - at2014)**2)),
np.sqrt(np.sum((sk2014 - be2014)**2)),
np.sqrt(np.sum((sk2014 - bg2014)**2)),
np.sqrt(np.sum((sk2014 - by2014)**2)),
np.sqrt(np.sum((sk2014 - ch2014)**2)),
np.sqrt(np.sum((sk2014 - cz2014)**2)),
np.sqrt(np.sum((sk2014 - de2014)**2)),
np.sqrt(np.sum((sk2014 - dk2014)**2)),
np.sqrt(np.sum((sk2014 - ee2014)**2)),
np.sqrt(np.sum((sk2014 - es2014)**2)),
np.sqrt(np.sum((sk2014 - fi2014)**2)),
np.sqrt(np.sum((sk2014 - fr2014)**2)),
np.sqrt(np.sum((sk2014 - gb2014)**2)),
np.sqrt(np.sum((sk2014 - gr2014)**2)),
np.sqrt(np.sum((sk2014 - hr2014)**2)),
np.sqrt(np.sum((sk2014 - hu2014)**2)),
np.sqrt(np.sum((sk2014 - ie2014)**2)),
np.sqrt(np.sum((sk2014 - it2014)**2)),
np.sqrt(np.sum((sk2014 - nl2014)**2)),
np.sqrt(np.sum((sk2014 - no2014)**2)),
np.sqrt(np.sum((sk2014 - pl2014)**2)),
np.sqrt(np.sum((sk2014 - pt2014)**2)),
np.sqrt(np.sum((sk2014 - se2014)**2)),
np.sqrt(np.sum((sk2014 - si2014)**2)),
np.sqrt(np.sum((sk2014 - sk2014)**2)),
np.sqrt(np.sum((sk2014 - tr2014)**2)),
])
tr2014_all =  np.array([
np.sqrt(np.sum((tr2014 - at2014)**2)),
np.sqrt(np.sum((tr2014 - be2014)**2)),
np.sqrt(np.sum((tr2014 - bg2014)**2)),
np.sqrt(np.sum((tr2014 - by2014)**2)),
np.sqrt(np.sum((tr2014 - ch2014)**2)),
np.sqrt(np.sum((tr2014 - cz2014)**2)),
np.sqrt(np.sum((tr2014 - de2014)**2)),
np.sqrt(np.sum((tr2014 - dk2014)**2)),
np.sqrt(np.sum((tr2014 - ee2014)**2)),
np.sqrt(np.sum((tr2014 - es2014)**2)),
np.sqrt(np.sum((tr2014 - fi2014)**2)),
np.sqrt(np.sum((tr2014 - fr2014)**2)),
np.sqrt(np.sum((tr2014 - gb2014)**2)),
np.sqrt(np.sum((tr2014 - gr2014)**2)),
np.sqrt(np.sum((tr2014 - hr2014)**2)),
np.sqrt(np.sum((tr2014 - hu2014)**2)),
np.sqrt(np.sum((tr2014 - ie2014)**2)),
np.sqrt(np.sum((tr2014 - it2014)**2)),
np.sqrt(np.sum((tr2014 - nl2014)**2)),
np.sqrt(np.sum((tr2014 - no2014)**2)),
np.sqrt(np.sum((tr2014 - pl2014)**2)),
np.sqrt(np.sum((tr2014 - pt2014)**2)),
np.sqrt(np.sum((tr2014 - se2014)**2)),
np.sqrt(np.sum((tr2014 - si2014)**2)),
np.sqrt(np.sum((tr2014 - sk2014)**2)),
np.sqrt(np.sum((tr2014 - tr2014)**2)),
])

In [ ]:
#2015
at2015_all =  np.array([
np.sqrt(np.sum((at2015 - at2015)**2)),
np.sqrt(np.sum((at2015 - be2015)**2)),
np.sqrt(np.sum((at2015 - bg2015)**2)),
np.sqrt(np.sum((at2015 - by2015)**2)),
np.sqrt(np.sum((at2015 - ch2015)**2)),
np.sqrt(np.sum((at2015 - cz2015)**2)),
np.sqrt(np.sum((at2015 - de2015)**2)),
np.sqrt(np.sum((at2015 - dk2015)**2)),
np.sqrt(np.sum((at2015 - ee2015)**2)),
np.sqrt(np.sum((at2015 - es2015)**2)),
np.sqrt(np.sum((at2015 - fi2015)**2)),
np.sqrt(np.sum((at2015 - fr2015)**2)),
np.sqrt(np.sum((at2015 - gb2015)**2)),
np.sqrt(np.sum((at2015 - gr2015)**2)),
np.sqrt(np.sum((at2015 - hr2015)**2)),
np.sqrt(np.sum((at2015 - hu2015)**2)),
np.sqrt(np.sum((at2015 - ie2015)**2)),
np.sqrt(np.sum((at2015 - it2015)**2)),
np.sqrt(np.sum((at2015 - nl2015)**2)),
np.sqrt(np.sum((at2015 - no2015)**2)),
np.sqrt(np.sum((at2015 - pl2015)**2)),
np.sqrt(np.sum((at2015 - pt2015)**2)),
np.sqrt(np.sum((at2015 - se2015)**2)),
np.sqrt(np.sum((at2015 - si2015)**2)),
np.sqrt(np.sum((at2015 - sk2015)**2)),
np.sqrt(np.sum((at2015 - tr2015)**2)),
])
be2015_all =  np.array([
np.sqrt(np.sum((be2015 - at2015)**2)),
np.sqrt(np.sum((be2015 - be2015)**2)),
np.sqrt(np.sum((be2015 - bg2015)**2)),
np.sqrt(np.sum((be2015 - by2015)**2)),
np.sqrt(np.sum((be2015 - ch2015)**2)),
np.sqrt(np.sum((be2015 - cz2015)**2)),
np.sqrt(np.sum((be2015 - de2015)**2)),
np.sqrt(np.sum((be2015 - dk2015)**2)),
np.sqrt(np.sum((be2015 - ee2015)**2)),
np.sqrt(np.sum((be2015 - es2015)**2)),
np.sqrt(np.sum((be2015 - fi2015)**2)),
np.sqrt(np.sum((be2015 - fr2015)**2)),
np.sqrt(np.sum((be2015 - gb2015)**2)),
np.sqrt(np.sum((be2015 - gr2015)**2)),
np.sqrt(np.sum((be2015 - hr2015)**2)),
np.sqrt(np.sum((be2015 - hu2015)**2)),
np.sqrt(np.sum((be2015 - ie2015)**2)),
np.sqrt(np.sum((be2015 - it2015)**2)),
np.sqrt(np.sum((be2015 - nl2015)**2)),
np.sqrt(np.sum((be2015 - no2015)**2)),
np.sqrt(np.sum((be2015 - pl2015)**2)),
np.sqrt(np.sum((be2015 - pt2015)**2)),
np.sqrt(np.sum((be2015 - se2015)**2)),
np.sqrt(np.sum((be2015 - si2015)**2)),
np.sqrt(np.sum((be2015 - sk2015)**2)),
np.sqrt(np.sum((be2015 - tr2015)**2)),
])
bg2015_all =  np.array([
np.sqrt(np.sum((bg2015 - at2015)**2)),
np.sqrt(np.sum((bg2015 - be2015)**2)),
np.sqrt(np.sum((bg2015 - bg2015)**2)),
np.sqrt(np.sum((bg2015 - by2015)**2)),
np.sqrt(np.sum((bg2015 - ch2015)**2)),
np.sqrt(np.sum((bg2015 - cz2015)**2)),
np.sqrt(np.sum((bg2015 - de2015)**2)),
np.sqrt(np.sum((bg2015 - dk2015)**2)),
np.sqrt(np.sum((bg2015 - ee2015)**2)),
np.sqrt(np.sum((bg2015 - es2015)**2)),
np.sqrt(np.sum((bg2015 - fi2015)**2)),
np.sqrt(np.sum((bg2015 - fr2015)**2)),
np.sqrt(np.sum((bg2015 - gb2015)**2)),
np.sqrt(np.sum((bg2015 - gr2015)**2)),
np.sqrt(np.sum((bg2015 - hr2015)**2)),
np.sqrt(np.sum((bg2015 - hu2015)**2)),
np.sqrt(np.sum((bg2015 - ie2015)**2)),
np.sqrt(np.sum((bg2015 - it2015)**2)),
np.sqrt(np.sum((bg2015 - nl2015)**2)),
np.sqrt(np.sum((bg2015 - no2015)**2)),
np.sqrt(np.sum((bg2015 - pl2015)**2)),
np.sqrt(np.sum((bg2015 - pt2015)**2)),
np.sqrt(np.sum((bg2015 - se2015)**2)),
np.sqrt(np.sum((bg2015 - si2015)**2)),
np.sqrt(np.sum((bg2015 - sk2015)**2)),
np.sqrt(np.sum((bg2015 - tr2015)**2)),
])
by2015_all =  np.array([
np.sqrt(np.sum((by2015 - at2015)**2)),
np.sqrt(np.sum((by2015 - be2015)**2)),
np.sqrt(np.sum((by2015 - bg2015)**2)),
np.sqrt(np.sum((by2015 - by2015)**2)),
np.sqrt(np.sum((by2015 - ch2015)**2)),
np.sqrt(np.sum((by2015 - cz2015)**2)),
np.sqrt(np.sum((by2015 - de2015)**2)),
np.sqrt(np.sum((by2015 - dk2015)**2)),
np.sqrt(np.sum((by2015 - ee2015)**2)),
np.sqrt(np.sum((by2015 - es2015)**2)),
np.sqrt(np.sum((by2015 - fi2015)**2)),
np.sqrt(np.sum((by2015 - fr2015)**2)),
np.sqrt(np.sum((by2015 - gb2015)**2)),
np.sqrt(np.sum((by2015 - gr2015)**2)),
np.sqrt(np.sum((by2015 - hr2015)**2)),
np.sqrt(np.sum((by2015 - hu2015)**2)),
np.sqrt(np.sum((by2015 - ie2015)**2)),
np.sqrt(np.sum((by2015 - it2015)**2)),
np.sqrt(np.sum((by2015 - nl2015)**2)),
np.sqrt(np.sum((by2015 - no2015)**2)),
np.sqrt(np.sum((by2015 - pl2015)**2)),
np.sqrt(np.sum((by2015 - pt2015)**2)),
np.sqrt(np.sum((by2015 - se2015)**2)),
np.sqrt(np.sum((by2015 - si2015)**2)),
np.sqrt(np.sum((by2015 - sk2015)**2)),
np.sqrt(np.sum((by2015 - tr2015)**2)),
])
ch2015_all =  np.array([
np.sqrt(np.sum((ch2015 - at2015)**2)),
np.sqrt(np.sum((ch2015 - be2015)**2)),
np.sqrt(np.sum((ch2015 - bg2015)**2)),
np.sqrt(np.sum((ch2015 - by2015)**2)),
np.sqrt(np.sum((ch2015 - ch2015)**2)),
np.sqrt(np.sum((ch2015 - cz2015)**2)),
np.sqrt(np.sum((ch2015 - de2015)**2)),
np.sqrt(np.sum((ch2015 - dk2015)**2)),
np.sqrt(np.sum((ch2015 - ee2015)**2)),
np.sqrt(np.sum((ch2015 - es2015)**2)),
np.sqrt(np.sum((ch2015 - fi2015)**2)),
np.sqrt(np.sum((ch2015 - fr2015)**2)),
np.sqrt(np.sum((ch2015 - gb2015)**2)),
np.sqrt(np.sum((ch2015 - gr2015)**2)),
np.sqrt(np.sum((ch2015 - hr2015)**2)),
np.sqrt(np.sum((ch2015 - hu2015)**2)),
np.sqrt(np.sum((ch2015 - ie2015)**2)),
np.sqrt(np.sum((ch2015 - it2015)**2)),
np.sqrt(np.sum((ch2015 - nl2015)**2)),
np.sqrt(np.sum((ch2015 - no2015)**2)),
np.sqrt(np.sum((ch2015 - pl2015)**2)),
np.sqrt(np.sum((ch2015 - pt2015)**2)),
np.sqrt(np.sum((ch2015 - se2015)**2)),
np.sqrt(np.sum((ch2015 - si2015)**2)),
np.sqrt(np.sum((ch2015 - sk2015)**2)),
np.sqrt(np.sum((ch2015 - tr2015)**2)),
])
cz2015_all =  np.array([
np.sqrt(np.sum((cz2015 - at2015)**2)),
np.sqrt(np.sum((cz2015 - be2015)**2)),
np.sqrt(np.sum((cz2015 - bg2015)**2)),
np.sqrt(np.sum((cz2015 - by2015)**2)),
np.sqrt(np.sum((cz2015 - ch2015)**2)),
np.sqrt(np.sum((cz2015 - cz2015)**2)),
np.sqrt(np.sum((cz2015 - de2015)**2)),
np.sqrt(np.sum((cz2015 - dk2015)**2)),
np.sqrt(np.sum((cz2015 - ee2015)**2)),
np.sqrt(np.sum((cz2015 - es2015)**2)),
np.sqrt(np.sum((cz2015 - fi2015)**2)),
np.sqrt(np.sum((cz2015 - fr2015)**2)),
np.sqrt(np.sum((cz2015 - gb2015)**2)),
np.sqrt(np.sum((cz2015 - gr2015)**2)),
np.sqrt(np.sum((cz2015 - hr2015)**2)),
np.sqrt(np.sum((cz2015 - hu2015)**2)),
np.sqrt(np.sum((cz2015 - ie2015)**2)),
np.sqrt(np.sum((cz2015 - it2015)**2)),
np.sqrt(np.sum((cz2015 - nl2015)**2)),
np.sqrt(np.sum((cz2015 - no2015)**2)),
np.sqrt(np.sum((cz2015 - pl2015)**2)),
np.sqrt(np.sum((cz2015 - pt2015)**2)),
np.sqrt(np.sum((cz2015 - se2015)**2)),
np.sqrt(np.sum((cz2015 - si2015)**2)),
np.sqrt(np.sum((cz2015 - sk2015)**2)),
np.sqrt(np.sum((cz2015 - tr2015)**2)),
])
de2015_all =  np.array([
np.sqrt(np.sum((de2015 - at2015)**2)),
np.sqrt(np.sum((de2015 - be2015)**2)),
np.sqrt(np.sum((de2015 - bg2015)**2)),
np.sqrt(np.sum((de2015 - by2015)**2)),
np.sqrt(np.sum((de2015 - ch2015)**2)),
np.sqrt(np.sum((de2015 - cz2015)**2)),
np.sqrt(np.sum((de2015 - de2015)**2)),
np.sqrt(np.sum((de2015 - dk2015)**2)),
np.sqrt(np.sum((de2015 - ee2015)**2)),
np.sqrt(np.sum((de2015 - es2015)**2)),
np.sqrt(np.sum((de2015 - fi2015)**2)),
np.sqrt(np.sum((de2015 - fr2015)**2)),
np.sqrt(np.sum((de2015 - gb2015)**2)),
np.sqrt(np.sum((de2015 - gr2015)**2)),
np.sqrt(np.sum((de2015 - hr2015)**2)),
np.sqrt(np.sum((de2015 - hu2015)**2)),
np.sqrt(np.sum((de2015 - ie2015)**2)),
np.sqrt(np.sum((de2015 - it2015)**2)),
np.sqrt(np.sum((de2015 - nl2015)**2)),
np.sqrt(np.sum((de2015 - no2015)**2)),
np.sqrt(np.sum((de2015 - pl2015)**2)),
np.sqrt(np.sum((de2015 - pt2015)**2)),
np.sqrt(np.sum((de2015 - se2015)**2)),
np.sqrt(np.sum((de2015 - si2015)**2)),
np.sqrt(np.sum((de2015 - sk2015)**2)),
np.sqrt(np.sum((de2015 - tr2015)**2)),
])
dk2015_all =  np.array([
np.sqrt(np.sum((dk2015 - at2015)**2)),
np.sqrt(np.sum((dk2015 - be2015)**2)),
np.sqrt(np.sum((dk2015 - bg2015)**2)),
np.sqrt(np.sum((dk2015 - by2015)**2)),
np.sqrt(np.sum((dk2015 - ch2015)**2)),
np.sqrt(np.sum((dk2015 - cz2015)**2)),
np.sqrt(np.sum((dk2015 - de2015)**2)),
np.sqrt(np.sum((dk2015 - dk2015)**2)),
np.sqrt(np.sum((dk2015 - ee2015)**2)),
np.sqrt(np.sum((dk2015 - es2015)**2)),
np.sqrt(np.sum((dk2015 - fi2015)**2)),
np.sqrt(np.sum((dk2015 - fr2015)**2)),
np.sqrt(np.sum((dk2015 - gb2015)**2)),
np.sqrt(np.sum((dk2015 - gr2015)**2)),
np.sqrt(np.sum((dk2015 - hr2015)**2)),
np.sqrt(np.sum((dk2015 - hu2015)**2)),
np.sqrt(np.sum((dk2015 - ie2015)**2)),
np.sqrt(np.sum((dk2015 - it2015)**2)),
np.sqrt(np.sum((dk2015 - nl2015)**2)),
np.sqrt(np.sum((dk2015 - no2015)**2)),
np.sqrt(np.sum((dk2015 - pl2015)**2)),
np.sqrt(np.sum((dk2015 - pt2015)**2)),
np.sqrt(np.sum((dk2015 - se2015)**2)),
np.sqrt(np.sum((dk2015 - si2015)**2)),
np.sqrt(np.sum((dk2015 - sk2015)**2)),
np.sqrt(np.sum((dk2015 - tr2015)**2)),
])
ee2015_all =  np.array([
np.sqrt(np.sum((ee2015 - at2015)**2)),
np.sqrt(np.sum((ee2015 - be2015)**2)),
np.sqrt(np.sum((ee2015 - bg2015)**2)),
np.sqrt(np.sum((ee2015 - by2015)**2)),
np.sqrt(np.sum((ee2015 - ch2015)**2)),
np.sqrt(np.sum((ee2015 - cz2015)**2)),
np.sqrt(np.sum((ee2015 - de2015)**2)),
np.sqrt(np.sum((ee2015 - dk2015)**2)),
np.sqrt(np.sum((ee2015 - ee2015)**2)),
np.sqrt(np.sum((ee2015 - es2015)**2)),
np.sqrt(np.sum((ee2015 - fi2015)**2)),
np.sqrt(np.sum((ee2015 - fr2015)**2)),
np.sqrt(np.sum((ee2015 - gb2015)**2)),
np.sqrt(np.sum((ee2015 - gr2015)**2)),
np.sqrt(np.sum((ee2015 - hr2015)**2)),
np.sqrt(np.sum((ee2015 - hu2015)**2)),
np.sqrt(np.sum((ee2015 - ie2015)**2)),
np.sqrt(np.sum((ee2015 - it2015)**2)),
np.sqrt(np.sum((ee2015 - nl2015)**2)),
np.sqrt(np.sum((ee2015 - no2015)**2)),
np.sqrt(np.sum((ee2015 - pl2015)**2)),
np.sqrt(np.sum((ee2015 - pt2015)**2)),
np.sqrt(np.sum((ee2015 - se2015)**2)),
np.sqrt(np.sum((ee2015 - si2015)**2)),
np.sqrt(np.sum((ee2015 - sk2015)**2)),
np.sqrt(np.sum((ee2015 - tr2015)**2)),
])
es2015_all =  np.array([
np.sqrt(np.sum((es2015 - at2015)**2)),
np.sqrt(np.sum((es2015 - be2015)**2)),
np.sqrt(np.sum((es2015 - bg2015)**2)),
np.sqrt(np.sum((es2015 - by2015)**2)),
np.sqrt(np.sum((es2015 - ch2015)**2)),
np.sqrt(np.sum((es2015 - cz2015)**2)),
np.sqrt(np.sum((es2015 - de2015)**2)),
np.sqrt(np.sum((es2015 - dk2015)**2)),
np.sqrt(np.sum((es2015 - ee2015)**2)),
np.sqrt(np.sum((es2015 - es2015)**2)),
np.sqrt(np.sum((es2015 - fi2015)**2)),
np.sqrt(np.sum((es2015 - fr2015)**2)),
np.sqrt(np.sum((es2015 - gb2015)**2)),
np.sqrt(np.sum((es2015 - gr2015)**2)),
np.sqrt(np.sum((es2015 - hr2015)**2)),
np.sqrt(np.sum((es2015 - hu2015)**2)),
np.sqrt(np.sum((es2015 - ie2015)**2)),
np.sqrt(np.sum((es2015 - it2015)**2)),
np.sqrt(np.sum((es2015 - nl2015)**2)),
np.sqrt(np.sum((es2015 - no2015)**2)),
np.sqrt(np.sum((es2015 - pl2015)**2)),
np.sqrt(np.sum((es2015 - pt2015)**2)),
np.sqrt(np.sum((es2015 - se2015)**2)),
np.sqrt(np.sum((es2015 - si2015)**2)),
np.sqrt(np.sum((es2015 - sk2015)**2)),
np.sqrt(np.sum((es2015 - tr2015)**2)),
])
fi2015_all =  np.array([
np.sqrt(np.sum((fi2015 - at2015)**2)),
np.sqrt(np.sum((fi2015 - be2015)**2)),
np.sqrt(np.sum((fi2015 - bg2015)**2)),
np.sqrt(np.sum((fi2015 - by2015)**2)),
np.sqrt(np.sum((fi2015 - ch2015)**2)),
np.sqrt(np.sum((fi2015 - cz2015)**2)),
np.sqrt(np.sum((fi2015 - de2015)**2)),
np.sqrt(np.sum((fi2015 - dk2015)**2)),
np.sqrt(np.sum((fi2015 - ee2015)**2)),
np.sqrt(np.sum((fi2015 - es2015)**2)),
np.sqrt(np.sum((fi2015 - fi2015)**2)),
np.sqrt(np.sum((fi2015 - fr2015)**2)),
np.sqrt(np.sum((fi2015 - gb2015)**2)),
np.sqrt(np.sum((fi2015 - gr2015)**2)),
np.sqrt(np.sum((fi2015 - hr2015)**2)),
np.sqrt(np.sum((fi2015 - hu2015)**2)),
np.sqrt(np.sum((fi2015 - ie2015)**2)),
np.sqrt(np.sum((fi2015 - it2015)**2)),
np.sqrt(np.sum((fi2015 - nl2015)**2)),
np.sqrt(np.sum((fi2015 - no2015)**2)),
np.sqrt(np.sum((fi2015 - pl2015)**2)),
np.sqrt(np.sum((fi2015 - pt2015)**2)),
np.sqrt(np.sum((fi2015 - se2015)**2)),
np.sqrt(np.sum((fi2015 - si2015)**2)),
np.sqrt(np.sum((fi2015 - sk2015)**2)),
np.sqrt(np.sum((fi2015 - tr2015)**2)),
])
fr2015_all =  np.array([
np.sqrt(np.sum((fr2015 - at2015)**2)),
np.sqrt(np.sum((fr2015 - be2015)**2)),
np.sqrt(np.sum((fr2015 - bg2015)**2)),
np.sqrt(np.sum((fr2015 - by2015)**2)),
np.sqrt(np.sum((fr2015 - ch2015)**2)),
np.sqrt(np.sum((fr2015 - cz2015)**2)),
np.sqrt(np.sum((fr2015 - de2015)**2)),
np.sqrt(np.sum((fr2015 - dk2015)**2)),
np.sqrt(np.sum((fr2015 - ee2015)**2)),
np.sqrt(np.sum((fr2015 - es2015)**2)),
np.sqrt(np.sum((fr2015 - fi2015)**2)),
np.sqrt(np.sum((fr2015 - fr2015)**2)),
np.sqrt(np.sum((fr2015 - gb2015)**2)),
np.sqrt(np.sum((fr2015 - gr2015)**2)),
np.sqrt(np.sum((fr2015 - hr2015)**2)),
np.sqrt(np.sum((fr2015 - hu2015)**2)),
np.sqrt(np.sum((fr2015 - ie2015)**2)),
np.sqrt(np.sum((fr2015 - it2015)**2)),
np.sqrt(np.sum((fr2015 - nl2015)**2)),
np.sqrt(np.sum((fr2015 - no2015)**2)),
np.sqrt(np.sum((fr2015 - pl2015)**2)),
np.sqrt(np.sum((fr2015 - pt2015)**2)),
np.sqrt(np.sum((fr2015 - se2015)**2)),
np.sqrt(np.sum((fr2015 - si2015)**2)),
np.sqrt(np.sum((fr2015 - sk2015)**2)),
np.sqrt(np.sum((fr2015 - tr2015)**2)),
])
gb2015_all =  np.array([
np.sqrt(np.sum((gb2015 - at2015)**2)),
np.sqrt(np.sum((gb2015 - be2015)**2)),
np.sqrt(np.sum((gb2015 - bg2015)**2)),
np.sqrt(np.sum((gb2015 - by2015)**2)),
np.sqrt(np.sum((gb2015 - ch2015)**2)),
np.sqrt(np.sum((gb2015 - cz2015)**2)),
np.sqrt(np.sum((gb2015 - de2015)**2)),
np.sqrt(np.sum((gb2015 - dk2015)**2)),
np.sqrt(np.sum((gb2015 - ee2015)**2)),
np.sqrt(np.sum((gb2015 - es2015)**2)),
np.sqrt(np.sum((gb2015 - fi2015)**2)),
np.sqrt(np.sum((gb2015 - fr2015)**2)),
np.sqrt(np.sum((gb2015 - gb2015)**2)),
np.sqrt(np.sum((gb2015 - gr2015)**2)),
np.sqrt(np.sum((gb2015 - hr2015)**2)),
np.sqrt(np.sum((gb2015 - hu2015)**2)),
np.sqrt(np.sum((gb2015 - ie2015)**2)),
np.sqrt(np.sum((gb2015 - it2015)**2)),
np.sqrt(np.sum((gb2015 - nl2015)**2)),
np.sqrt(np.sum((gb2015 - no2015)**2)),
np.sqrt(np.sum((gb2015 - pl2015)**2)),
np.sqrt(np.sum((gb2015 - pt2015)**2)),
np.sqrt(np.sum((gb2015 - se2015)**2)),
np.sqrt(np.sum((gb2015 - si2015)**2)),
np.sqrt(np.sum((gb2015 - sk2015)**2)),
np.sqrt(np.sum((gb2015 - tr2015)**2)),
])
gr2015_all =  np.array([
np.sqrt(np.sum((gr2015 - at2015)**2)),
np.sqrt(np.sum((gr2015 - be2015)**2)),
np.sqrt(np.sum((gr2015 - bg2015)**2)),
np.sqrt(np.sum((gr2015 - by2015)**2)),
np.sqrt(np.sum((gr2015 - ch2015)**2)),
np.sqrt(np.sum((gr2015 - cz2015)**2)),
np.sqrt(np.sum((gr2015 - de2015)**2)),
np.sqrt(np.sum((gr2015 - dk2015)**2)),
np.sqrt(np.sum((gr2015 - ee2015)**2)),
np.sqrt(np.sum((gr2015 - es2015)**2)),
np.sqrt(np.sum((gr2015 - fi2015)**2)),
np.sqrt(np.sum((gr2015 - fr2015)**2)),
np.sqrt(np.sum((gr2015 - gb2015)**2)),
np.sqrt(np.sum((gr2015 - gr2015)**2)),
np.sqrt(np.sum((gr2015 - hr2015)**2)),
np.sqrt(np.sum((gr2015 - hu2015)**2)),
np.sqrt(np.sum((gr2015 - ie2015)**2)),
np.sqrt(np.sum((gr2015 - it2015)**2)),
np.sqrt(np.sum((gr2015 - nl2015)**2)),
np.sqrt(np.sum((gr2015 - no2015)**2)),
np.sqrt(np.sum((gr2015 - pl2015)**2)),
np.sqrt(np.sum((gr2015 - pt2015)**2)),
np.sqrt(np.sum((gr2015 - se2015)**2)),
np.sqrt(np.sum((gr2015 - si2015)**2)),
np.sqrt(np.sum((gr2015 - sk2015)**2)),
np.sqrt(np.sum((gr2015 - tr2015)**2)),
])
hr2015_all =  np.array([
np.sqrt(np.sum((hr2015 - at2015)**2)),
np.sqrt(np.sum((hr2015 - be2015)**2)),
np.sqrt(np.sum((hr2015 - bg2015)**2)),
np.sqrt(np.sum((hr2015 - by2015)**2)),
np.sqrt(np.sum((hr2015 - ch2015)**2)),
np.sqrt(np.sum((hr2015 - cz2015)**2)),
np.sqrt(np.sum((hr2015 - de2015)**2)),
np.sqrt(np.sum((hr2015 - dk2015)**2)),
np.sqrt(np.sum((hr2015 - ee2015)**2)),
np.sqrt(np.sum((hr2015 - es2015)**2)),
np.sqrt(np.sum((hr2015 - fi2015)**2)),
np.sqrt(np.sum((hr2015 - fr2015)**2)),
np.sqrt(np.sum((hr2015 - gb2015)**2)),
np.sqrt(np.sum((hr2015 - gr2015)**2)),
np.sqrt(np.sum((hr2015 - hr2015)**2)),
np.sqrt(np.sum((hr2015 - hu2015)**2)),
np.sqrt(np.sum((hr2015 - ie2015)**2)),
np.sqrt(np.sum((hr2015 - it2015)**2)),
np.sqrt(np.sum((hr2015 - nl2015)**2)),
np.sqrt(np.sum((hr2015 - no2015)**2)),
np.sqrt(np.sum((hr2015 - pl2015)**2)),
np.sqrt(np.sum((hr2015 - pt2015)**2)),
np.sqrt(np.sum((hr2015 - se2015)**2)),
np.sqrt(np.sum((hr2015 - si2015)**2)),
np.sqrt(np.sum((hr2015 - sk2015)**2)),
np.sqrt(np.sum((hr2015 - tr2015)**2)),
])
hu2015_all =  np.array([
np.sqrt(np.sum((hu2015 - at2015)**2)),
np.sqrt(np.sum((hu2015 - be2015)**2)),
np.sqrt(np.sum((hu2015 - bg2015)**2)),
np.sqrt(np.sum((hu2015 - by2015)**2)),
np.sqrt(np.sum((hu2015 - ch2015)**2)),
np.sqrt(np.sum((hu2015 - cz2015)**2)),
np.sqrt(np.sum((hu2015 - de2015)**2)),
np.sqrt(np.sum((hu2015 - dk2015)**2)),
np.sqrt(np.sum((hu2015 - ee2015)**2)),
np.sqrt(np.sum((hu2015 - es2015)**2)),
np.sqrt(np.sum((hu2015 - fi2015)**2)),
np.sqrt(np.sum((hu2015 - fr2015)**2)),
np.sqrt(np.sum((hu2015 - gb2015)**2)),
np.sqrt(np.sum((hu2015 - gr2015)**2)),
np.sqrt(np.sum((hu2015 - hr2015)**2)),
np.sqrt(np.sum((hu2015 - hu2015)**2)),
np.sqrt(np.sum((hu2015 - ie2015)**2)),
np.sqrt(np.sum((hu2015 - it2015)**2)),
np.sqrt(np.sum((hu2015 - nl2015)**2)),
np.sqrt(np.sum((hu2015 - no2015)**2)),
np.sqrt(np.sum((hu2015 - pl2015)**2)),
np.sqrt(np.sum((hu2015 - pt2015)**2)),
np.sqrt(np.sum((hu2015 - se2015)**2)),
np.sqrt(np.sum((hu2015 - si2015)**2)),
np.sqrt(np.sum((hu2015 - sk2015)**2)),
np.sqrt(np.sum((hu2015 - tr2015)**2)),
])
ie2015_all =  np.array([
np.sqrt(np.sum((ie2015 - at2015)**2)),
np.sqrt(np.sum((ie2015 - be2015)**2)),
np.sqrt(np.sum((ie2015 - bg2015)**2)),
np.sqrt(np.sum((ie2015 - by2015)**2)),
np.sqrt(np.sum((ie2015 - ch2015)**2)),
np.sqrt(np.sum((ie2015 - cz2015)**2)),
np.sqrt(np.sum((ie2015 - de2015)**2)),
np.sqrt(np.sum((ie2015 - dk2015)**2)),
np.sqrt(np.sum((ie2015 - ee2015)**2)),
np.sqrt(np.sum((ie2015 - es2015)**2)),
np.sqrt(np.sum((ie2015 - fi2015)**2)),
np.sqrt(np.sum((ie2015 - fr2015)**2)),
np.sqrt(np.sum((ie2015 - gb2015)**2)),
np.sqrt(np.sum((ie2015 - gr2015)**2)),
np.sqrt(np.sum((ie2015 - hr2015)**2)),
np.sqrt(np.sum((ie2015 - hu2015)**2)),
np.sqrt(np.sum((ie2015 - ie2015)**2)),
np.sqrt(np.sum((ie2015 - it2015)**2)),
np.sqrt(np.sum((ie2015 - nl2015)**2)),
np.sqrt(np.sum((ie2015 - no2015)**2)),
np.sqrt(np.sum((ie2015 - pl2015)**2)),
np.sqrt(np.sum((ie2015 - pt2015)**2)),
np.sqrt(np.sum((ie2015 - se2015)**2)),
np.sqrt(np.sum((ie2015 - si2015)**2)),
np.sqrt(np.sum((ie2015 - sk2015)**2)),
np.sqrt(np.sum((ie2015 - tr2015)**2)),
])
it2015_all =  np.array([
np.sqrt(np.sum((it2015 - at2015)**2)),
np.sqrt(np.sum((it2015 - be2015)**2)),
np.sqrt(np.sum((it2015 - bg2015)**2)),
np.sqrt(np.sum((it2015 - by2015)**2)),
np.sqrt(np.sum((it2015 - ch2015)**2)),
np.sqrt(np.sum((it2015 - cz2015)**2)),
np.sqrt(np.sum((it2015 - de2015)**2)),
np.sqrt(np.sum((it2015 - dk2015)**2)),
np.sqrt(np.sum((it2015 - ee2015)**2)),
np.sqrt(np.sum((it2015 - es2015)**2)),
np.sqrt(np.sum((it2015 - fi2015)**2)),
np.sqrt(np.sum((it2015 - fr2015)**2)),
np.sqrt(np.sum((it2015 - gb2015)**2)),
np.sqrt(np.sum((it2015 - gr2015)**2)),
np.sqrt(np.sum((it2015 - hr2015)**2)),
np.sqrt(np.sum((it2015 - hu2015)**2)),
np.sqrt(np.sum((it2015 - ie2015)**2)),
np.sqrt(np.sum((it2015 - it2015)**2)),
np.sqrt(np.sum((it2015 - nl2015)**2)),
np.sqrt(np.sum((it2015 - no2015)**2)),
np.sqrt(np.sum((it2015 - pl2015)**2)),
np.sqrt(np.sum((it2015 - pt2015)**2)),
np.sqrt(np.sum((it2015 - se2015)**2)),
np.sqrt(np.sum((it2015 - si2015)**2)),
np.sqrt(np.sum((it2015 - sk2015)**2)),
np.sqrt(np.sum((it2015 - tr2015)**2)),
])
nl2015_all =  np.array([
np.sqrt(np.sum((nl2015 - at2015)**2)),
np.sqrt(np.sum((nl2015 - be2015)**2)),
np.sqrt(np.sum((nl2015 - bg2015)**2)),
np.sqrt(np.sum((nl2015 - by2015)**2)),
np.sqrt(np.sum((nl2015 - ch2015)**2)),
np.sqrt(np.sum((nl2015 - cz2015)**2)),
np.sqrt(np.sum((nl2015 - de2015)**2)),
np.sqrt(np.sum((nl2015 - dk2015)**2)),
np.sqrt(np.sum((nl2015 - ee2015)**2)),
np.sqrt(np.sum((nl2015 - es2015)**2)),
np.sqrt(np.sum((nl2015 - fi2015)**2)),
np.sqrt(np.sum((nl2015 - fr2015)**2)),
np.sqrt(np.sum((nl2015 - gb2015)**2)),
np.sqrt(np.sum((nl2015 - gr2015)**2)),
np.sqrt(np.sum((nl2015 - hr2015)**2)),
np.sqrt(np.sum((nl2015 - hu2015)**2)),
np.sqrt(np.sum((nl2015 - ie2015)**2)),
np.sqrt(np.sum((nl2015 - it2015)**2)),
np.sqrt(np.sum((nl2015 - nl2015)**2)),
np.sqrt(np.sum((nl2015 - no2015)**2)),
np.sqrt(np.sum((nl2015 - pl2015)**2)),
np.sqrt(np.sum((nl2015 - pt2015)**2)),
np.sqrt(np.sum((nl2015 - se2015)**2)),
np.sqrt(np.sum((nl2015 - si2015)**2)),
np.sqrt(np.sum((nl2015 - sk2015)**2)),
np.sqrt(np.sum((nl2015 - tr2015)**2)),
])
no2015_all =  np.array([
np.sqrt(np.sum((no2015 - at2015)**2)),
np.sqrt(np.sum((no2015 - be2015)**2)),
np.sqrt(np.sum((no2015 - bg2015)**2)),
np.sqrt(np.sum((no2015 - by2015)**2)),
np.sqrt(np.sum((no2015 - ch2015)**2)),
np.sqrt(np.sum((no2015 - cz2015)**2)),
np.sqrt(np.sum((no2015 - de2015)**2)),
np.sqrt(np.sum((no2015 - dk2015)**2)),
np.sqrt(np.sum((no2015 - ee2015)**2)),
np.sqrt(np.sum((no2015 - es2015)**2)),
np.sqrt(np.sum((no2015 - fi2015)**2)),
np.sqrt(np.sum((no2015 - fr2015)**2)),
np.sqrt(np.sum((no2015 - gb2015)**2)),
np.sqrt(np.sum((no2015 - gr2015)**2)),
np.sqrt(np.sum((no2015 - hr2015)**2)),
np.sqrt(np.sum((no2015 - hu2015)**2)),
np.sqrt(np.sum((no2015 - ie2015)**2)),
np.sqrt(np.sum((no2015 - it2015)**2)),
np.sqrt(np.sum((no2015 - nl2015)**2)),
np.sqrt(np.sum((no2015 - no2015)**2)),
np.sqrt(np.sum((no2015 - pl2015)**2)),
np.sqrt(np.sum((no2015 - pt2015)**2)),
np.sqrt(np.sum((no2015 - se2015)**2)),
np.sqrt(np.sum((no2015 - si2015)**2)),
np.sqrt(np.sum((no2015 - sk2015)**2)),
np.sqrt(np.sum((no2015 - tr2015)**2)),
])
pl2015_all =  np.array([
np.sqrt(np.sum((pl2015 - at2015)**2)),
np.sqrt(np.sum((pl2015 - be2015)**2)),
np.sqrt(np.sum((pl2015 - bg2015)**2)),
np.sqrt(np.sum((pl2015 - by2015)**2)),
np.sqrt(np.sum((pl2015 - ch2015)**2)),
np.sqrt(np.sum((pl2015 - cz2015)**2)),
np.sqrt(np.sum((pl2015 - de2015)**2)),
np.sqrt(np.sum((pl2015 - dk2015)**2)),
np.sqrt(np.sum((pl2015 - ee2015)**2)),
np.sqrt(np.sum((pl2015 - es2015)**2)),
np.sqrt(np.sum((pl2015 - fi2015)**2)),
np.sqrt(np.sum((pl2015 - fr2015)**2)),
np.sqrt(np.sum((pl2015 - gb2015)**2)),
np.sqrt(np.sum((pl2015 - gr2015)**2)),
np.sqrt(np.sum((pl2015 - hr2015)**2)),
np.sqrt(np.sum((pl2015 - hu2015)**2)),
np.sqrt(np.sum((pl2015 - ie2015)**2)),
np.sqrt(np.sum((pl2015 - it2015)**2)),
np.sqrt(np.sum((pl2015 - nl2015)**2)),
np.sqrt(np.sum((pl2015 - no2015)**2)),
np.sqrt(np.sum((pl2015 - pl2015)**2)),
np.sqrt(np.sum((pl2015 - pt2015)**2)),
np.sqrt(np.sum((pl2015 - se2015)**2)),
np.sqrt(np.sum((pl2015 - si2015)**2)),
np.sqrt(np.sum((pl2015 - sk2015)**2)),
np.sqrt(np.sum((pl2015 - tr2015)**2)),
])
pt2015_all =  np.array([
np.sqrt(np.sum((pt2015 - at2015)**2)),
np.sqrt(np.sum((pt2015 - be2015)**2)),
np.sqrt(np.sum((pt2015 - bg2015)**2)),
np.sqrt(np.sum((pt2015 - by2015)**2)),
np.sqrt(np.sum((pt2015 - ch2015)**2)),
np.sqrt(np.sum((pt2015 - cz2015)**2)),
np.sqrt(np.sum((pt2015 - de2015)**2)),
np.sqrt(np.sum((pt2015 - dk2015)**2)),
np.sqrt(np.sum((pt2015 - ee2015)**2)),
np.sqrt(np.sum((pt2015 - es2015)**2)),
np.sqrt(np.sum((pt2015 - fi2015)**2)),
np.sqrt(np.sum((pt2015 - fr2015)**2)),
np.sqrt(np.sum((pt2015 - gb2015)**2)),
np.sqrt(np.sum((pt2015 - gr2015)**2)),
np.sqrt(np.sum((pt2015 - hr2015)**2)),
np.sqrt(np.sum((pt2015 - hu2015)**2)),
np.sqrt(np.sum((pt2015 - ie2015)**2)),
np.sqrt(np.sum((pt2015 - it2015)**2)),
np.sqrt(np.sum((pt2015 - nl2015)**2)),
np.sqrt(np.sum((pt2015 - no2015)**2)),
np.sqrt(np.sum((pt2015 - pl2015)**2)),
np.sqrt(np.sum((pt2015 - pt2015)**2)),
np.sqrt(np.sum((pt2015 - se2015)**2)),
np.sqrt(np.sum((pt2015 - si2015)**2)),
np.sqrt(np.sum((pt2015 - sk2015)**2)),
np.sqrt(np.sum((pt2015 - tr2015)**2)),
])
se2015_all =  np.array([
np.sqrt(np.sum((se2015 - at2015)**2)),
np.sqrt(np.sum((se2015 - be2015)**2)),
np.sqrt(np.sum((se2015 - bg2015)**2)),
np.sqrt(np.sum((se2015 - by2015)**2)),
np.sqrt(np.sum((se2015 - ch2015)**2)),
np.sqrt(np.sum((se2015 - cz2015)**2)),
np.sqrt(np.sum((se2015 - de2015)**2)),
np.sqrt(np.sum((se2015 - dk2015)**2)),
np.sqrt(np.sum((se2015 - ee2015)**2)),
np.sqrt(np.sum((se2015 - es2015)**2)),
np.sqrt(np.sum((se2015 - fi2015)**2)),
np.sqrt(np.sum((se2015 - fr2015)**2)),
np.sqrt(np.sum((se2015 - gb2015)**2)),
np.sqrt(np.sum((se2015 - gr2015)**2)),
np.sqrt(np.sum((se2015 - hr2015)**2)),
np.sqrt(np.sum((se2015 - hu2015)**2)),
np.sqrt(np.sum((se2015 - ie2015)**2)),
np.sqrt(np.sum((se2015 - it2015)**2)),
np.sqrt(np.sum((se2015 - nl2015)**2)),
np.sqrt(np.sum((se2015 - no2015)**2)),
np.sqrt(np.sum((se2015 - pl2015)**2)),
np.sqrt(np.sum((se2015 - pt2015)**2)),
np.sqrt(np.sum((se2015 - se2015)**2)),
np.sqrt(np.sum((se2015 - si2015)**2)),
np.sqrt(np.sum((se2015 - sk2015)**2)),
np.sqrt(np.sum((se2015 - tr2015)**2)),
])
si2015_all =  np.array([
np.sqrt(np.sum((si2015 - at2015)**2)),
np.sqrt(np.sum((si2015 - be2015)**2)),
np.sqrt(np.sum((si2015 - bg2015)**2)),
np.sqrt(np.sum((si2015 - by2015)**2)),
np.sqrt(np.sum((si2015 - ch2015)**2)),
np.sqrt(np.sum((si2015 - cz2015)**2)),
np.sqrt(np.sum((si2015 - de2015)**2)),
np.sqrt(np.sum((si2015 - dk2015)**2)),
np.sqrt(np.sum((si2015 - ee2015)**2)),
np.sqrt(np.sum((si2015 - es2015)**2)),
np.sqrt(np.sum((si2015 - fi2015)**2)),
np.sqrt(np.sum((si2015 - fr2015)**2)),
np.sqrt(np.sum((si2015 - gb2015)**2)),
np.sqrt(np.sum((si2015 - gr2015)**2)),
np.sqrt(np.sum((si2015 - hr2015)**2)),
np.sqrt(np.sum((si2015 - hu2015)**2)),
np.sqrt(np.sum((si2015 - ie2015)**2)),
np.sqrt(np.sum((si2015 - it2015)**2)),
np.sqrt(np.sum((si2015 - nl2015)**2)),
np.sqrt(np.sum((si2015 - no2015)**2)),
np.sqrt(np.sum((si2015 - pl2015)**2)),
np.sqrt(np.sum((si2015 - pt2015)**2)),
np.sqrt(np.sum((si2015 - se2015)**2)),
np.sqrt(np.sum((si2015 - si2015)**2)),
np.sqrt(np.sum((si2015 - sk2015)**2)),
np.sqrt(np.sum((si2015 - tr2015)**2)),
])
sk2015_all =  np.array([
np.sqrt(np.sum((sk2015 - at2015)**2)),
np.sqrt(np.sum((sk2015 - be2015)**2)),
np.sqrt(np.sum((sk2015 - bg2015)**2)),
np.sqrt(np.sum((sk2015 - by2015)**2)),
np.sqrt(np.sum((sk2015 - ch2015)**2)),
np.sqrt(np.sum((sk2015 - cz2015)**2)),
np.sqrt(np.sum((sk2015 - de2015)**2)),
np.sqrt(np.sum((sk2015 - dk2015)**2)),
np.sqrt(np.sum((sk2015 - ee2015)**2)),
np.sqrt(np.sum((sk2015 - es2015)**2)),
np.sqrt(np.sum((sk2015 - fi2015)**2)),
np.sqrt(np.sum((sk2015 - fr2015)**2)),
np.sqrt(np.sum((sk2015 - gb2015)**2)),
np.sqrt(np.sum((sk2015 - gr2015)**2)),
np.sqrt(np.sum((sk2015 - hr2015)**2)),
np.sqrt(np.sum((sk2015 - hu2015)**2)),
np.sqrt(np.sum((sk2015 - ie2015)**2)),
np.sqrt(np.sum((sk2015 - it2015)**2)),
np.sqrt(np.sum((sk2015 - nl2015)**2)),
np.sqrt(np.sum((sk2015 - no2015)**2)),
np.sqrt(np.sum((sk2015 - pl2015)**2)),
np.sqrt(np.sum((sk2015 - pt2015)**2)),
np.sqrt(np.sum((sk2015 - se2015)**2)),
np.sqrt(np.sum((sk2015 - si2015)**2)),
np.sqrt(np.sum((sk2015 - sk2015)**2)),
np.sqrt(np.sum((sk2015 - tr2015)**2)),
])
tr2015_all =  np.array([
np.sqrt(np.sum((tr2015 - at2015)**2)),
np.sqrt(np.sum((tr2015 - be2015)**2)),
np.sqrt(np.sum((tr2015 - bg2015)**2)),
np.sqrt(np.sum((tr2015 - by2015)**2)),
np.sqrt(np.sum((tr2015 - ch2015)**2)),
np.sqrt(np.sum((tr2015 - cz2015)**2)),
np.sqrt(np.sum((tr2015 - de2015)**2)),
np.sqrt(np.sum((tr2015 - dk2015)**2)),
np.sqrt(np.sum((tr2015 - ee2015)**2)),
np.sqrt(np.sum((tr2015 - es2015)**2)),
np.sqrt(np.sum((tr2015 - fi2015)**2)),
np.sqrt(np.sum((tr2015 - fr2015)**2)),
np.sqrt(np.sum((tr2015 - gb2015)**2)),
np.sqrt(np.sum((tr2015 - gr2015)**2)),
np.sqrt(np.sum((tr2015 - hr2015)**2)),
np.sqrt(np.sum((tr2015 - hu2015)**2)),
np.sqrt(np.sum((tr2015 - ie2015)**2)),
np.sqrt(np.sum((tr2015 - it2015)**2)),
np.sqrt(np.sum((tr2015 - nl2015)**2)),
np.sqrt(np.sum((tr2015 - no2015)**2)),
np.sqrt(np.sum((tr2015 - pl2015)**2)),
np.sqrt(np.sum((tr2015 - pt2015)**2)),
np.sqrt(np.sum((tr2015 - se2015)**2)),
np.sqrt(np.sum((tr2015 - si2015)**2)),
np.sqrt(np.sum((tr2015 - sk2015)**2)),
np.sqrt(np.sum((tr2015 - tr2015)**2)),
])

In [ ]:
#2016
at2016_all =  np.array([
np.sqrt(np.sum((at2016 - at2016)**2)),
np.sqrt(np.sum((at2016 - be2016)**2)),
np.sqrt(np.sum((at2016 - bg2016)**2)),
np.sqrt(np.sum((at2016 - by2016)**2)),
np.sqrt(np.sum((at2016 - ch2016)**2)),
np.sqrt(np.sum((at2016 - cz2016)**2)),
np.sqrt(np.sum((at2016 - de2016)**2)),
np.sqrt(np.sum((at2016 - dk2016)**2)),
np.sqrt(np.sum((at2016 - ee2016)**2)),
np.sqrt(np.sum((at2016 - es2016)**2)),
np.sqrt(np.sum((at2016 - fi2016)**2)),
np.sqrt(np.sum((at2016 - fr2016)**2)),
np.sqrt(np.sum((at2016 - gb2016)**2)),
np.sqrt(np.sum((at2016 - gr2016)**2)),
np.sqrt(np.sum((at2016 - hr2016)**2)),
np.sqrt(np.sum((at2016 - hu2016)**2)),
np.sqrt(np.sum((at2016 - ie2016)**2)),
np.sqrt(np.sum((at2016 - it2016)**2)),
np.sqrt(np.sum((at2016 - nl2016)**2)),
np.sqrt(np.sum((at2016 - no2016)**2)),
np.sqrt(np.sum((at2016 - pl2016)**2)),
np.sqrt(np.sum((at2016 - pt2016)**2)),
np.sqrt(np.sum((at2016 - se2016)**2)),
np.sqrt(np.sum((at2016 - si2016)**2)),
np.sqrt(np.sum((at2016 - sk2016)**2)),
np.sqrt(np.sum((at2016 - tr2016)**2)),
])
be2016_all =  np.array([
np.sqrt(np.sum((be2016 - at2016)**2)),
np.sqrt(np.sum((be2016 - be2016)**2)),
np.sqrt(np.sum((be2016 - bg2016)**2)),
np.sqrt(np.sum((be2016 - by2016)**2)),
np.sqrt(np.sum((be2016 - ch2016)**2)),
np.sqrt(np.sum((be2016 - cz2016)**2)),
np.sqrt(np.sum((be2016 - de2016)**2)),
np.sqrt(np.sum((be2016 - dk2016)**2)),
np.sqrt(np.sum((be2016 - ee2016)**2)),
np.sqrt(np.sum((be2016 - es2016)**2)),
np.sqrt(np.sum((be2016 - fi2016)**2)),
np.sqrt(np.sum((be2016 - fr2016)**2)),
np.sqrt(np.sum((be2016 - gb2016)**2)),
np.sqrt(np.sum((be2016 - gr2016)**2)),
np.sqrt(np.sum((be2016 - hr2016)**2)),
np.sqrt(np.sum((be2016 - hu2016)**2)),
np.sqrt(np.sum((be2016 - ie2016)**2)),
np.sqrt(np.sum((be2016 - it2016)**2)),
np.sqrt(np.sum((be2016 - nl2016)**2)),
np.sqrt(np.sum((be2016 - no2016)**2)),
np.sqrt(np.sum((be2016 - pl2016)**2)),
np.sqrt(np.sum((be2016 - pt2016)**2)),
np.sqrt(np.sum((be2016 - se2016)**2)),
np.sqrt(np.sum((be2016 - si2016)**2)),
np.sqrt(np.sum((be2016 - sk2016)**2)),
np.sqrt(np.sum((be2016 - tr2016)**2)),
])
bg2016_all =  np.array([
np.sqrt(np.sum((bg2016 - at2016)**2)),
np.sqrt(np.sum((bg2016 - be2016)**2)),
np.sqrt(np.sum((bg2016 - bg2016)**2)),
np.sqrt(np.sum((bg2016 - by2016)**2)),
np.sqrt(np.sum((bg2016 - ch2016)**2)),
np.sqrt(np.sum((bg2016 - cz2016)**2)),
np.sqrt(np.sum((bg2016 - de2016)**2)),
np.sqrt(np.sum((bg2016 - dk2016)**2)),
np.sqrt(np.sum((bg2016 - ee2016)**2)),
np.sqrt(np.sum((bg2016 - es2016)**2)),
np.sqrt(np.sum((bg2016 - fi2016)**2)),
np.sqrt(np.sum((bg2016 - fr2016)**2)),
np.sqrt(np.sum((bg2016 - gb2016)**2)),
np.sqrt(np.sum((bg2016 - gr2016)**2)),
np.sqrt(np.sum((bg2016 - hr2016)**2)),
np.sqrt(np.sum((bg2016 - hu2016)**2)),
np.sqrt(np.sum((bg2016 - ie2016)**2)),
np.sqrt(np.sum((bg2016 - it2016)**2)),
np.sqrt(np.sum((bg2016 - nl2016)**2)),
np.sqrt(np.sum((bg2016 - no2016)**2)),
np.sqrt(np.sum((bg2016 - pl2016)**2)),
np.sqrt(np.sum((bg2016 - pt2016)**2)),
np.sqrt(np.sum((bg2016 - se2016)**2)),
np.sqrt(np.sum((bg2016 - si2016)**2)),
np.sqrt(np.sum((bg2016 - sk2016)**2)),
np.sqrt(np.sum((bg2016 - tr2016)**2)),
])
by2016_all =  np.array([
np.sqrt(np.sum((by2016 - at2016)**2)),
np.sqrt(np.sum((by2016 - be2016)**2)),
np.sqrt(np.sum((by2016 - bg2016)**2)),
np.sqrt(np.sum((by2016 - by2016)**2)),
np.sqrt(np.sum((by2016 - ch2016)**2)),
np.sqrt(np.sum((by2016 - cz2016)**2)),
np.sqrt(np.sum((by2016 - de2016)**2)),
np.sqrt(np.sum((by2016 - dk2016)**2)),
np.sqrt(np.sum((by2016 - ee2016)**2)),
np.sqrt(np.sum((by2016 - es2016)**2)),
np.sqrt(np.sum((by2016 - fi2016)**2)),
np.sqrt(np.sum((by2016 - fr2016)**2)),
np.sqrt(np.sum((by2016 - gb2016)**2)),
np.sqrt(np.sum((by2016 - gr2016)**2)),
np.sqrt(np.sum((by2016 - hr2016)**2)),
np.sqrt(np.sum((by2016 - hu2016)**2)),
np.sqrt(np.sum((by2016 - ie2016)**2)),
np.sqrt(np.sum((by2016 - it2016)**2)),
np.sqrt(np.sum((by2016 - nl2016)**2)),
np.sqrt(np.sum((by2016 - no2016)**2)),
np.sqrt(np.sum((by2016 - pl2016)**2)),
np.sqrt(np.sum((by2016 - pt2016)**2)),
np.sqrt(np.sum((by2016 - se2016)**2)),
np.sqrt(np.sum((by2016 - si2016)**2)),
np.sqrt(np.sum((by2016 - sk2016)**2)),
np.sqrt(np.sum((by2016 - tr2016)**2)),
])
ch2016_all =  np.array([
np.sqrt(np.sum((ch2016 - at2016)**2)),
np.sqrt(np.sum((ch2016 - be2016)**2)),
np.sqrt(np.sum((ch2016 - bg2016)**2)),
np.sqrt(np.sum((ch2016 - by2016)**2)),
np.sqrt(np.sum((ch2016 - ch2016)**2)),
np.sqrt(np.sum((ch2016 - cz2016)**2)),
np.sqrt(np.sum((ch2016 - de2016)**2)),
np.sqrt(np.sum((ch2016 - dk2016)**2)),
np.sqrt(np.sum((ch2016 - ee2016)**2)),
np.sqrt(np.sum((ch2016 - es2016)**2)),
np.sqrt(np.sum((ch2016 - fi2016)**2)),
np.sqrt(np.sum((ch2016 - fr2016)**2)),
np.sqrt(np.sum((ch2016 - gb2016)**2)),
np.sqrt(np.sum((ch2016 - gr2016)**2)),
np.sqrt(np.sum((ch2016 - hr2016)**2)),
np.sqrt(np.sum((ch2016 - hu2016)**2)),
np.sqrt(np.sum((ch2016 - ie2016)**2)),
np.sqrt(np.sum((ch2016 - it2016)**2)),
np.sqrt(np.sum((ch2016 - nl2016)**2)),
np.sqrt(np.sum((ch2016 - no2016)**2)),
np.sqrt(np.sum((ch2016 - pl2016)**2)),
np.sqrt(np.sum((ch2016 - pt2016)**2)),
np.sqrt(np.sum((ch2016 - se2016)**2)),
np.sqrt(np.sum((ch2016 - si2016)**2)),
np.sqrt(np.sum((ch2016 - sk2016)**2)),
np.sqrt(np.sum((ch2016 - tr2016)**2)),
])
cz2016_all =  np.array([
np.sqrt(np.sum((cz2016 - at2016)**2)),
np.sqrt(np.sum((cz2016 - be2016)**2)),
np.sqrt(np.sum((cz2016 - bg2016)**2)),
np.sqrt(np.sum((cz2016 - by2016)**2)),
np.sqrt(np.sum((cz2016 - ch2016)**2)),
np.sqrt(np.sum((cz2016 - cz2016)**2)),
np.sqrt(np.sum((cz2016 - de2016)**2)),
np.sqrt(np.sum((cz2016 - dk2016)**2)),
np.sqrt(np.sum((cz2016 - ee2016)**2)),
np.sqrt(np.sum((cz2016 - es2016)**2)),
np.sqrt(np.sum((cz2016 - fi2016)**2)),
np.sqrt(np.sum((cz2016 - fr2016)**2)),
np.sqrt(np.sum((cz2016 - gb2016)**2)),
np.sqrt(np.sum((cz2016 - gr2016)**2)),
np.sqrt(np.sum((cz2016 - hr2016)**2)),
np.sqrt(np.sum((cz2016 - hu2016)**2)),
np.sqrt(np.sum((cz2016 - ie2016)**2)),
np.sqrt(np.sum((cz2016 - it2016)**2)),
np.sqrt(np.sum((cz2016 - nl2016)**2)),
np.sqrt(np.sum((cz2016 - no2016)**2)),
np.sqrt(np.sum((cz2016 - pl2016)**2)),
np.sqrt(np.sum((cz2016 - pt2016)**2)),
np.sqrt(np.sum((cz2016 - se2016)**2)),
np.sqrt(np.sum((cz2016 - si2016)**2)),
np.sqrt(np.sum((cz2016 - sk2016)**2)),
np.sqrt(np.sum((cz2016 - tr2016)**2)),
])
de2016_all =  np.array([
np.sqrt(np.sum((de2016 - at2016)**2)),
np.sqrt(np.sum((de2016 - be2016)**2)),
np.sqrt(np.sum((de2016 - bg2016)**2)),
np.sqrt(np.sum((de2016 - by2016)**2)),
np.sqrt(np.sum((de2016 - ch2016)**2)),
np.sqrt(np.sum((de2016 - cz2016)**2)),
np.sqrt(np.sum((de2016 - de2016)**2)),
np.sqrt(np.sum((de2016 - dk2016)**2)),
np.sqrt(np.sum((de2016 - ee2016)**2)),
np.sqrt(np.sum((de2016 - es2016)**2)),
np.sqrt(np.sum((de2016 - fi2016)**2)),
np.sqrt(np.sum((de2016 - fr2016)**2)),
np.sqrt(np.sum((de2016 - gb2016)**2)),
np.sqrt(np.sum((de2016 - gr2016)**2)),
np.sqrt(np.sum((de2016 - hr2016)**2)),
np.sqrt(np.sum((de2016 - hu2016)**2)),
np.sqrt(np.sum((de2016 - ie2016)**2)),
np.sqrt(np.sum((de2016 - it2016)**2)),
np.sqrt(np.sum((de2016 - nl2016)**2)),
np.sqrt(np.sum((de2016 - no2016)**2)),
np.sqrt(np.sum((de2016 - pl2016)**2)),
np.sqrt(np.sum((de2016 - pt2016)**2)),
np.sqrt(np.sum((de2016 - se2016)**2)),
np.sqrt(np.sum((de2016 - si2016)**2)),
np.sqrt(np.sum((de2016 - sk2016)**2)),
np.sqrt(np.sum((de2016 - tr2016)**2)),
])
dk2016_all =  np.array([
np.sqrt(np.sum((dk2016 - at2016)**2)),
np.sqrt(np.sum((dk2016 - be2016)**2)),
np.sqrt(np.sum((dk2016 - bg2016)**2)),
np.sqrt(np.sum((dk2016 - by2016)**2)),
np.sqrt(np.sum((dk2016 - ch2016)**2)),
np.sqrt(np.sum((dk2016 - cz2016)**2)),
np.sqrt(np.sum((dk2016 - de2016)**2)),
np.sqrt(np.sum((dk2016 - dk2016)**2)),
np.sqrt(np.sum((dk2016 - ee2016)**2)),
np.sqrt(np.sum((dk2016 - es2016)**2)),
np.sqrt(np.sum((dk2016 - fi2016)**2)),
np.sqrt(np.sum((dk2016 - fr2016)**2)),
np.sqrt(np.sum((dk2016 - gb2016)**2)),
np.sqrt(np.sum((dk2016 - gr2016)**2)),
np.sqrt(np.sum((dk2016 - hr2016)**2)),
np.sqrt(np.sum((dk2016 - hu2016)**2)),
np.sqrt(np.sum((dk2016 - ie2016)**2)),
np.sqrt(np.sum((dk2016 - it2016)**2)),
np.sqrt(np.sum((dk2016 - nl2016)**2)),
np.sqrt(np.sum((dk2016 - no2016)**2)),
np.sqrt(np.sum((dk2016 - pl2016)**2)),
np.sqrt(np.sum((dk2016 - pt2016)**2)),
np.sqrt(np.sum((dk2016 - se2016)**2)),
np.sqrt(np.sum((dk2016 - si2016)**2)),
np.sqrt(np.sum((dk2016 - sk2016)**2)),
np.sqrt(np.sum((dk2016 - tr2016)**2)),
])
ee2016_all =  np.array([
np.sqrt(np.sum((ee2016 - at2016)**2)),
np.sqrt(np.sum((ee2016 - be2016)**2)),
np.sqrt(np.sum((ee2016 - bg2016)**2)),
np.sqrt(np.sum((ee2016 - by2016)**2)),
np.sqrt(np.sum((ee2016 - ch2016)**2)),
np.sqrt(np.sum((ee2016 - cz2016)**2)),
np.sqrt(np.sum((ee2016 - de2016)**2)),
np.sqrt(np.sum((ee2016 - dk2016)**2)),
np.sqrt(np.sum((ee2016 - ee2016)**2)),
np.sqrt(np.sum((ee2016 - es2016)**2)),
np.sqrt(np.sum((ee2016 - fi2016)**2)),
np.sqrt(np.sum((ee2016 - fr2016)**2)),
np.sqrt(np.sum((ee2016 - gb2016)**2)),
np.sqrt(np.sum((ee2016 - gr2016)**2)),
np.sqrt(np.sum((ee2016 - hr2016)**2)),
np.sqrt(np.sum((ee2016 - hu2016)**2)),
np.sqrt(np.sum((ee2016 - ie2016)**2)),
np.sqrt(np.sum((ee2016 - it2016)**2)),
np.sqrt(np.sum((ee2016 - nl2016)**2)),
np.sqrt(np.sum((ee2016 - no2016)**2)),
np.sqrt(np.sum((ee2016 - pl2016)**2)),
np.sqrt(np.sum((ee2016 - pt2016)**2)),
np.sqrt(np.sum((ee2016 - se2016)**2)),
np.sqrt(np.sum((ee2016 - si2016)**2)),
np.sqrt(np.sum((ee2016 - sk2016)**2)),
np.sqrt(np.sum((ee2016 - tr2016)**2)),
])
es2016_all =  np.array([
np.sqrt(np.sum((es2016 - at2016)**2)),
np.sqrt(np.sum((es2016 - be2016)**2)),
np.sqrt(np.sum((es2016 - bg2016)**2)),
np.sqrt(np.sum((es2016 - by2016)**2)),
np.sqrt(np.sum((es2016 - ch2016)**2)),
np.sqrt(np.sum((es2016 - cz2016)**2)),
np.sqrt(np.sum((es2016 - de2016)**2)),
np.sqrt(np.sum((es2016 - dk2016)**2)),
np.sqrt(np.sum((es2016 - ee2016)**2)),
np.sqrt(np.sum((es2016 - es2016)**2)),
np.sqrt(np.sum((es2016 - fi2016)**2)),
np.sqrt(np.sum((es2016 - fr2016)**2)),
np.sqrt(np.sum((es2016 - gb2016)**2)),
np.sqrt(np.sum((es2016 - gr2016)**2)),
np.sqrt(np.sum((es2016 - hr2016)**2)),
np.sqrt(np.sum((es2016 - hu2016)**2)),
np.sqrt(np.sum((es2016 - ie2016)**2)),
np.sqrt(np.sum((es2016 - it2016)**2)),
np.sqrt(np.sum((es2016 - nl2016)**2)),
np.sqrt(np.sum((es2016 - no2016)**2)),
np.sqrt(np.sum((es2016 - pl2016)**2)),
np.sqrt(np.sum((es2016 - pt2016)**2)),
np.sqrt(np.sum((es2016 - se2016)**2)),
np.sqrt(np.sum((es2016 - si2016)**2)),
np.sqrt(np.sum((es2016 - sk2016)**2)),
np.sqrt(np.sum((es2016 - tr2016)**2)),
])
fi2016_all =  np.array([
np.sqrt(np.sum((fi2016 - at2016)**2)),
np.sqrt(np.sum((fi2016 - be2016)**2)),
np.sqrt(np.sum((fi2016 - bg2016)**2)),
np.sqrt(np.sum((fi2016 - by2016)**2)),
np.sqrt(np.sum((fi2016 - ch2016)**2)),
np.sqrt(np.sum((fi2016 - cz2016)**2)),
np.sqrt(np.sum((fi2016 - de2016)**2)),
np.sqrt(np.sum((fi2016 - dk2016)**2)),
np.sqrt(np.sum((fi2016 - ee2016)**2)),
np.sqrt(np.sum((fi2016 - es2016)**2)),
np.sqrt(np.sum((fi2016 - fi2016)**2)),
np.sqrt(np.sum((fi2016 - fr2016)**2)),
np.sqrt(np.sum((fi2016 - gb2016)**2)),
np.sqrt(np.sum((fi2016 - gr2016)**2)),
np.sqrt(np.sum((fi2016 - hr2016)**2)),
np.sqrt(np.sum((fi2016 - hu2016)**2)),
np.sqrt(np.sum((fi2016 - ie2016)**2)),
np.sqrt(np.sum((fi2016 - it2016)**2)),
np.sqrt(np.sum((fi2016 - nl2016)**2)),
np.sqrt(np.sum((fi2016 - no2016)**2)),
np.sqrt(np.sum((fi2016 - pl2016)**2)),
np.sqrt(np.sum((fi2016 - pt2016)**2)),
np.sqrt(np.sum((fi2016 - se2016)**2)),
np.sqrt(np.sum((fi2016 - si2016)**2)),
np.sqrt(np.sum((fi2016 - sk2016)**2)),
np.sqrt(np.sum((fi2016 - tr2016)**2)),
])
fr2016_all =  np.array([
np.sqrt(np.sum((fr2016 - at2016)**2)),
np.sqrt(np.sum((fr2016 - be2016)**2)),
np.sqrt(np.sum((fr2016 - bg2016)**2)),
np.sqrt(np.sum((fr2016 - by2016)**2)),
np.sqrt(np.sum((fr2016 - ch2016)**2)),
np.sqrt(np.sum((fr2016 - cz2016)**2)),
np.sqrt(np.sum((fr2016 - de2016)**2)),
np.sqrt(np.sum((fr2016 - dk2016)**2)),
np.sqrt(np.sum((fr2016 - ee2016)**2)),
np.sqrt(np.sum((fr2016 - es2016)**2)),
np.sqrt(np.sum((fr2016 - fi2016)**2)),
np.sqrt(np.sum((fr2016 - fr2016)**2)),
np.sqrt(np.sum((fr2016 - gb2016)**2)),
np.sqrt(np.sum((fr2016 - gr2016)**2)),
np.sqrt(np.sum((fr2016 - hr2016)**2)),
np.sqrt(np.sum((fr2016 - hu2016)**2)),
np.sqrt(np.sum((fr2016 - ie2016)**2)),
np.sqrt(np.sum((fr2016 - it2016)**2)),
np.sqrt(np.sum((fr2016 - nl2016)**2)),
np.sqrt(np.sum((fr2016 - no2016)**2)),
np.sqrt(np.sum((fr2016 - pl2016)**2)),
np.sqrt(np.sum((fr2016 - pt2016)**2)),
np.sqrt(np.sum((fr2016 - se2016)**2)),
np.sqrt(np.sum((fr2016 - si2016)**2)),
np.sqrt(np.sum((fr2016 - sk2016)**2)),
np.sqrt(np.sum((fr2016 - tr2016)**2)),
])
gb2016_all =  np.array([
np.sqrt(np.sum((gb2016 - at2016)**2)),
np.sqrt(np.sum((gb2016 - be2016)**2)),
np.sqrt(np.sum((gb2016 - bg2016)**2)),
np.sqrt(np.sum((gb2016 - by2016)**2)),
np.sqrt(np.sum((gb2016 - ch2016)**2)),
np.sqrt(np.sum((gb2016 - cz2016)**2)),
np.sqrt(np.sum((gb2016 - de2016)**2)),
np.sqrt(np.sum((gb2016 - dk2016)**2)),
np.sqrt(np.sum((gb2016 - ee2016)**2)),
np.sqrt(np.sum((gb2016 - es2016)**2)),
np.sqrt(np.sum((gb2016 - fi2016)**2)),
np.sqrt(np.sum((gb2016 - fr2016)**2)),
np.sqrt(np.sum((gb2016 - gb2016)**2)),
np.sqrt(np.sum((gb2016 - gr2016)**2)),
np.sqrt(np.sum((gb2016 - hr2016)**2)),
np.sqrt(np.sum((gb2016 - hu2016)**2)),
np.sqrt(np.sum((gb2016 - ie2016)**2)),
np.sqrt(np.sum((gb2016 - it2016)**2)),
np.sqrt(np.sum((gb2016 - nl2016)**2)),
np.sqrt(np.sum((gb2016 - no2016)**2)),
np.sqrt(np.sum((gb2016 - pl2016)**2)),
np.sqrt(np.sum((gb2016 - pt2016)**2)),
np.sqrt(np.sum((gb2016 - se2016)**2)),
np.sqrt(np.sum((gb2016 - si2016)**2)),
np.sqrt(np.sum((gb2016 - sk2016)**2)),
np.sqrt(np.sum((gb2016 - tr2016)**2)),
])
gr2016_all =  np.array([
np.sqrt(np.sum((gr2016 - at2016)**2)),
np.sqrt(np.sum((gr2016 - be2016)**2)),
np.sqrt(np.sum((gr2016 - bg2016)**2)),
np.sqrt(np.sum((gr2016 - by2016)**2)),
np.sqrt(np.sum((gr2016 - ch2016)**2)),
np.sqrt(np.sum((gr2016 - cz2016)**2)),
np.sqrt(np.sum((gr2016 - de2016)**2)),
np.sqrt(np.sum((gr2016 - dk2016)**2)),
np.sqrt(np.sum((gr2016 - ee2016)**2)),
np.sqrt(np.sum((gr2016 - es2016)**2)),
np.sqrt(np.sum((gr2016 - fi2016)**2)),
np.sqrt(np.sum((gr2016 - fr2016)**2)),
np.sqrt(np.sum((gr2016 - gb2016)**2)),
np.sqrt(np.sum((gr2016 - gr2016)**2)),
np.sqrt(np.sum((gr2016 - hr2016)**2)),
np.sqrt(np.sum((gr2016 - hu2016)**2)),
np.sqrt(np.sum((gr2016 - ie2016)**2)),
np.sqrt(np.sum((gr2016 - it2016)**2)),
np.sqrt(np.sum((gr2016 - nl2016)**2)),
np.sqrt(np.sum((gr2016 - no2016)**2)),
np.sqrt(np.sum((gr2016 - pl2016)**2)),
np.sqrt(np.sum((gr2016 - pt2016)**2)),
np.sqrt(np.sum((gr2016 - se2016)**2)),
np.sqrt(np.sum((gr2016 - si2016)**2)),
np.sqrt(np.sum((gr2016 - sk2016)**2)),
np.sqrt(np.sum((gr2016 - tr2016)**2)),
])
hr2016_all =  np.array([
np.sqrt(np.sum((hr2016 - at2016)**2)),
np.sqrt(np.sum((hr2016 - be2016)**2)),
np.sqrt(np.sum((hr2016 - bg2016)**2)),
np.sqrt(np.sum((hr2016 - by2016)**2)),
np.sqrt(np.sum((hr2016 - ch2016)**2)),
np.sqrt(np.sum((hr2016 - cz2016)**2)),
np.sqrt(np.sum((hr2016 - de2016)**2)),
np.sqrt(np.sum((hr2016 - dk2016)**2)),
np.sqrt(np.sum((hr2016 - ee2016)**2)),
np.sqrt(np.sum((hr2016 - es2016)**2)),
np.sqrt(np.sum((hr2016 - fi2016)**2)),
np.sqrt(np.sum((hr2016 - fr2016)**2)),
np.sqrt(np.sum((hr2016 - gb2016)**2)),
np.sqrt(np.sum((hr2016 - gr2016)**2)),
np.sqrt(np.sum((hr2016 - hr2016)**2)),
np.sqrt(np.sum((hr2016 - hu2016)**2)),
np.sqrt(np.sum((hr2016 - ie2016)**2)),
np.sqrt(np.sum((hr2016 - it2016)**2)),
np.sqrt(np.sum((hr2016 - nl2016)**2)),
np.sqrt(np.sum((hr2016 - no2016)**2)),
np.sqrt(np.sum((hr2016 - pl2016)**2)),
np.sqrt(np.sum((hr2016 - pt2016)**2)),
np.sqrt(np.sum((hr2016 - se2016)**2)),
np.sqrt(np.sum((hr2016 - si2016)**2)),
np.sqrt(np.sum((hr2016 - sk2016)**2)),
np.sqrt(np.sum((hr2016 - tr2016)**2)),
])
hu2016_all =  np.array([
np.sqrt(np.sum((hu2016 - at2016)**2)),
np.sqrt(np.sum((hu2016 - be2016)**2)),
np.sqrt(np.sum((hu2016 - bg2016)**2)),
np.sqrt(np.sum((hu2016 - by2016)**2)),
np.sqrt(np.sum((hu2016 - ch2016)**2)),
np.sqrt(np.sum((hu2016 - cz2016)**2)),
np.sqrt(np.sum((hu2016 - de2016)**2)),
np.sqrt(np.sum((hu2016 - dk2016)**2)),
np.sqrt(np.sum((hu2016 - ee2016)**2)),
np.sqrt(np.sum((hu2016 - es2016)**2)),
np.sqrt(np.sum((hu2016 - fi2016)**2)),
np.sqrt(np.sum((hu2016 - fr2016)**2)),
np.sqrt(np.sum((hu2016 - gb2016)**2)),
np.sqrt(np.sum((hu2016 - gr2016)**2)),
np.sqrt(np.sum((hu2016 - hr2016)**2)),
np.sqrt(np.sum((hu2016 - hu2016)**2)),
np.sqrt(np.sum((hu2016 - ie2016)**2)),
np.sqrt(np.sum((hu2016 - it2016)**2)),
np.sqrt(np.sum((hu2016 - nl2016)**2)),
np.sqrt(np.sum((hu2016 - no2016)**2)),
np.sqrt(np.sum((hu2016 - pl2016)**2)),
np.sqrt(np.sum((hu2016 - pt2016)**2)),
np.sqrt(np.sum((hu2016 - se2016)**2)),
np.sqrt(np.sum((hu2016 - si2016)**2)),
np.sqrt(np.sum((hu2016 - sk2016)**2)),
np.sqrt(np.sum((hu2016 - tr2016)**2)),
])
ie2016_all =  np.array([
np.sqrt(np.sum((ie2016 - at2016)**2)),
np.sqrt(np.sum((ie2016 - be2016)**2)),
np.sqrt(np.sum((ie2016 - bg2016)**2)),
np.sqrt(np.sum((ie2016 - by2016)**2)),
np.sqrt(np.sum((ie2016 - ch2016)**2)),
np.sqrt(np.sum((ie2016 - cz2016)**2)),
np.sqrt(np.sum((ie2016 - de2016)**2)),
np.sqrt(np.sum((ie2016 - dk2016)**2)),
np.sqrt(np.sum((ie2016 - ee2016)**2)),
np.sqrt(np.sum((ie2016 - es2016)**2)),
np.sqrt(np.sum((ie2016 - fi2016)**2)),
np.sqrt(np.sum((ie2016 - fr2016)**2)),
np.sqrt(np.sum((ie2016 - gb2016)**2)),
np.sqrt(np.sum((ie2016 - gr2016)**2)),
np.sqrt(np.sum((ie2016 - hr2016)**2)),
np.sqrt(np.sum((ie2016 - hu2016)**2)),
np.sqrt(np.sum((ie2016 - ie2016)**2)),
np.sqrt(np.sum((ie2016 - it2016)**2)),
np.sqrt(np.sum((ie2016 - nl2016)**2)),
np.sqrt(np.sum((ie2016 - no2016)**2)),
np.sqrt(np.sum((ie2016 - pl2016)**2)),
np.sqrt(np.sum((ie2016 - pt2016)**2)),
np.sqrt(np.sum((ie2016 - se2016)**2)),
np.sqrt(np.sum((ie2016 - si2016)**2)),
np.sqrt(np.sum((ie2016 - sk2016)**2)),
np.sqrt(np.sum((ie2016 - tr2016)**2)),
])
it2016_all =  np.array([
np.sqrt(np.sum((it2016 - at2016)**2)),
np.sqrt(np.sum((it2016 - be2016)**2)),
np.sqrt(np.sum((it2016 - bg2016)**2)),
np.sqrt(np.sum((it2016 - by2016)**2)),
np.sqrt(np.sum((it2016 - ch2016)**2)),
np.sqrt(np.sum((it2016 - cz2016)**2)),
np.sqrt(np.sum((it2016 - de2016)**2)),
np.sqrt(np.sum((it2016 - dk2016)**2)),
np.sqrt(np.sum((it2016 - ee2016)**2)),
np.sqrt(np.sum((it2016 - es2016)**2)),
np.sqrt(np.sum((it2016 - fi2016)**2)),
np.sqrt(np.sum((it2016 - fr2016)**2)),
np.sqrt(np.sum((it2016 - gb2016)**2)),
np.sqrt(np.sum((it2016 - gr2016)**2)),
np.sqrt(np.sum((it2016 - hr2016)**2)),
np.sqrt(np.sum((it2016 - hu2016)**2)),
np.sqrt(np.sum((it2016 - ie2016)**2)),
np.sqrt(np.sum((it2016 - it2016)**2)),
np.sqrt(np.sum((it2016 - nl2016)**2)),
np.sqrt(np.sum((it2016 - no2016)**2)),
np.sqrt(np.sum((it2016 - pl2016)**2)),
np.sqrt(np.sum((it2016 - pt2016)**2)),
np.sqrt(np.sum((it2016 - se2016)**2)),
np.sqrt(np.sum((it2016 - si2016)**2)),
np.sqrt(np.sum((it2016 - sk2016)**2)),
np.sqrt(np.sum((it2016 - tr2016)**2)),
])
nl2016_all =  np.array([
np.sqrt(np.sum((nl2016 - at2016)**2)),
np.sqrt(np.sum((nl2016 - be2016)**2)),
np.sqrt(np.sum((nl2016 - bg2016)**2)),
np.sqrt(np.sum((nl2016 - by2016)**2)),
np.sqrt(np.sum((nl2016 - ch2016)**2)),
np.sqrt(np.sum((nl2016 - cz2016)**2)),
np.sqrt(np.sum((nl2016 - de2016)**2)),
np.sqrt(np.sum((nl2016 - dk2016)**2)),
np.sqrt(np.sum((nl2016 - ee2016)**2)),
np.sqrt(np.sum((nl2016 - es2016)**2)),
np.sqrt(np.sum((nl2016 - fi2016)**2)),
np.sqrt(np.sum((nl2016 - fr2016)**2)),
np.sqrt(np.sum((nl2016 - gb2016)**2)),
np.sqrt(np.sum((nl2016 - gr2016)**2)),
np.sqrt(np.sum((nl2016 - hr2016)**2)),
np.sqrt(np.sum((nl2016 - hu2016)**2)),
np.sqrt(np.sum((nl2016 - ie2016)**2)),
np.sqrt(np.sum((nl2016 - it2016)**2)),
np.sqrt(np.sum((nl2016 - nl2016)**2)),
np.sqrt(np.sum((nl2016 - no2016)**2)),
np.sqrt(np.sum((nl2016 - pl2016)**2)),
np.sqrt(np.sum((nl2016 - pt2016)**2)),
np.sqrt(np.sum((nl2016 - se2016)**2)),
np.sqrt(np.sum((nl2016 - si2016)**2)),
np.sqrt(np.sum((nl2016 - sk2016)**2)),
np.sqrt(np.sum((nl2016 - tr2016)**2)),
])
no2016_all =  np.array([
np.sqrt(np.sum((no2016 - at2016)**2)),
np.sqrt(np.sum((no2016 - be2016)**2)),
np.sqrt(np.sum((no2016 - bg2016)**2)),
np.sqrt(np.sum((no2016 - by2016)**2)),
np.sqrt(np.sum((no2016 - ch2016)**2)),
np.sqrt(np.sum((no2016 - cz2016)**2)),
np.sqrt(np.sum((no2016 - de2016)**2)),
np.sqrt(np.sum((no2016 - dk2016)**2)),
np.sqrt(np.sum((no2016 - ee2016)**2)),
np.sqrt(np.sum((no2016 - es2016)**2)),
np.sqrt(np.sum((no2016 - fi2016)**2)),
np.sqrt(np.sum((no2016 - fr2016)**2)),
np.sqrt(np.sum((no2016 - gb2016)**2)),
np.sqrt(np.sum((no2016 - gr2016)**2)),
np.sqrt(np.sum((no2016 - hr2016)**2)),
np.sqrt(np.sum((no2016 - hu2016)**2)),
np.sqrt(np.sum((no2016 - ie2016)**2)),
np.sqrt(np.sum((no2016 - it2016)**2)),
np.sqrt(np.sum((no2016 - nl2016)**2)),
np.sqrt(np.sum((no2016 - no2016)**2)),
np.sqrt(np.sum((no2016 - pl2016)**2)),
np.sqrt(np.sum((no2016 - pt2016)**2)),
np.sqrt(np.sum((no2016 - se2016)**2)),
np.sqrt(np.sum((no2016 - si2016)**2)),
np.sqrt(np.sum((no2016 - sk2016)**2)),
np.sqrt(np.sum((no2016 - tr2016)**2)),
])
pl2016_all =  np.array([
np.sqrt(np.sum((pl2016 - at2016)**2)),
np.sqrt(np.sum((pl2016 - be2016)**2)),
np.sqrt(np.sum((pl2016 - bg2016)**2)),
np.sqrt(np.sum((pl2016 - by2016)**2)),
np.sqrt(np.sum((pl2016 - ch2016)**2)),
np.sqrt(np.sum((pl2016 - cz2016)**2)),
np.sqrt(np.sum((pl2016 - de2016)**2)),
np.sqrt(np.sum((pl2016 - dk2016)**2)),
np.sqrt(np.sum((pl2016 - ee2016)**2)),
np.sqrt(np.sum((pl2016 - es2016)**2)),
np.sqrt(np.sum((pl2016 - fi2016)**2)),
np.sqrt(np.sum((pl2016 - fr2016)**2)),
np.sqrt(np.sum((pl2016 - gb2016)**2)),
np.sqrt(np.sum((pl2016 - gr2016)**2)),
np.sqrt(np.sum((pl2016 - hr2016)**2)),
np.sqrt(np.sum((pl2016 - hu2016)**2)),
np.sqrt(np.sum((pl2016 - ie2016)**2)),
np.sqrt(np.sum((pl2016 - it2016)**2)),
np.sqrt(np.sum((pl2016 - nl2016)**2)),
np.sqrt(np.sum((pl2016 - no2016)**2)),
np.sqrt(np.sum((pl2016 - pl2016)**2)),
np.sqrt(np.sum((pl2016 - pt2016)**2)),
np.sqrt(np.sum((pl2016 - se2016)**2)),
np.sqrt(np.sum((pl2016 - si2016)**2)),
np.sqrt(np.sum((pl2016 - sk2016)**2)),
np.sqrt(np.sum((pl2016 - tr2016)**2)),
])
pt2016_all =  np.array([
np.sqrt(np.sum((pt2016 - at2016)**2)),
np.sqrt(np.sum((pt2016 - be2016)**2)),
np.sqrt(np.sum((pt2016 - bg2016)**2)),
np.sqrt(np.sum((pt2016 - by2016)**2)),
np.sqrt(np.sum((pt2016 - ch2016)**2)),
np.sqrt(np.sum((pt2016 - cz2016)**2)),
np.sqrt(np.sum((pt2016 - de2016)**2)),
np.sqrt(np.sum((pt2016 - dk2016)**2)),
np.sqrt(np.sum((pt2016 - ee2016)**2)),
np.sqrt(np.sum((pt2016 - es2016)**2)),
np.sqrt(np.sum((pt2016 - fi2016)**2)),
np.sqrt(np.sum((pt2016 - fr2016)**2)),
np.sqrt(np.sum((pt2016 - gb2016)**2)),
np.sqrt(np.sum((pt2016 - gr2016)**2)),
np.sqrt(np.sum((pt2016 - hr2016)**2)),
np.sqrt(np.sum((pt2016 - hu2016)**2)),
np.sqrt(np.sum((pt2016 - ie2016)**2)),
np.sqrt(np.sum((pt2016 - it2016)**2)),
np.sqrt(np.sum((pt2016 - nl2016)**2)),
np.sqrt(np.sum((pt2016 - no2016)**2)),
np.sqrt(np.sum((pt2016 - pl2016)**2)),
np.sqrt(np.sum((pt2016 - pt2016)**2)),
np.sqrt(np.sum((pt2016 - se2016)**2)),
np.sqrt(np.sum((pt2016 - si2016)**2)),
np.sqrt(np.sum((pt2016 - sk2016)**2)),
np.sqrt(np.sum((pt2016 - tr2016)**2)),
])
se2016_all =  np.array([
np.sqrt(np.sum((se2016 - at2016)**2)),
np.sqrt(np.sum((se2016 - be2016)**2)),
np.sqrt(np.sum((se2016 - bg2016)**2)),
np.sqrt(np.sum((se2016 - by2016)**2)),
np.sqrt(np.sum((se2016 - ch2016)**2)),
np.sqrt(np.sum((se2016 - cz2016)**2)),
np.sqrt(np.sum((se2016 - de2016)**2)),
np.sqrt(np.sum((se2016 - dk2016)**2)),
np.sqrt(np.sum((se2016 - ee2016)**2)),
np.sqrt(np.sum((se2016 - es2016)**2)),
np.sqrt(np.sum((se2016 - fi2016)**2)),
np.sqrt(np.sum((se2016 - fr2016)**2)),
np.sqrt(np.sum((se2016 - gb2016)**2)),
np.sqrt(np.sum((se2016 - gr2016)**2)),
np.sqrt(np.sum((se2016 - hr2016)**2)),
np.sqrt(np.sum((se2016 - hu2016)**2)),
np.sqrt(np.sum((se2016 - ie2016)**2)),
np.sqrt(np.sum((se2016 - it2016)**2)),
np.sqrt(np.sum((se2016 - nl2016)**2)),
np.sqrt(np.sum((se2016 - no2016)**2)),
np.sqrt(np.sum((se2016 - pl2016)**2)),
np.sqrt(np.sum((se2016 - pt2016)**2)),
np.sqrt(np.sum((se2016 - se2016)**2)),
np.sqrt(np.sum((se2016 - si2016)**2)),
np.sqrt(np.sum((se2016 - sk2016)**2)),
np.sqrt(np.sum((se2016 - tr2016)**2)),
])
si2016_all =  np.array([
np.sqrt(np.sum((si2016 - at2016)**2)),
np.sqrt(np.sum((si2016 - be2016)**2)),
np.sqrt(np.sum((si2016 - bg2016)**2)),
np.sqrt(np.sum((si2016 - by2016)**2)),
np.sqrt(np.sum((si2016 - ch2016)**2)),
np.sqrt(np.sum((si2016 - cz2016)**2)),
np.sqrt(np.sum((si2016 - de2016)**2)),
np.sqrt(np.sum((si2016 - dk2016)**2)),
np.sqrt(np.sum((si2016 - ee2016)**2)),
np.sqrt(np.sum((si2016 - es2016)**2)),
np.sqrt(np.sum((si2016 - fi2016)**2)),
np.sqrt(np.sum((si2016 - fr2016)**2)),
np.sqrt(np.sum((si2016 - gb2016)**2)),
np.sqrt(np.sum((si2016 - gr2016)**2)),
np.sqrt(np.sum((si2016 - hr2016)**2)),
np.sqrt(np.sum((si2016 - hu2016)**2)),
np.sqrt(np.sum((si2016 - ie2016)**2)),
np.sqrt(np.sum((si2016 - it2016)**2)),
np.sqrt(np.sum((si2016 - nl2016)**2)),
np.sqrt(np.sum((si2016 - no2016)**2)),
np.sqrt(np.sum((si2016 - pl2016)**2)),
np.sqrt(np.sum((si2016 - pt2016)**2)),
np.sqrt(np.sum((si2016 - se2016)**2)),
np.sqrt(np.sum((si2016 - si2016)**2)),
np.sqrt(np.sum((si2016 - sk2016)**2)),
np.sqrt(np.sum((si2016 - tr2016)**2)),
])
sk2016_all =  np.array([
np.sqrt(np.sum((sk2016 - at2016)**2)),
np.sqrt(np.sum((sk2016 - be2016)**2)),
np.sqrt(np.sum((sk2016 - bg2016)**2)),
np.sqrt(np.sum((sk2016 - by2016)**2)),
np.sqrt(np.sum((sk2016 - ch2016)**2)),
np.sqrt(np.sum((sk2016 - cz2016)**2)),
np.sqrt(np.sum((sk2016 - de2016)**2)),
np.sqrt(np.sum((sk2016 - dk2016)**2)),
np.sqrt(np.sum((sk2016 - ee2016)**2)),
np.sqrt(np.sum((sk2016 - es2016)**2)),
np.sqrt(np.sum((sk2016 - fi2016)**2)),
np.sqrt(np.sum((sk2016 - fr2016)**2)),
np.sqrt(np.sum((sk2016 - gb2016)**2)),
np.sqrt(np.sum((sk2016 - gr2016)**2)),
np.sqrt(np.sum((sk2016 - hr2016)**2)),
np.sqrt(np.sum((sk2016 - hu2016)**2)),
np.sqrt(np.sum((sk2016 - ie2016)**2)),
np.sqrt(np.sum((sk2016 - it2016)**2)),
np.sqrt(np.sum((sk2016 - nl2016)**2)),
np.sqrt(np.sum((sk2016 - no2016)**2)),
np.sqrt(np.sum((sk2016 - pl2016)**2)),
np.sqrt(np.sum((sk2016 - pt2016)**2)),
np.sqrt(np.sum((sk2016 - se2016)**2)),
np.sqrt(np.sum((sk2016 - si2016)**2)),
np.sqrt(np.sum((sk2016 - sk2016)**2)),
np.sqrt(np.sum((sk2016 - tr2016)**2)),
])
tr2016_all =  np.array([
np.sqrt(np.sum((tr2016 - at2016)**2)),
np.sqrt(np.sum((tr2016 - be2016)**2)),
np.sqrt(np.sum((tr2016 - bg2016)**2)),
np.sqrt(np.sum((tr2016 - by2016)**2)),
np.sqrt(np.sum((tr2016 - ch2016)**2)),
np.sqrt(np.sum((tr2016 - cz2016)**2)),
np.sqrt(np.sum((tr2016 - de2016)**2)),
np.sqrt(np.sum((tr2016 - dk2016)**2)),
np.sqrt(np.sum((tr2016 - ee2016)**2)),
np.sqrt(np.sum((tr2016 - es2016)**2)),
np.sqrt(np.sum((tr2016 - fi2016)**2)),
np.sqrt(np.sum((tr2016 - fr2016)**2)),
np.sqrt(np.sum((tr2016 - gb2016)**2)),
np.sqrt(np.sum((tr2016 - gr2016)**2)),
np.sqrt(np.sum((tr2016 - hr2016)**2)),
np.sqrt(np.sum((tr2016 - hu2016)**2)),
np.sqrt(np.sum((tr2016 - ie2016)**2)),
np.sqrt(np.sum((tr2016 - it2016)**2)),
np.sqrt(np.sum((tr2016 - nl2016)**2)),
np.sqrt(np.sum((tr2016 - no2016)**2)),
np.sqrt(np.sum((tr2016 - pl2016)**2)),
np.sqrt(np.sum((tr2016 - pt2016)**2)),
np.sqrt(np.sum((tr2016 - se2016)**2)),
np.sqrt(np.sum((tr2016 - si2016)**2)),
np.sqrt(np.sum((tr2016 - sk2016)**2)),
np.sqrt(np.sum((tr2016 - tr2016)**2)),
])

In [ ]:
#2017
at2017_all =  np.array([
np.sqrt(np.sum((at2017 - at2017)**2)),
np.sqrt(np.sum((at2017 - be2017)**2)),
np.sqrt(np.sum((at2017 - bg2017)**2)),
np.sqrt(np.sum((at2017 - by2017)**2)),
np.sqrt(np.sum((at2017 - ch2017)**2)),
np.sqrt(np.sum((at2017 - cz2017)**2)),
np.sqrt(np.sum((at2017 - de2017)**2)),
np.sqrt(np.sum((at2017 - dk2017)**2)),
np.sqrt(np.sum((at2017 - ee2017)**2)),
np.sqrt(np.sum((at2017 - es2017)**2)),
np.sqrt(np.sum((at2017 - fi2017)**2)),
np.sqrt(np.sum((at2017 - fr2017)**2)),
np.sqrt(np.sum((at2017 - gb2017)**2)),
np.sqrt(np.sum((at2017 - gr2017)**2)),
np.sqrt(np.sum((at2017 - hr2017)**2)),
np.sqrt(np.sum((at2017 - hu2017)**2)),
np.sqrt(np.sum((at2017 - ie2017)**2)),
np.sqrt(np.sum((at2017 - it2017)**2)),
np.sqrt(np.sum((at2017 - nl2017)**2)),
np.sqrt(np.sum((at2017 - no2017)**2)),
np.sqrt(np.sum((at2017 - pl2017)**2)),
np.sqrt(np.sum((at2017 - pt2017)**2)),
np.sqrt(np.sum((at2017 - se2017)**2)),
np.sqrt(np.sum((at2017 - si2017)**2)),
np.sqrt(np.sum((at2017 - sk2017)**2)),
np.sqrt(np.sum((at2017 - tr2017)**2)),
])
be2017_all =  np.array([
np.sqrt(np.sum((be2017 - at2017)**2)),
np.sqrt(np.sum((be2017 - be2017)**2)),
np.sqrt(np.sum((be2017 - bg2017)**2)),
np.sqrt(np.sum((be2017 - by2017)**2)),
np.sqrt(np.sum((be2017 - ch2017)**2)),
np.sqrt(np.sum((be2017 - cz2017)**2)),
np.sqrt(np.sum((be2017 - de2017)**2)),
np.sqrt(np.sum((be2017 - dk2017)**2)),
np.sqrt(np.sum((be2017 - ee2017)**2)),
np.sqrt(np.sum((be2017 - es2017)**2)),
np.sqrt(np.sum((be2017 - fi2017)**2)),
np.sqrt(np.sum((be2017 - fr2017)**2)),
np.sqrt(np.sum((be2017 - gb2017)**2)),
np.sqrt(np.sum((be2017 - gr2017)**2)),
np.sqrt(np.sum((be2017 - hr2017)**2)),
np.sqrt(np.sum((be2017 - hu2017)**2)),
np.sqrt(np.sum((be2017 - ie2017)**2)),
np.sqrt(np.sum((be2017 - it2017)**2)),
np.sqrt(np.sum((be2017 - nl2017)**2)),
np.sqrt(np.sum((be2017 - no2017)**2)),
np.sqrt(np.sum((be2017 - pl2017)**2)),
np.sqrt(np.sum((be2017 - pt2017)**2)),
np.sqrt(np.sum((be2017 - se2017)**2)),
np.sqrt(np.sum((be2017 - si2017)**2)),
np.sqrt(np.sum((be2017 - sk2017)**2)),
np.sqrt(np.sum((be2017 - tr2017)**2)),
])
bg2017_all =  np.array([
np.sqrt(np.sum((bg2017 - at2017)**2)),
np.sqrt(np.sum((bg2017 - be2017)**2)),
np.sqrt(np.sum((bg2017 - bg2017)**2)),
np.sqrt(np.sum((bg2017 - by2017)**2)),
np.sqrt(np.sum((bg2017 - ch2017)**2)),
np.sqrt(np.sum((bg2017 - cz2017)**2)),
np.sqrt(np.sum((bg2017 - de2017)**2)),
np.sqrt(np.sum((bg2017 - dk2017)**2)),
np.sqrt(np.sum((bg2017 - ee2017)**2)),
np.sqrt(np.sum((bg2017 - es2017)**2)),
np.sqrt(np.sum((bg2017 - fi2017)**2)),
np.sqrt(np.sum((bg2017 - fr2017)**2)),
np.sqrt(np.sum((bg2017 - gb2017)**2)),
np.sqrt(np.sum((bg2017 - gr2017)**2)),
np.sqrt(np.sum((bg2017 - hr2017)**2)),
np.sqrt(np.sum((bg2017 - hu2017)**2)),
np.sqrt(np.sum((bg2017 - ie2017)**2)),
np.sqrt(np.sum((bg2017 - it2017)**2)),
np.sqrt(np.sum((bg2017 - nl2017)**2)),
np.sqrt(np.sum((bg2017 - no2017)**2)),
np.sqrt(np.sum((bg2017 - pl2017)**2)),
np.sqrt(np.sum((bg2017 - pt2017)**2)),
np.sqrt(np.sum((bg2017 - se2017)**2)),
np.sqrt(np.sum((bg2017 - si2017)**2)),
np.sqrt(np.sum((bg2017 - sk2017)**2)),
np.sqrt(np.sum((bg2017 - tr2017)**2)),
])
by2017_all =  np.array([
np.sqrt(np.sum((by2017 - at2017)**2)),
np.sqrt(np.sum((by2017 - be2017)**2)),
np.sqrt(np.sum((by2017 - bg2017)**2)),
np.sqrt(np.sum((by2017 - by2017)**2)),
np.sqrt(np.sum((by2017 - ch2017)**2)),
np.sqrt(np.sum((by2017 - cz2017)**2)),
np.sqrt(np.sum((by2017 - de2017)**2)),
np.sqrt(np.sum((by2017 - dk2017)**2)),
np.sqrt(np.sum((by2017 - ee2017)**2)),
np.sqrt(np.sum((by2017 - es2017)**2)),
np.sqrt(np.sum((by2017 - fi2017)**2)),
np.sqrt(np.sum((by2017 - fr2017)**2)),
np.sqrt(np.sum((by2017 - gb2017)**2)),
np.sqrt(np.sum((by2017 - gr2017)**2)),
np.sqrt(np.sum((by2017 - hr2017)**2)),
np.sqrt(np.sum((by2017 - hu2017)**2)),
np.sqrt(np.sum((by2017 - ie2017)**2)),
np.sqrt(np.sum((by2017 - it2017)**2)),
np.sqrt(np.sum((by2017 - nl2017)**2)),
np.sqrt(np.sum((by2017 - no2017)**2)),
np.sqrt(np.sum((by2017 - pl2017)**2)),
np.sqrt(np.sum((by2017 - pt2017)**2)),
np.sqrt(np.sum((by2017 - se2017)**2)),
np.sqrt(np.sum((by2017 - si2017)**2)),
np.sqrt(np.sum((by2017 - sk2017)**2)),
np.sqrt(np.sum((by2017 - tr2017)**2)),
])
ch2017_all =  np.array([
np.sqrt(np.sum((ch2017 - at2017)**2)),
np.sqrt(np.sum((ch2017 - be2017)**2)),
np.sqrt(np.sum((ch2017 - bg2017)**2)),
np.sqrt(np.sum((ch2017 - by2017)**2)),
np.sqrt(np.sum((ch2017 - ch2017)**2)),
np.sqrt(np.sum((ch2017 - cz2017)**2)),
np.sqrt(np.sum((ch2017 - de2017)**2)),
np.sqrt(np.sum((ch2017 - dk2017)**2)),
np.sqrt(np.sum((ch2017 - ee2017)**2)),
np.sqrt(np.sum((ch2017 - es2017)**2)),
np.sqrt(np.sum((ch2017 - fi2017)**2)),
np.sqrt(np.sum((ch2017 - fr2017)**2)),
np.sqrt(np.sum((ch2017 - gb2017)**2)),
np.sqrt(np.sum((ch2017 - gr2017)**2)),
np.sqrt(np.sum((ch2017 - hr2017)**2)),
np.sqrt(np.sum((ch2017 - hu2017)**2)),
np.sqrt(np.sum((ch2017 - ie2017)**2)),
np.sqrt(np.sum((ch2017 - it2017)**2)),
np.sqrt(np.sum((ch2017 - nl2017)**2)),
np.sqrt(np.sum((ch2017 - no2017)**2)),
np.sqrt(np.sum((ch2017 - pl2017)**2)),
np.sqrt(np.sum((ch2017 - pt2017)**2)),
np.sqrt(np.sum((ch2017 - se2017)**2)),
np.sqrt(np.sum((ch2017 - si2017)**2)),
np.sqrt(np.sum((ch2017 - sk2017)**2)),
np.sqrt(np.sum((ch2017 - tr2017)**2)),
])
cz2017_all =  np.array([
np.sqrt(np.sum((cz2017 - at2017)**2)),
np.sqrt(np.sum((cz2017 - be2017)**2)),
np.sqrt(np.sum((cz2017 - bg2017)**2)),
np.sqrt(np.sum((cz2017 - by2017)**2)),
np.sqrt(np.sum((cz2017 - ch2017)**2)),
np.sqrt(np.sum((cz2017 - cz2017)**2)),
np.sqrt(np.sum((cz2017 - de2017)**2)),
np.sqrt(np.sum((cz2017 - dk2017)**2)),
np.sqrt(np.sum((cz2017 - ee2017)**2)),
np.sqrt(np.sum((cz2017 - es2017)**2)),
np.sqrt(np.sum((cz2017 - fi2017)**2)),
np.sqrt(np.sum((cz2017 - fr2017)**2)),
np.sqrt(np.sum((cz2017 - gb2017)**2)),
np.sqrt(np.sum((cz2017 - gr2017)**2)),
np.sqrt(np.sum((cz2017 - hr2017)**2)),
np.sqrt(np.sum((cz2017 - hu2017)**2)),
np.sqrt(np.sum((cz2017 - ie2017)**2)),
np.sqrt(np.sum((cz2017 - it2017)**2)),
np.sqrt(np.sum((cz2017 - nl2017)**2)),
np.sqrt(np.sum((cz2017 - no2017)**2)),
np.sqrt(np.sum((cz2017 - pl2017)**2)),
np.sqrt(np.sum((cz2017 - pt2017)**2)),
np.sqrt(np.sum((cz2017 - se2017)**2)),
np.sqrt(np.sum((cz2017 - si2017)**2)),
np.sqrt(np.sum((cz2017 - sk2017)**2)),
np.sqrt(np.sum((cz2017 - tr2017)**2)),
])
de2017_all =  np.array([
np.sqrt(np.sum((de2017 - at2017)**2)),
np.sqrt(np.sum((de2017 - be2017)**2)),
np.sqrt(np.sum((de2017 - bg2017)**2)),
np.sqrt(np.sum((de2017 - by2017)**2)),
np.sqrt(np.sum((de2017 - ch2017)**2)),
np.sqrt(np.sum((de2017 - cz2017)**2)),
np.sqrt(np.sum((de2017 - de2017)**2)),
np.sqrt(np.sum((de2017 - dk2017)**2)),
np.sqrt(np.sum((de2017 - ee2017)**2)),
np.sqrt(np.sum((de2017 - es2017)**2)),
np.sqrt(np.sum((de2017 - fi2017)**2)),
np.sqrt(np.sum((de2017 - fr2017)**2)),
np.sqrt(np.sum((de2017 - gb2017)**2)),
np.sqrt(np.sum((de2017 - gr2017)**2)),
np.sqrt(np.sum((de2017 - hr2017)**2)),
np.sqrt(np.sum((de2017 - hu2017)**2)),
np.sqrt(np.sum((de2017 - ie2017)**2)),
np.sqrt(np.sum((de2017 - it2017)**2)),
np.sqrt(np.sum((de2017 - nl2017)**2)),
np.sqrt(np.sum((de2017 - no2017)**2)),
np.sqrt(np.sum((de2017 - pl2017)**2)),
np.sqrt(np.sum((de2017 - pt2017)**2)),
np.sqrt(np.sum((de2017 - se2017)**2)),
np.sqrt(np.sum((de2017 - si2017)**2)),
np.sqrt(np.sum((de2017 - sk2017)**2)),
np.sqrt(np.sum((de2017 - tr2017)**2)),
])
dk2017_all =  np.array([
np.sqrt(np.sum((dk2017 - at2017)**2)),
np.sqrt(np.sum((dk2017 - be2017)**2)),
np.sqrt(np.sum((dk2017 - bg2017)**2)),
np.sqrt(np.sum((dk2017 - by2017)**2)),
np.sqrt(np.sum((dk2017 - ch2017)**2)),
np.sqrt(np.sum((dk2017 - cz2017)**2)),
np.sqrt(np.sum((dk2017 - de2017)**2)),
np.sqrt(np.sum((dk2017 - dk2017)**2)),
np.sqrt(np.sum((dk2017 - ee2017)**2)),
np.sqrt(np.sum((dk2017 - es2017)**2)),
np.sqrt(np.sum((dk2017 - fi2017)**2)),
np.sqrt(np.sum((dk2017 - fr2017)**2)),
np.sqrt(np.sum((dk2017 - gb2017)**2)),
np.sqrt(np.sum((dk2017 - gr2017)**2)),
np.sqrt(np.sum((dk2017 - hr2017)**2)),
np.sqrt(np.sum((dk2017 - hu2017)**2)),
np.sqrt(np.sum((dk2017 - ie2017)**2)),
np.sqrt(np.sum((dk2017 - it2017)**2)),
np.sqrt(np.sum((dk2017 - nl2017)**2)),
np.sqrt(np.sum((dk2017 - no2017)**2)),
np.sqrt(np.sum((dk2017 - pl2017)**2)),
np.sqrt(np.sum((dk2017 - pt2017)**2)),
np.sqrt(np.sum((dk2017 - se2017)**2)),
np.sqrt(np.sum((dk2017 - si2017)**2)),
np.sqrt(np.sum((dk2017 - sk2017)**2)),
np.sqrt(np.sum((dk2017 - tr2017)**2)),
])
ee2017_all =  np.array([
np.sqrt(np.sum((ee2017 - at2017)**2)),
np.sqrt(np.sum((ee2017 - be2017)**2)),
np.sqrt(np.sum((ee2017 - bg2017)**2)),
np.sqrt(np.sum((ee2017 - by2017)**2)),
np.sqrt(np.sum((ee2017 - ch2017)**2)),
np.sqrt(np.sum((ee2017 - cz2017)**2)),
np.sqrt(np.sum((ee2017 - de2017)**2)),
np.sqrt(np.sum((ee2017 - dk2017)**2)),
np.sqrt(np.sum((ee2017 - ee2017)**2)),
np.sqrt(np.sum((ee2017 - es2017)**2)),
np.sqrt(np.sum((ee2017 - fi2017)**2)),
np.sqrt(np.sum((ee2017 - fr2017)**2)),
np.sqrt(np.sum((ee2017 - gb2017)**2)),
np.sqrt(np.sum((ee2017 - gr2017)**2)),
np.sqrt(np.sum((ee2017 - hr2017)**2)),
np.sqrt(np.sum((ee2017 - hu2017)**2)),
np.sqrt(np.sum((ee2017 - ie2017)**2)),
np.sqrt(np.sum((ee2017 - it2017)**2)),
np.sqrt(np.sum((ee2017 - nl2017)**2)),
np.sqrt(np.sum((ee2017 - no2017)**2)),
np.sqrt(np.sum((ee2017 - pl2017)**2)),
np.sqrt(np.sum((ee2017 - pt2017)**2)),
np.sqrt(np.sum((ee2017 - se2017)**2)),
np.sqrt(np.sum((ee2017 - si2017)**2)),
np.sqrt(np.sum((ee2017 - sk2017)**2)),
np.sqrt(np.sum((ee2017 - tr2017)**2)),
])
es2017_all =  np.array([
np.sqrt(np.sum((es2017 - at2017)**2)),
np.sqrt(np.sum((es2017 - be2017)**2)),
np.sqrt(np.sum((es2017 - bg2017)**2)),
np.sqrt(np.sum((es2017 - by2017)**2)),
np.sqrt(np.sum((es2017 - ch2017)**2)),
np.sqrt(np.sum((es2017 - cz2017)**2)),
np.sqrt(np.sum((es2017 - de2017)**2)),
np.sqrt(np.sum((es2017 - dk2017)**2)),
np.sqrt(np.sum((es2017 - ee2017)**2)),
np.sqrt(np.sum((es2017 - es2017)**2)),
np.sqrt(np.sum((es2017 - fi2017)**2)),
np.sqrt(np.sum((es2017 - fr2017)**2)),
np.sqrt(np.sum((es2017 - gb2017)**2)),
np.sqrt(np.sum((es2017 - gr2017)**2)),
np.sqrt(np.sum((es2017 - hr2017)**2)),
np.sqrt(np.sum((es2017 - hu2017)**2)),
np.sqrt(np.sum((es2017 - ie2017)**2)),
np.sqrt(np.sum((es2017 - it2017)**2)),
np.sqrt(np.sum((es2017 - nl2017)**2)),
np.sqrt(np.sum((es2017 - no2017)**2)),
np.sqrt(np.sum((es2017 - pl2017)**2)),
np.sqrt(np.sum((es2017 - pt2017)**2)),
np.sqrt(np.sum((es2017 - se2017)**2)),
np.sqrt(np.sum((es2017 - si2017)**2)),
np.sqrt(np.sum((es2017 - sk2017)**2)),
np.sqrt(np.sum((es2017 - tr2017)**2)),
])
fi2017_all =  np.array([
np.sqrt(np.sum((fi2017 - at2017)**2)),
np.sqrt(np.sum((fi2017 - be2017)**2)),
np.sqrt(np.sum((fi2017 - bg2017)**2)),
np.sqrt(np.sum((fi2017 - by2017)**2)),
np.sqrt(np.sum((fi2017 - ch2017)**2)),
np.sqrt(np.sum((fi2017 - cz2017)**2)),
np.sqrt(np.sum((fi2017 - de2017)**2)),
np.sqrt(np.sum((fi2017 - dk2017)**2)),
np.sqrt(np.sum((fi2017 - ee2017)**2)),
np.sqrt(np.sum((fi2017 - es2017)**2)),
np.sqrt(np.sum((fi2017 - fi2017)**2)),
np.sqrt(np.sum((fi2017 - fr2017)**2)),
np.sqrt(np.sum((fi2017 - gb2017)**2)),
np.sqrt(np.sum((fi2017 - gr2017)**2)),
np.sqrt(np.sum((fi2017 - hr2017)**2)),
np.sqrt(np.sum((fi2017 - hu2017)**2)),
np.sqrt(np.sum((fi2017 - ie2017)**2)),
np.sqrt(np.sum((fi2017 - it2017)**2)),
np.sqrt(np.sum((fi2017 - nl2017)**2)),
np.sqrt(np.sum((fi2017 - no2017)**2)),
np.sqrt(np.sum((fi2017 - pl2017)**2)),
np.sqrt(np.sum((fi2017 - pt2017)**2)),
np.sqrt(np.sum((fi2017 - se2017)**2)),
np.sqrt(np.sum((fi2017 - si2017)**2)),
np.sqrt(np.sum((fi2017 - sk2017)**2)),
np.sqrt(np.sum((fi2017 - tr2017)**2)),
])
fr2017_all =  np.array([
np.sqrt(np.sum((fr2017 - at2017)**2)),
np.sqrt(np.sum((fr2017 - be2017)**2)),
np.sqrt(np.sum((fr2017 - bg2017)**2)),
np.sqrt(np.sum((fr2017 - by2017)**2)),
np.sqrt(np.sum((fr2017 - ch2017)**2)),
np.sqrt(np.sum((fr2017 - cz2017)**2)),
np.sqrt(np.sum((fr2017 - de2017)**2)),
np.sqrt(np.sum((fr2017 - dk2017)**2)),
np.sqrt(np.sum((fr2017 - ee2017)**2)),
np.sqrt(np.sum((fr2017 - es2017)**2)),
np.sqrt(np.sum((fr2017 - fi2017)**2)),
np.sqrt(np.sum((fr2017 - fr2017)**2)),
np.sqrt(np.sum((fr2017 - gb2017)**2)),
np.sqrt(np.sum((fr2017 - gr2017)**2)),
np.sqrt(np.sum((fr2017 - hr2017)**2)),
np.sqrt(np.sum((fr2017 - hu2017)**2)),
np.sqrt(np.sum((fr2017 - ie2017)**2)),
np.sqrt(np.sum((fr2017 - it2017)**2)),
np.sqrt(np.sum((fr2017 - nl2017)**2)),
np.sqrt(np.sum((fr2017 - no2017)**2)),
np.sqrt(np.sum((fr2017 - pl2017)**2)),
np.sqrt(np.sum((fr2017 - pt2017)**2)),
np.sqrt(np.sum((fr2017 - se2017)**2)),
np.sqrt(np.sum((fr2017 - si2017)**2)),
np.sqrt(np.sum((fr2017 - sk2017)**2)),
np.sqrt(np.sum((fr2017 - tr2017)**2)),
])
gb2017_all =  np.array([
np.sqrt(np.sum((gb2017 - at2017)**2)),
np.sqrt(np.sum((gb2017 - be2017)**2)),
np.sqrt(np.sum((gb2017 - bg2017)**2)),
np.sqrt(np.sum((gb2017 - by2017)**2)),
np.sqrt(np.sum((gb2017 - ch2017)**2)),
np.sqrt(np.sum((gb2017 - cz2017)**2)),
np.sqrt(np.sum((gb2017 - de2017)**2)),
np.sqrt(np.sum((gb2017 - dk2017)**2)),
np.sqrt(np.sum((gb2017 - ee2017)**2)),
np.sqrt(np.sum((gb2017 - es2017)**2)),
np.sqrt(np.sum((gb2017 - fi2017)**2)),
np.sqrt(np.sum((gb2017 - fr2017)**2)),
np.sqrt(np.sum((gb2017 - gb2017)**2)),
np.sqrt(np.sum((gb2017 - gr2017)**2)),
np.sqrt(np.sum((gb2017 - hr2017)**2)),
np.sqrt(np.sum((gb2017 - hu2017)**2)),
np.sqrt(np.sum((gb2017 - ie2017)**2)),
np.sqrt(np.sum((gb2017 - it2017)**2)),
np.sqrt(np.sum((gb2017 - nl2017)**2)),
np.sqrt(np.sum((gb2017 - no2017)**2)),
np.sqrt(np.sum((gb2017 - pl2017)**2)),
np.sqrt(np.sum((gb2017 - pt2017)**2)),
np.sqrt(np.sum((gb2017 - se2017)**2)),
np.sqrt(np.sum((gb2017 - si2017)**2)),
np.sqrt(np.sum((gb2017 - sk2017)**2)),
np.sqrt(np.sum((gb2017 - tr2017)**2)),
])
gr2017_all =  np.array([
np.sqrt(np.sum((gr2017 - at2017)**2)),
np.sqrt(np.sum((gr2017 - be2017)**2)),
np.sqrt(np.sum((gr2017 - bg2017)**2)),
np.sqrt(np.sum((gr2017 - by2017)**2)),
np.sqrt(np.sum((gr2017 - ch2017)**2)),
np.sqrt(np.sum((gr2017 - cz2017)**2)),
np.sqrt(np.sum((gr2017 - de2017)**2)),
np.sqrt(np.sum((gr2017 - dk2017)**2)),
np.sqrt(np.sum((gr2017 - ee2017)**2)),
np.sqrt(np.sum((gr2017 - es2017)**2)),
np.sqrt(np.sum((gr2017 - fi2017)**2)),
np.sqrt(np.sum((gr2017 - fr2017)**2)),
np.sqrt(np.sum((gr2017 - gb2017)**2)),
np.sqrt(np.sum((gr2017 - gr2017)**2)),
np.sqrt(np.sum((gr2017 - hr2017)**2)),
np.sqrt(np.sum((gr2017 - hu2017)**2)),
np.sqrt(np.sum((gr2017 - ie2017)**2)),
np.sqrt(np.sum((gr2017 - it2017)**2)),
np.sqrt(np.sum((gr2017 - nl2017)**2)),
np.sqrt(np.sum((gr2017 - no2017)**2)),
np.sqrt(np.sum((gr2017 - pl2017)**2)),
np.sqrt(np.sum((gr2017 - pt2017)**2)),
np.sqrt(np.sum((gr2017 - se2017)**2)),
np.sqrt(np.sum((gr2017 - si2017)**2)),
np.sqrt(np.sum((gr2017 - sk2017)**2)),
np.sqrt(np.sum((gr2017 - tr2017)**2)),
])
hr2017_all =  np.array([
np.sqrt(np.sum((hr2017 - at2017)**2)),
np.sqrt(np.sum((hr2017 - be2017)**2)),
np.sqrt(np.sum((hr2017 - bg2017)**2)),
np.sqrt(np.sum((hr2017 - by2017)**2)),
np.sqrt(np.sum((hr2017 - ch2017)**2)),
np.sqrt(np.sum((hr2017 - cz2017)**2)),
np.sqrt(np.sum((hr2017 - de2017)**2)),
np.sqrt(np.sum((hr2017 - dk2017)**2)),
np.sqrt(np.sum((hr2017 - ee2017)**2)),
np.sqrt(np.sum((hr2017 - es2017)**2)),
np.sqrt(np.sum((hr2017 - fi2017)**2)),
np.sqrt(np.sum((hr2017 - fr2017)**2)),
np.sqrt(np.sum((hr2017 - gb2017)**2)),
np.sqrt(np.sum((hr2017 - gr2017)**2)),
np.sqrt(np.sum((hr2017 - hr2017)**2)),
np.sqrt(np.sum((hr2017 - hu2017)**2)),
np.sqrt(np.sum((hr2017 - ie2017)**2)),
np.sqrt(np.sum((hr2017 - it2017)**2)),
np.sqrt(np.sum((hr2017 - nl2017)**2)),
np.sqrt(np.sum((hr2017 - no2017)**2)),
np.sqrt(np.sum((hr2017 - pl2017)**2)),
np.sqrt(np.sum((hr2017 - pt2017)**2)),
np.sqrt(np.sum((hr2017 - se2017)**2)),
np.sqrt(np.sum((hr2017 - si2017)**2)),
np.sqrt(np.sum((hr2017 - sk2017)**2)),
np.sqrt(np.sum((hr2017 - tr2017)**2)),
])
hu2017_all =  np.array([
np.sqrt(np.sum((hu2017 - at2017)**2)),
np.sqrt(np.sum((hu2017 - be2017)**2)),
np.sqrt(np.sum((hu2017 - bg2017)**2)),
np.sqrt(np.sum((hu2017 - by2017)**2)),
np.sqrt(np.sum((hu2017 - ch2017)**2)),
np.sqrt(np.sum((hu2017 - cz2017)**2)),
np.sqrt(np.sum((hu2017 - de2017)**2)),
np.sqrt(np.sum((hu2017 - dk2017)**2)),
np.sqrt(np.sum((hu2017 - ee2017)**2)),
np.sqrt(np.sum((hu2017 - es2017)**2)),
np.sqrt(np.sum((hu2017 - fi2017)**2)),
np.sqrt(np.sum((hu2017 - fr2017)**2)),
np.sqrt(np.sum((hu2017 - gb2017)**2)),
np.sqrt(np.sum((hu2017 - gr2017)**2)),
np.sqrt(np.sum((hu2017 - hr2017)**2)),
np.sqrt(np.sum((hu2017 - hu2017)**2)),
np.sqrt(np.sum((hu2017 - ie2017)**2)),
np.sqrt(np.sum((hu2017 - it2017)**2)),
np.sqrt(np.sum((hu2017 - nl2017)**2)),
np.sqrt(np.sum((hu2017 - no2017)**2)),
np.sqrt(np.sum((hu2017 - pl2017)**2)),
np.sqrt(np.sum((hu2017 - pt2017)**2)),
np.sqrt(np.sum((hu2017 - se2017)**2)),
np.sqrt(np.sum((hu2017 - si2017)**2)),
np.sqrt(np.sum((hu2017 - sk2017)**2)),
np.sqrt(np.sum((hu2017 - tr2017)**2)),
])
ie2017_all =  np.array([
np.sqrt(np.sum((ie2017 - at2017)**2)),
np.sqrt(np.sum((ie2017 - be2017)**2)),
np.sqrt(np.sum((ie2017 - bg2017)**2)),
np.sqrt(np.sum((ie2017 - by2017)**2)),
np.sqrt(np.sum((ie2017 - ch2017)**2)),
np.sqrt(np.sum((ie2017 - cz2017)**2)),
np.sqrt(np.sum((ie2017 - de2017)**2)),
np.sqrt(np.sum((ie2017 - dk2017)**2)),
np.sqrt(np.sum((ie2017 - ee2017)**2)),
np.sqrt(np.sum((ie2017 - es2017)**2)),
np.sqrt(np.sum((ie2017 - fi2017)**2)),
np.sqrt(np.sum((ie2017 - fr2017)**2)),
np.sqrt(np.sum((ie2017 - gb2017)**2)),
np.sqrt(np.sum((ie2017 - gr2017)**2)),
np.sqrt(np.sum((ie2017 - hr2017)**2)),
np.sqrt(np.sum((ie2017 - hu2017)**2)),
np.sqrt(np.sum((ie2017 - ie2017)**2)),
np.sqrt(np.sum((ie2017 - it2017)**2)),
np.sqrt(np.sum((ie2017 - nl2017)**2)),
np.sqrt(np.sum((ie2017 - no2017)**2)),
np.sqrt(np.sum((ie2017 - pl2017)**2)),
np.sqrt(np.sum((ie2017 - pt2017)**2)),
np.sqrt(np.sum((ie2017 - se2017)**2)),
np.sqrt(np.sum((ie2017 - si2017)**2)),
np.sqrt(np.sum((ie2017 - sk2017)**2)),
np.sqrt(np.sum((ie2017 - tr2017)**2)),
])
it2017_all =  np.array([
np.sqrt(np.sum((it2017 - at2017)**2)),
np.sqrt(np.sum((it2017 - be2017)**2)),
np.sqrt(np.sum((it2017 - bg2017)**2)),
np.sqrt(np.sum((it2017 - by2017)**2)),
np.sqrt(np.sum((it2017 - ch2017)**2)),
np.sqrt(np.sum((it2017 - cz2017)**2)),
np.sqrt(np.sum((it2017 - de2017)**2)),
np.sqrt(np.sum((it2017 - dk2017)**2)),
np.sqrt(np.sum((it2017 - ee2017)**2)),
np.sqrt(np.sum((it2017 - es2017)**2)),
np.sqrt(np.sum((it2017 - fi2017)**2)),
np.sqrt(np.sum((it2017 - fr2017)**2)),
np.sqrt(np.sum((it2017 - gb2017)**2)),
np.sqrt(np.sum((it2017 - gr2017)**2)),
np.sqrt(np.sum((it2017 - hr2017)**2)),
np.sqrt(np.sum((it2017 - hu2017)**2)),
np.sqrt(np.sum((it2017 - ie2017)**2)),
np.sqrt(np.sum((it2017 - it2017)**2)),
np.sqrt(np.sum((it2017 - nl2017)**2)),
np.sqrt(np.sum((it2017 - no2017)**2)),
np.sqrt(np.sum((it2017 - pl2017)**2)),
np.sqrt(np.sum((it2017 - pt2017)**2)),
np.sqrt(np.sum((it2017 - se2017)**2)),
np.sqrt(np.sum((it2017 - si2017)**2)),
np.sqrt(np.sum((it2017 - sk2017)**2)),
np.sqrt(np.sum((it2017 - tr2017)**2)),
])
nl2017_all =  np.array([
np.sqrt(np.sum((nl2017 - at2017)**2)),
np.sqrt(np.sum((nl2017 - be2017)**2)),
np.sqrt(np.sum((nl2017 - bg2017)**2)),
np.sqrt(np.sum((nl2017 - by2017)**2)),
np.sqrt(np.sum((nl2017 - ch2017)**2)),
np.sqrt(np.sum((nl2017 - cz2017)**2)),
np.sqrt(np.sum((nl2017 - de2017)**2)),
np.sqrt(np.sum((nl2017 - dk2017)**2)),
np.sqrt(np.sum((nl2017 - ee2017)**2)),
np.sqrt(np.sum((nl2017 - es2017)**2)),
np.sqrt(np.sum((nl2017 - fi2017)**2)),
np.sqrt(np.sum((nl2017 - fr2017)**2)),
np.sqrt(np.sum((nl2017 - gb2017)**2)),
np.sqrt(np.sum((nl2017 - gr2017)**2)),
np.sqrt(np.sum((nl2017 - hr2017)**2)),
np.sqrt(np.sum((nl2017 - hu2017)**2)),
np.sqrt(np.sum((nl2017 - ie2017)**2)),
np.sqrt(np.sum((nl2017 - it2017)**2)),
np.sqrt(np.sum((nl2017 - nl2017)**2)),
np.sqrt(np.sum((nl2017 - no2017)**2)),
np.sqrt(np.sum((nl2017 - pl2017)**2)),
np.sqrt(np.sum((nl2017 - pt2017)**2)),
np.sqrt(np.sum((nl2017 - se2017)**2)),
np.sqrt(np.sum((nl2017 - si2017)**2)),
np.sqrt(np.sum((nl2017 - sk2017)**2)),
np.sqrt(np.sum((nl2017 - tr2017)**2)),
])
no2017_all =  np.array([
np.sqrt(np.sum((no2017 - at2017)**2)),
np.sqrt(np.sum((no2017 - be2017)**2)),
np.sqrt(np.sum((no2017 - bg2017)**2)),
np.sqrt(np.sum((no2017 - by2017)**2)),
np.sqrt(np.sum((no2017 - ch2017)**2)),
np.sqrt(np.sum((no2017 - cz2017)**2)),
np.sqrt(np.sum((no2017 - de2017)**2)),
np.sqrt(np.sum((no2017 - dk2017)**2)),
np.sqrt(np.sum((no2017 - ee2017)**2)),
np.sqrt(np.sum((no2017 - es2017)**2)),
np.sqrt(np.sum((no2017 - fi2017)**2)),
np.sqrt(np.sum((no2017 - fr2017)**2)),
np.sqrt(np.sum((no2017 - gb2017)**2)),
np.sqrt(np.sum((no2017 - gr2017)**2)),
np.sqrt(np.sum((no2017 - hr2017)**2)),
np.sqrt(np.sum((no2017 - hu2017)**2)),
np.sqrt(np.sum((no2017 - ie2017)**2)),
np.sqrt(np.sum((no2017 - it2017)**2)),
np.sqrt(np.sum((no2017 - nl2017)**2)),
np.sqrt(np.sum((no2017 - no2017)**2)),
np.sqrt(np.sum((no2017 - pl2017)**2)),
np.sqrt(np.sum((no2017 - pt2017)**2)),
np.sqrt(np.sum((no2017 - se2017)**2)),
np.sqrt(np.sum((no2017 - si2017)**2)),
np.sqrt(np.sum((no2017 - sk2017)**2)),
np.sqrt(np.sum((no2017 - tr2017)**2)),
])
pl2017_all =  np.array([
np.sqrt(np.sum((pl2017 - at2017)**2)),
np.sqrt(np.sum((pl2017 - be2017)**2)),
np.sqrt(np.sum((pl2017 - bg2017)**2)),
np.sqrt(np.sum((pl2017 - by2017)**2)),
np.sqrt(np.sum((pl2017 - ch2017)**2)),
np.sqrt(np.sum((pl2017 - cz2017)**2)),
np.sqrt(np.sum((pl2017 - de2017)**2)),
np.sqrt(np.sum((pl2017 - dk2017)**2)),
np.sqrt(np.sum((pl2017 - ee2017)**2)),
np.sqrt(np.sum((pl2017 - es2017)**2)),
np.sqrt(np.sum((pl2017 - fi2017)**2)),
np.sqrt(np.sum((pl2017 - fr2017)**2)),
np.sqrt(np.sum((pl2017 - gb2017)**2)),
np.sqrt(np.sum((pl2017 - gr2017)**2)),
np.sqrt(np.sum((pl2017 - hr2017)**2)),
np.sqrt(np.sum((pl2017 - hu2017)**2)),
np.sqrt(np.sum((pl2017 - ie2017)**2)),
np.sqrt(np.sum((pl2017 - it2017)**2)),
np.sqrt(np.sum((pl2017 - nl2017)**2)),
np.sqrt(np.sum((pl2017 - no2017)**2)),
np.sqrt(np.sum((pl2017 - pl2017)**2)),
np.sqrt(np.sum((pl2017 - pt2017)**2)),
np.sqrt(np.sum((pl2017 - se2017)**2)),
np.sqrt(np.sum((pl2017 - si2017)**2)),
np.sqrt(np.sum((pl2017 - sk2017)**2)),
np.sqrt(np.sum((pl2017 - tr2017)**2)),
])
pt2017_all =  np.array([
np.sqrt(np.sum((pt2017 - at2017)**2)),
np.sqrt(np.sum((pt2017 - be2017)**2)),
np.sqrt(np.sum((pt2017 - bg2017)**2)),
np.sqrt(np.sum((pt2017 - by2017)**2)),
np.sqrt(np.sum((pt2017 - ch2017)**2)),
np.sqrt(np.sum((pt2017 - cz2017)**2)),
np.sqrt(np.sum((pt2017 - de2017)**2)),
np.sqrt(np.sum((pt2017 - dk2017)**2)),
np.sqrt(np.sum((pt2017 - ee2017)**2)),
np.sqrt(np.sum((pt2017 - es2017)**2)),
np.sqrt(np.sum((pt2017 - fi2017)**2)),
np.sqrt(np.sum((pt2017 - fr2017)**2)),
np.sqrt(np.sum((pt2017 - gb2017)**2)),
np.sqrt(np.sum((pt2017 - gr2017)**2)),
np.sqrt(np.sum((pt2017 - hr2017)**2)),
np.sqrt(np.sum((pt2017 - hu2017)**2)),
np.sqrt(np.sum((pt2017 - ie2017)**2)),
np.sqrt(np.sum((pt2017 - it2017)**2)),
np.sqrt(np.sum((pt2017 - nl2017)**2)),
np.sqrt(np.sum((pt2017 - no2017)**2)),
np.sqrt(np.sum((pt2017 - pl2017)**2)),
np.sqrt(np.sum((pt2017 - pt2017)**2)),
np.sqrt(np.sum((pt2017 - se2017)**2)),
np.sqrt(np.sum((pt2017 - si2017)**2)),
np.sqrt(np.sum((pt2017 - sk2017)**2)),
np.sqrt(np.sum((pt2017 - tr2017)**2)),
])
se2017_all =  np.array([
np.sqrt(np.sum((se2017 - at2017)**2)),
np.sqrt(np.sum((se2017 - be2017)**2)),
np.sqrt(np.sum((se2017 - bg2017)**2)),
np.sqrt(np.sum((se2017 - by2017)**2)),
np.sqrt(np.sum((se2017 - ch2017)**2)),
np.sqrt(np.sum((se2017 - cz2017)**2)),
np.sqrt(np.sum((se2017 - de2017)**2)),
np.sqrt(np.sum((se2017 - dk2017)**2)),
np.sqrt(np.sum((se2017 - ee2017)**2)),
np.sqrt(np.sum((se2017 - es2017)**2)),
np.sqrt(np.sum((se2017 - fi2017)**2)),
np.sqrt(np.sum((se2017 - fr2017)**2)),
np.sqrt(np.sum((se2017 - gb2017)**2)),
np.sqrt(np.sum((se2017 - gr2017)**2)),
np.sqrt(np.sum((se2017 - hr2017)**2)),
np.sqrt(np.sum((se2017 - hu2017)**2)),
np.sqrt(np.sum((se2017 - ie2017)**2)),
np.sqrt(np.sum((se2017 - it2017)**2)),
np.sqrt(np.sum((se2017 - nl2017)**2)),
np.sqrt(np.sum((se2017 - no2017)**2)),
np.sqrt(np.sum((se2017 - pl2017)**2)),
np.sqrt(np.sum((se2017 - pt2017)**2)),
np.sqrt(np.sum((se2017 - se2017)**2)),
np.sqrt(np.sum((se2017 - si2017)**2)),
np.sqrt(np.sum((se2017 - sk2017)**2)),
np.sqrt(np.sum((se2017 - tr2017)**2)),
])
si2017_all =  np.array([
np.sqrt(np.sum((si2017 - at2017)**2)),
np.sqrt(np.sum((si2017 - be2017)**2)),
np.sqrt(np.sum((si2017 - bg2017)**2)),
np.sqrt(np.sum((si2017 - by2017)**2)),
np.sqrt(np.sum((si2017 - ch2017)**2)),
np.sqrt(np.sum((si2017 - cz2017)**2)),
np.sqrt(np.sum((si2017 - de2017)**2)),
np.sqrt(np.sum((si2017 - dk2017)**2)),
np.sqrt(np.sum((si2017 - ee2017)**2)),
np.sqrt(np.sum((si2017 - es2017)**2)),
np.sqrt(np.sum((si2017 - fi2017)**2)),
np.sqrt(np.sum((si2017 - fr2017)**2)),
np.sqrt(np.sum((si2017 - gb2017)**2)),
np.sqrt(np.sum((si2017 - gr2017)**2)),
np.sqrt(np.sum((si2017 - hr2017)**2)),
np.sqrt(np.sum((si2017 - hu2017)**2)),
np.sqrt(np.sum((si2017 - ie2017)**2)),
np.sqrt(np.sum((si2017 - it2017)**2)),
np.sqrt(np.sum((si2017 - nl2017)**2)),
np.sqrt(np.sum((si2017 - no2017)**2)),
np.sqrt(np.sum((si2017 - pl2017)**2)),
np.sqrt(np.sum((si2017 - pt2017)**2)),
np.sqrt(np.sum((si2017 - se2017)**2)),
np.sqrt(np.sum((si2017 - si2017)**2)),
np.sqrt(np.sum((si2017 - sk2017)**2)),
np.sqrt(np.sum((si2017 - tr2017)**2)),
])
sk2017_all =  np.array([
np.sqrt(np.sum((sk2017 - at2017)**2)),
np.sqrt(np.sum((sk2017 - be2017)**2)),
np.sqrt(np.sum((sk2017 - bg2017)**2)),
np.sqrt(np.sum((sk2017 - by2017)**2)),
np.sqrt(np.sum((sk2017 - ch2017)**2)),
np.sqrt(np.sum((sk2017 - cz2017)**2)),
np.sqrt(np.sum((sk2017 - de2017)**2)),
np.sqrt(np.sum((sk2017 - dk2017)**2)),
np.sqrt(np.sum((sk2017 - ee2017)**2)),
np.sqrt(np.sum((sk2017 - es2017)**2)),
np.sqrt(np.sum((sk2017 - fi2017)**2)),
np.sqrt(np.sum((sk2017 - fr2017)**2)),
np.sqrt(np.sum((sk2017 - gb2017)**2)),
np.sqrt(np.sum((sk2017 - gr2017)**2)),
np.sqrt(np.sum((sk2017 - hr2017)**2)),
np.sqrt(np.sum((sk2017 - hu2017)**2)),
np.sqrt(np.sum((sk2017 - ie2017)**2)),
np.sqrt(np.sum((sk2017 - it2017)**2)),
np.sqrt(np.sum((sk2017 - nl2017)**2)),
np.sqrt(np.sum((sk2017 - no2017)**2)),
np.sqrt(np.sum((sk2017 - pl2017)**2)),
np.sqrt(np.sum((sk2017 - pt2017)**2)),
np.sqrt(np.sum((sk2017 - se2017)**2)),
np.sqrt(np.sum((sk2017 - si2017)**2)),
np.sqrt(np.sum((sk2017 - sk2017)**2)),
np.sqrt(np.sum((sk2017 - tr2017)**2)),
])
tr2017_all =  np.array([
np.sqrt(np.sum((tr2017 - at2017)**2)),
np.sqrt(np.sum((tr2017 - be2017)**2)),
np.sqrt(np.sum((tr2017 - bg2017)**2)),
np.sqrt(np.sum((tr2017 - by2017)**2)),
np.sqrt(np.sum((tr2017 - ch2017)**2)),
np.sqrt(np.sum((tr2017 - cz2017)**2)),
np.sqrt(np.sum((tr2017 - de2017)**2)),
np.sqrt(np.sum((tr2017 - dk2017)**2)),
np.sqrt(np.sum((tr2017 - ee2017)**2)),
np.sqrt(np.sum((tr2017 - es2017)**2)),
np.sqrt(np.sum((tr2017 - fi2017)**2)),
np.sqrt(np.sum((tr2017 - fr2017)**2)),
np.sqrt(np.sum((tr2017 - gb2017)**2)),
np.sqrt(np.sum((tr2017 - gr2017)**2)),
np.sqrt(np.sum((tr2017 - hr2017)**2)),
np.sqrt(np.sum((tr2017 - hu2017)**2)),
np.sqrt(np.sum((tr2017 - ie2017)**2)),
np.sqrt(np.sum((tr2017 - it2017)**2)),
np.sqrt(np.sum((tr2017 - nl2017)**2)),
np.sqrt(np.sum((tr2017 - no2017)**2)),
np.sqrt(np.sum((tr2017 - pl2017)**2)),
np.sqrt(np.sum((tr2017 - pt2017)**2)),
np.sqrt(np.sum((tr2017 - se2017)**2)),
np.sqrt(np.sum((tr2017 - si2017)**2)),
np.sqrt(np.sum((tr2017 - sk2017)**2)),
np.sqrt(np.sum((tr2017 - tr2017)**2)),
])

#Do not delete - this is used for generating variables in the next cell

temporal_1_similiarity=[]
temporal_2_similiarity=[]
temporal_3_similiarity=[]
temporal_4_similiarity=[]
year = 2017
print('temporal_4_similiarity = np.array([')
for x in range(0,len(var)):
   # print('temporal_1_similiarity = np.array([')
    print(str(var[x])+str(year)+'_all', end=', ')
print('])')
#temporal_1_similiarity 
temporal_1_similiarity = np.array([Armenia2016_all, Austria2016_all, Belarus2016_all, Belgium2016_all, Bulgaria2016_all, Croatia2016_all, Czech_Republic2016_all, 
          Denmark2016_all, Estonia2016_all, Finland2016_all, France2016_all, Georgia2016_all, 
          Germany2016_all, Greece2016_all, Hungary2016_all, Iceland2016_all, Ireland2016_all, Italy2016_all, Latvia2016_all, Lithuania2016_all, 
          Luxembourg2016_all, Netherlands2016_all, Norway2016_all, Poland2016_all, Portugal2016_all, Romania2016_all, Slovakia2016_all, Slovenia2016_all, 
          Spain2016_all, Sweden2016_all, Switzerland2016_all, Turkey2016_all, Ukraine2016_all, United_Kingdom2016_all])

In [ ]:
temporal_1_similiarity = np.array([
at2014_all, be2014_all, bg2014_all, by2014_all, ch2014_all, cz2014_all, de2014_all, dk2014_all, ee2014_all, es2014_all, fi2014_all, fr2014_all, gb2014_all, gr2014_all, hr2014_all, hu2014_all, ie2014_all, it2014_all, nl2014_all, no2014_all, pl2014_all, pt2014_all, se2014_all, si2014_all, sk2014_all, tr2014_all, ])

In [ ]:
temporal_2_similiarity = np.array([
at2015_all, be2015_all, bg2015_all, by2015_all, ch2015_all, cz2015_all, de2015_all, dk2015_all, ee2015_all, es2015_all, fi2015_all, fr2015_all, gb2015_all, gr2015_all, hr2015_all, hu2015_all, ie2015_all, it2015_all, nl2015_all, no2015_all, pl2015_all, pt2015_all, se2015_all, si2015_all, sk2015_all, tr2015_all, ])

In [ ]:
temporal_3_similiarity = np.array([
at2016_all, be2016_all, bg2016_all, by2016_all, ch2016_all, cz2016_all, de2016_all, dk2016_all, ee2016_all, es2016_all, fi2016_all, fr2016_all, gb2016_all, gr2016_all, hr2016_all, hu2016_all, ie2016_all, it2016_all, nl2016_all, no2016_all, pl2016_all, pt2016_all, se2016_all, si2016_all, sk2016_all, tr2016_all, ])

In [ ]:
temporal_4_similiarity = np.array([
at2017_all, be2017_all, bg2017_all, by2017_all, ch2017_all, cz2017_all, de2017_all, dk2017_all, ee2017_all, es2017_all, fi2017_all, fr2017_all, gb2017_all, gr2017_all, hr2017_all, hu2017_all, ie2017_all, it2017_all, nl2017_all, no2017_all, pl2017_all, pt2017_all, se2017_all, si2017_all, sk2017_all, tr2017_all, ])

In [ ]:
temporal_1_matx = np.reshape(temporal_1_similiarity, (len(var), len(var)))
temporal_2_matx = np.reshape(temporal_2_similiarity, (len(var), len(var)))
temporal_3_matx = np.reshape(temporal_3_similiarity, (len(var), len(var)))
temporal_4_matx = np.reshape(temporal_4_similiarity, (len(var), len(var)))

#temporal_4_matx

In [ ]:
temporal_1_df = pd.DataFrame(temporal_1_matx, index=var, columns=var)
temporal_2_df = pd.DataFrame(temporal_2_matx, index=var, columns=var)
temporal_3_df = pd.DataFrame(temporal_3_matx, index=var, columns=var)
temporal_4_df = pd.DataFrame(temporal_4_matx, index=var, columns=var)


In [ ]:
temporal_1_comparison = temporal_1_df
temporal_1_distance = temporal_1_df
temporal_1_distance1 = np.array(temporal_1_comparison)

temporal_2_comparison = temporal_2_df
temporal_2_distance = temporal_2_df
temporal_2_distance1 = np.array(temporal_2_comparison)

temporal_3_comparison = temporal_3_df
temporal_3_distance = temporal_3_df
temporal_3_distance1 = np.array(temporal_3_comparison)

temporal_4_comparison = temporal_4_df
temporal_4_distance = temporal_4_df
temporal_4_distance1 = np.array(temporal_4_comparison)

In [ ]:
#temporal_1_distance
var

In [ ]:
temporal_1 = 'yr1'
temporal_2= 'yr2'
temporal_3= 'yr3'
temporal_4= 'yr4'
topic

In [ ]:
temporallist = [temporal_1,temporal_2,temporal_3,temporal_4]
llist = [temporal_1_comparison,temporal_2_comparison,temporal_3_comparison,temporal_4_comparison]

#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Jaccard\tp"+str(topic)+"\\"+str(drugnamepath))
#os.chdir(r"G:\Research\FAERS_Public\Results\Jaccard\tp10"+str('\\')+str(drugnamepath))
os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Jaccard\tp"+str(topic)+"\\"+str(drugnamepath))
         
sns.set(font_scale=2, rc={'figure.figsize':(50,50)})


plt.tick_params(labelsize=25)
plt.xticks(fontsize=35, rotation=90) #font size for x ticks
plt.yticks(fontsize=35) #font size for x ticks
for x,y in zip(llist,temporallist):
    number = y
    ax = sns.heatmap(x,annot=True, vmin=0, vmax=1.5)#change here for input

    #ax.set_ylabel('FAERS (Jaccard Index Coefficent (10 Topics)')
    #ax.set_xlabel("FAERS (Jaccard Index Coefficent (10 Topics), spatial: "+str(countofcountries)+" countires, Temporal: "+str(y)+" Drug: "+str(drugname))#change here
    for x in range(0,len(var)):
    
        ax.add_patch(Rectangle((x, x), 1, 1, fill=False, edgecolor='#ffff00', lw=10))
        
    #plt.title("Jaccard Index Coefficent (10 Topics)", fontsize=50)#change here
    plt.savefig("Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_topic"+str(topic)+"_version_naive.png")

    
    

    plt.show()
    plt.clf()

for x in range(0,len(var)):
    print(str(var[x])+'2016_all =  np.array([')
    for y in range(0,len(var)):
        print('np.sqrt(np.sum(('+str(var[x])+'2016 - '+str(var[y])+"2016)**2)),")
        #print(var[x], var[y])
    print('])')


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
temporal_1_mat = np.array([[temporal_1_distance1]])#lenalid_distance2
temporal_1_mat = temporal_1_mat.reshape(len(var), len(var))
########
temporal_2_mat = np.array([[temporal_2_distance1]])#lenalid_distance2
temporal_2_mat = temporal_2_mat.reshape(len(var), len(var))
########
temporal_3_mat = np.array([[temporal_3_distance1]])#lenalid_distance2
temporal_3_mat = temporal_3_mat.reshape(len(var), len(var))
########
temporal_4_mat = np.array([[temporal_4_distance1]])#lenalid_distance2
temporal_4_mat = temporal_4_mat.reshape(len(var), len(var))

In [ ]:
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\dendo\tp"+str(topic)+"\\"+str(drugnamepath))
#os.chdir(r"D:\Research\FAERS_Public\Journal_submission\IJGI_Dec2019\images_moran\dendo")
os.chdir(r"D:\Research\FAERS_Public_LDA\Results\dendo\tp"+str(topic)+"\\"+str(drugnamepath))

import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage

linkagemethod =['single','complete']
#complete


temporallist = [temporal_1,temporal_2,temporal_3,temporal_4]
llist = [temporal_1_distance1,temporal_2_distance1,temporal_3_distance1,temporal_4_distance1]
for lm in linkagemethod:
    print(lm)
    for x,y in zip(llist,temporallist):
        plt.figure(figsize=(25, 15))
        plt.tick_params(axis='both', left='on', top='off', right='on', bottom='on', labelleft='on', labeltop='off', labelright='on', labelbottom='on')
        sns.set(font_scale=3)#, rc={'figure.figsize':(400,400)})

        number = y
        #year ='y'+str(number)+'_comparison'
        #print(year)    
        #quarter ='q'+str(number)+'_comparison'
       # print(quarter)
        _mat = np.array([[x]])#lenalid_distance2
        _mat = _mat.reshape(len(var), len(var))
        Z = linkage(_mat, lm)#LBNE_mat_distArray





        plt.xticks(fontsize=5, rotation=90) #font size for x ticks
        plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        dendrogram(
            Z,
            leaf_rotation=90.,  # rotates the x axis labels
            leaf_font_size=35.,
            labels=temporal_1_df.index, count_sort=False, distance_sort=True,color_threshold=0.0 )#, color_threshold=0.212)#, color_threshold=0.3)#, color_threshold=0.02)

        plt.title("Hierarchical Cluster Analysis (Linkage Method = "+str(lm)+", 10 Topics), Pharmaceutical Drug: "+str(drugname)+", Temporal: "+str(y), fontsize=30)#Change here

        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        plt.box('off')

        plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.png", dpi=300)

       # plt.show()
        plt.clf()

In [ ]:
#single
#complete
import seaborn as sns
import scipy.spatial as sp, scipy.cluster.hierarchy as hc
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Dendo_plus_Jaccard\tp"+str(topic)+"\\"+str(drugnamepath))
os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Dendo_plus_Jaccard\tp"+str(topic)+"\\"+str(drugnamepath))

sns.set(font_scale=2, rc={'figure.figsize':(50,50)})

#single
#complete
#quarterlst1 = [q1_distance,q2_distance,q3_distance,q4_distance]
#llist = [q1_distance1,q2_distance1,q3_distance1,q4_distance1]
#llist = [y1_distance1,y2_distance1,y3_distance1,y4_distance1]

temporallistz = [temporal_1_distance,temporal_2_distance,temporal_3_distance,temporal_4_distance]
temporallist = [temporal_1,temporal_2,temporal_3,temporal_4]
llist = [temporal_1_distance1,temporal_2_distance1,temporal_3_distance1,temporal_4_distance1]
linkagemethod =['single','complete']
for lm in linkagemethod:
    for x,y,z in zip(llist,temporallist,temporallistz):
        plt.figure(figsize=(25, 15))
        #plt.tick_params(axis='both', left='on', top='off', right='on', bottom='on', labelleft='on', labeltop='off', labelright='on', labelbottom='on')
        sns.set(font_scale=1.0)#, rc={'figure.figsize':(400,400)})

        number = y
        #year ='y'+str(number)+'_comparison'
        #print(year)    
        #quarter ='q'+str(number)+'_comparison'
       # print(quarter)
        _mat = np.array([[x]])#lenalid_distance2
        _mat = _mat.reshape(len(var), len(var))
        Z = linkage(_mat, lm)

    #for x,y,z in zip(llist,range(1,5),quarterlst1):
        #sns.set(font_scale=1, rc={'figure.figsize':(200,200)})
        #number = y

        #year ='y'+str(number)+'_comparison'

        #_mat = np.array([[x]])#lenalid_distance2
        #_mat = _mat.reshape(14, 14)
        #Z = linkage(_mat, 'single')#LBNE_mat_distArray


        sns.clustermap(z,  row_linkage=Z, col_linkage=Z)# change here


        plt.title("Hierarchical Clustering (Method = "+str(lm)+", Top 10 frequent items), drug name: "+str(drugname)+", temporal: "+str(y), fontsize=15)#Change here

       # plt.title("Hierarchical Clustering (Method = "+str(linkagemethod)+", Top 4 frequent items), count of countries: "+str(countofcountries)+" Drug: ",+str(drugname)+
               #   " temporal: "+str(y))

    #    #plt.savefig("Dendrogram_Tree_plus_jaccard_FAERs_Similiarity_Top_4_frequent_items_19_countries_minsup_0dot01_"+year+".png", dpi=300, bbox_inches="tight")
        plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_"+str(lm)+"_.png", dpi=300, bbox_inches="tight")
        plt.show()
        plt.clf()

## Spatial Autocorrelation Test

In [ ]:
from scipy.cluster.hierarchy import linkage

_1row_clusters_single = linkage(temporal_1_distance1, method='single')#lenalid_distance2
_2row_clusters_single = linkage(temporal_2_distance1, method='single')
_3row_clusters_single = linkage(temporal_3_distance1, method='single')
_4row_clusters_single = linkage(temporal_4_distance1, method='single')

from scipy.cluster.hierarchy import linkage

_1row_clusters_complete = linkage(temporal_1_distance1, method='complete')
_2row_clusters_complete = linkage(temporal_2_distance1, method='complete')#complete
_3row_clusters_complete = linkage(temporal_3_distance1, method='complete')
_4row_clusters_complete = linkage(temporal_4_distance1, method='complete')

In [ ]:
from scipy.cluster.hierarchy import fcluster
#import pysal as ps 

#lenalid_distance = 1-lenalid_df
#bupreno_distance = 1-bupreno_df
#etanerc_distance = 1-etanerc_df
#nataliz_distance = 1-nataliz_df

#list1 = [irow_clusters_single, terow_clusters_single, tirow_clusters_single, prow_clusters_single,
 #       irow_clusters_complete, terow_clusters_complete, tirow_clusters_complete, prow_clusters_complete]
#list2 = ['irow_clusters_single', 'terow_clusters_single', 'tirow_clusters_single', 'prow_clusters_single',
 #       'irow_clusters_complete', 'terow_clusters_complete', 'tirow_clusters_complete', 'prow_clusters_complete']

cluster_output_appended_data = []
list1 = [_1row_clusters_single, _2row_clusters_single, _3row_clusters_single, _4row_clusters_single,
        _1row_clusters_complete, _2row_clusters_complete, _3row_clusters_complete, _4row_clusters_complete]
list2 = ['yr1row_clusters_single', 'yr2row_clusters_single', 'yr3row_clusters_single', 'yr4row_clusters_single',
        'yr1row_clusters_complete', 'yr2row_clusters_complete', 'yr3row_clusters_complete', 'yr4row_clusters_complete']

#list1 = [lrow_clusters_complete, brow_clusters_complete, nrow_clusters_complete, erow_clusters_complete]

#for x,x1 in zip(list1, list2):
 #   assignments = fcluster(x,0.1,'distance')  #0.45 e
    #labels=interfe_df.index###################################################
  #  labels=lenalid_df.index############################################################

#    print(set(assignments))
 #   print(str(x1))
  #  cluster_output = pd.DataFrame({'STATE_ABBR':labels , 'cluster':assignments})


list2 = np.linspace(0.0, 1, num=10)

for x,y in zip(list1, list2):
        #for x in range(1,12):#new

    #print(x+str(y))
    assignments = fcluster(x, y,"distance")  #0.45 ey,'distance',y*nataliz_distance2.max()
   # labels=interfe_df.index#######################################################################change this one
    labels=temporal_1_df.index#######################################################################change this one

    #print('cluster cut off point= ', str(y))############
    #print(set(assignments))
    cluster_output = pd.DataFrame({'occr_country':labels , 'cluster':assignments})
    #display(cluster_output)
    cluster_output_appended_data.append(cluster_output)
# see pd.concat documentation for more info
cluster_output_appended_data_1 = pd.concat(cluster_output_appended_data, axis=1)
cluster_output_appended_data_0 = pd.concat(cluster_output_appended_data, axis=0)
cluster_output_appended_data_0
   # mx = np.max(clusters_assignments)
   # mx = mx+1
   # print(mx-1, 'clusters')
#cluster_output['cluster1'] = (cluster_output['cluster'] ==1).astype(int)
#cluster_output['cluster2'] = (cluster_output['cluster'] ==2).astype(int)
#cluster_output['cluster3'] = (cluster_output['cluster'] ==3).astype(int)
#cluster_output['cluster4'] = (cluster_output['cluster'] ==4).astype(int)
#cluster_output['cluster5'] = (cluster_output['cluster'] ==5).astype(int)
#cluster_output['cluster6'] = (cluster_output['cluster'] ==6).astype(int)
#cluster_output['cluster7'] = (cluster_output['cluster'] ==7).astype(int)
#cluster_output['cluster8'] = (cluster_output['cluster'] ==8).astype(int)
#cluster_output['cluster9'] = (cluster_output['cluster'] ==9).astype(int)
#cluster_output['cluster10'] = (cluster_output['cluster'] ==10).astype(int)
#cluster_output['cluster11'] = (cluster_output['cluster'] ==11).astype(int)
#cluster_output['cluster12'] = (cluster_output['cluster'] ==12).astype(int)
#cluster_output['cluster13'] = (cluster_output['cluster'] ==13).astype(int)
#cluster_output['cluster14'] = (cluster_output['cluster'] ==14).astype(int)
#cluster_output['cluster15'] = (cluster_output['cluster'] ==15).astype(int)
#cluster_output['cluster16'] = (cluster_output['cluster'] ==16).astype(int)
#cluster_output['cluster17'] = (cluster_output['cluster'] ==17).astype(int)
#cluster_output['cluster18'] = (cluster_output['cluster'] ==18).astype(int)
#cluster_output['cluster19'] = (cluster_output['cluster'] ==19).astype(int)
#cluster_output['cluster20'] = (cluster_output['cluster'] ==20).astype(int)
#cluster_output['cluster21'] = (cluster_output['cluster'] ==21).astype(int)
#cluster_output['cluster22'] = (cluster_output['cluster'] ==22).astype(int)
#cluster_output['cluster23'] = (cluster_output['cluster'] ==23).astype(int)
#cluster_output['cluster24'] = (cluster_output['cluster'] ==24).astype(int)
#cluster_output['cluster25'] = (cluster_output['cluster'] ==25).astype(int)
#cluster_output['cluster26'] = (cluster_output['cluster'] ==26).astype(int)
#cluster_output['cluster27'] = (cluster_output['cluster'] ==27).astype(int)
#cluster_output['cluster28'] = (cluster_output['cluster'] ==28).astype(int)
#cluster_output['cluster29'] = (cluster_output['cluster'] ==29).astype(int)

#os.chdir(r"G:\Research\FAERS\Drug_Spatial\L_B_N_E\Data\Analysis_ready_data\Approri\summary_of_approri_dataset\spatial_auto\LBNE_method_single_complete_csv_cluster_at_different_cutoff")

#cluster_output.to_csv('spatial_auto_cutooff_0dot8_etanerc_method_single.csv')

#cluster_output
#clu = pd.get_dummies(cluster_output['cluster'])

#cluster_output_appended_data_0

In [ ]:
#spatial_list1 = [x for x in var if x != "ee"]
len(var)
spatial_list = var
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from libpysal.weights import fuzzy_contiguity

#os.chdir(r"C:\Users\AhmedAskar\Desktop\FAERS_Public\Data\Helper_dataset\World")
#plt.title("Etanercept Hierarchical Clustering, method:single, (min sup=0.2, top (2) frequent items), cluster 4")

#world_shp = gpd.read_file(filename='world.shp')

#world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.strip()
#world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.lower()

#world_join = world_shp.merge(cluster_output, left_on='ISO_2DIGIT', right_on='occr_country', how='inner')
#cluster_output
#usstates_join = pd.concat([usstates_shp, cluster_output], ignore_index=True)
#columns = ['ObjectID', 'FIPS_CNTRY', 'GMI_CNTRY', 'ISO_2DIGIT', 'ISO_3DIGIT',
#       'ISO_NUM', 'CNTRY_NAME', 'LONG_NAME', 'ISOSHRTNAM', 'UNSHRTNAM',
 #      'LOCSHRTNAM', 'LOCLNGNAM', 'STATUS', 'POP2007', 'SQKM', 'SQMI',
 #      'LAND_SQKM', 'COLORMAP']
#world_join.drop(columns, axis=1, inplace=True)
#cb = world_join['cluster']
#usstates_join.to_csv('ttest.csv')
#sns.set(font_scale=1, rc={'figure.figsize':(200,200)})
os.chdir(r"D:\shapefiles\World")
#os.chdir(r"\\Fdswp03752\ocmgis_data\Scripts\Situation_Awareness_Natural_hazards\shapefiles\World")
#os.chdir(r"\\Fdswp03752\ocmgis_data\Scripts\Situation_Awareness_Natural_hazards\shapefiles\World")

world_shp = gpd.read_file(filename='world.shp')
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.strip()#ISOSHRTNAM
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.lower()
#world_shp.head(1)
#cluster_output


In [ ]:
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from libpysal.weights import fuzzy_contiguity

#os.chdir(r"C:\Users\AhmedAskar\Desktop\FAERS_Public\Data\Helper_dataset\World")
#plt.title("Etanercept Hierarchical Clustering, method:single, (min sup=0.2, top (2) frequent items), cluster 4")

#world_shp = gpd.read_file(filename='world.shp')

#world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.strip()
#world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.lower()

#world_join = world_shp.merge(cluster_output, left_on='ISO_2DIGIT', right_on='occr_country', how='inner')
#cluster_output
#usstates_join = pd.concat([usstates_shp, cluster_output], ignore_index=True)
#columns = ['ObjectID', 'FIPS_CNTRY', 'GMI_CNTRY', 'ISO_2DIGIT', 'ISO_3DIGIT',
#       'ISO_NUM', 'CNTRY_NAME', 'LONG_NAME', 'ISOSHRTNAM', 'UNSHRTNAM',
 #      'LOCSHRTNAM', 'LOCLNGNAM', 'STATUS', 'POP2007', 'SQKM', 'SQMI',
 #      'LAND_SQKM', 'COLORMAP']
#world_join.drop(columns, axis=1, inplace=True)
#cb = world_join['cluster']
#usstates_join.to_csv('ttest.csv')
#sns.set(font_scale=1, rc={'figure.figsize':(200,200)})
os.chdir(r"D:\shapefiles\World")
#os.chdir(r"\\Fdswp03752\ocmgis_data\Scripts\Situation_Awareness_Natural_hazards\shapefiles\World")
#os.chdir(r"\\Fdswp03752\ocmgis_data\Scripts\Situation_Awareness_Natural_hazards\shapefiles\World")

world_shp = gpd.read_file(filename='world.shp')
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.strip()#ISOSHRTNAM
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.lower()
world_shp.head()
world_shp.ISO_2DIGIT
#europe = ['AL', 'AD', 'AM', 'AT', 'BY', 'BE', 'BA', 'BG', 'CH', 'CY', 'CZ', 'DE',
 #       'DK', 'EE', 'ES', 'FO', 'FI', 'FR', 'GB', 'GE', 'GI', 'GR', 'HU', 'HR',
   #     'IE', 'IS', 'IT', 'LT', 'LU', 'LV', 'MC', 'MK', 'MT', 'NO', 'NL', 'PL',
  #      'PT', 'RO', 'SE', 'SI', 'SK', 'SM', 'TR', 'UA', 'VA']
#spatial_list = ["de","fr","gb","it","be","ch","es","nl","cz","se","gr","at","ie","pt","dk","no","sk","fi","hu"] # took out us, ca, pr, ru
world_shp2 = world_shp[world_shp['ISO_2DIGIT'].str.contains('|'.join(spatial_list),case=False, na=False, regex=True)]
#world_shp1 = world_shp[world_shp['ISO_2DIGIT'].str.contains('|'.join(europe),case=False, na=False, regex=True)]
world_shp2['centroid'] = world_shp2['geometry'].centroid
world_shp2['coords'] = world_shp2['geometry'].apply(lambda x: x.representative_point().coords[:])
world_shp2['coords'] = [coords[0] for coords in world_shp2['coords']]
world_shp2['coords']

#world_shp1['Lon'] = world_shp1.centroid.x
#world_shp1['Lat'] = world_shp1.centroid.y

world_join = world_shp2.merge(cluster_output, left_on='ISO_2DIGIT', right_on='occr_country', how='inner')
#cluster_output
#usstates_join = pd.concat([usstates_shp, cluster_output], ignore_index=True)
columns = ['ObjectID', 'FIPS_CNTRY', 'GMI_CNTRY', 'ISO_2DIGIT', 'ISO_3DIGIT',
       'ISO_NUM', 'LONG_NAME', 'ISOSHRTNAM', 'UNSHRTNAM',
       'LOCSHRTNAM', 'LOCLNGNAM', 'STATUS', 'POP2007', 'SQKM', 'SQMI',
       'LAND_SQKM', 'COLORMAP']
world_join = world_join.sort_values(by=['occr_country'], ascending=True)
world_join.drop(columns, axis=1, inplace=True)
cb = world_join['cluster']
#usstates_join.to_csv('ttest.csv')
sns.set(font_scale=1, rc={'figure.figsize':(200,200)})

world_join
temporal_1_df
labels
world_join.reset_index(drop=True, inplace=True)
world_join.info()
world_join['CNTRY_NAME']

## Spatial Weights and neigbors

In [ ]:
wf = fuzzy_contiguity(world_join)
wf[0]
Wmatrix, ids = wf.full()
wf.weights

In [ ]:
1-temporal_1_comparison[['ch','de','it','si','cz','hu','sk']].iloc[0]
#-0.31012435629837065, 0.33605636310312814, -0.3935991547652453, 0.16953451925539775, -0.20440984717637356, 0.9977914094924927, -0.39887803588359283, ],


In [ ]:
#wq = libpysal.weights.Queen.from_dataframe(world_join)
weights=[]
wf = fuzzy_contiguity(world_join)
wf[0]
Wmatrix, ids = wf.full()
#Wmatrix
n_neighbors = Wmatrix.sum(axis=1)
#qW.cardinalities[0]
#qW.transform = 'r'
#Wmatrix, ids = qW.full()
#Wmatrix.sum(axis=1)
#world_join.plot()
#Wmatrix[0]=temporal_1_distance1[0]
#qW[0] =Wmatrix[0]
#Wmatrix[0]
n_neighbors[1]
print('={')
#Wmatrix
for x in range(0,len(world_join['occr_country'])):
   # print(x)
    print(str(x)+':[')
    #print(list(wf[x]))
    for y in list(wf[x]):
        #print(x,y)
        #weights.append(1-temporal_4_comparison.iloc[x][y])
        #print('temporal_1_comparison.iloc['+str(x)+']['+str(y)+']', end = ',')
        #print(1-temporal_1_comparison.iloc[x][y],  end = ', ')
        #print(1-temporal_2_comparison.iloc[x][y],  end = ', ')
        #print(1-temporal_3_comparison.iloc[x][y],  end = ', ')
        print(1+temporal_4_comparison.iloc[x][y],  end = ', ')

    print('],',sep=',')
print('}')
    #print(wf[x].keys())
    #print(qW[x].values())
#n_neighbors[0]
#wf.cardinalities[0]
#list(wf[0])


In [ ]:
we1={
0:[
0.7333427788281248, 0.7500215623025277, -0.4030000388137083, -0.3949982013918871, -0.3660186821606526, -0.39959891581006235, 0.9679064750671387, ],
1:[
0.8123375630691299, -0.40200140628646297, 0.7483371450740779, ],
2:[
-0.16007266268241382, -0.16835961131701493, ],
3:[
-0.1803673140043318, ],
4:[
0.8792613471422818, 0.7385658869443741, -0.1363635191949013, 0.7333427788281248, ],
5:[
-0.25748881200455553, -0.3660186821606526, -0.35725698860259425, -0.3429134360339998, ],
6:[
0.7874353103941061, 0.7483371450740779, -0.2946521485477882, 0.7385658869443741, 0.7510859233852454, 0.7500215623025277, -0.25748881200455553, -0.14325142050327955, ],
7:[
0.7510859233852454, ],
8:[
],
9:[
0.7109577194184405, 0.8529250578005367, ],
10:[
-0.23447423791667554, -0.30881411843853424, ],
11:[
0.8529250578005367, 0.8123375630691299, 0.8792613471422818, 0.7874353103941061, -0.23398832095929034, ],
12:[
0.8419642860216574, ],
13:[
-0.16007266268241382, 0.989957332611084, ],
14:[
-0.2867987502745779, 0.8530625145035703, ],
15:[
-0.39959891581006235, -0.39285503303914227, 0.8530625145035703, -0.3770572180579208, ],
16:[
0.8419642860216574, ],
17:[
-0.23398832095929034, -0.1363635191949013, -0.4030000388137083, -0.39627258328631343, ],
18:[
-0.40200140628646297, -0.2946521485477882, ],
19:[
0.6016416497828834, -0.30881411843853424, ],
20:[
-0.14325142050327955, -0.35725698860259425, -0.37054766926517635, -0.1803673140043318, ],
21:[
0.7109577194184405, ],
22:[
0.6016416497828834, -0.23447423791667554, ],
23:[
-0.39627258328631343, -0.3949982013918871, -0.2867987502745779, -0.39285503303914227, ],
24:[
0.9679064750671387, -0.3429134360339998, -0.37054766926517635, -0.3770572180579208, ],
25:[
0.989957332611084, -0.16835961131701493, ],
}

we2={
0:[
-0.38135309194062583, 0.6045674386343609, -0.3201298880985226, -0.38669526787042074, 0.9922850728034973, -0.3982934931621187, -0.1865449955315317, ],
1:[
-0.18738291814362196, -0.1945892382961527, 0.7643575346533376, ],
2:[
0.7458911168055654, -0.26270532213100695, ],
3:[
-0.31219606214043183, ],
4:[
-0.3821193212406546, -0.2071005725125934, -0.21434139724953805, -0.38135309194062583, ],
5:[
0.5985333990261046, 0.9922850728034973, -0.34091425647270435, -0.19275303150006717, ],
6:[
-0.2241212078288155, 0.7643575346533376, -0.23135487468268257, -0.2071005725125934, -0.07702898211861675, 0.6045674386343609, 0.5985333990261046, -0.15458511714223278, ],
7:[
-0.07702898211861675, ],
8:[
],
9:[
-0.32171186775166727, -0.32851063386180557, ],
10:[
0.9795175194740295, -0.29547169845700894, ],
11:[
-0.32851063386180557, -0.18738291814362196, -0.3821193212406546, -0.2241212078288155, 0.8322150592539417, ],
12:[
-0.39253585437966576, ],
13:[
0.7458911168055654, -0.2711504040728121, ],
14:[
-0.3868312544069139, -0.3984283518597427, ],
15:[
-0.3982934931621187, -0.3895138029456795, -0.3984283518597427, 0.6925438530308934, ],
16:[
-0.39253585437966576, ],
17:[
0.8322150592539417, -0.21434139724953805, -0.3201298880985226, -0.21967566775478575, ],
18:[
-0.1945892382961527, -0.23135487468268257, ],
19:[
-0.2801030775117337, -0.29547169845700894, ],
20:[
-0.15458511714223278, -0.34091425647270435, -0.11755618086963304, -0.31219606214043183, ],
21:[
-0.32171186775166727, ],
22:[
-0.2801030775117337, 0.9795175194740295, ],
23:[
-0.21967566775478575, -0.38669526787042074, -0.3868312544069139, -0.3895138029456795, ],
24:[
-0.1865449955315317, -0.19275303150006717, -0.11755618086963304, 0.6925438530308934, ],
25:[
-0.2711504040728121, -0.26270532213100695, ],
}


we3 ={
0:[
0.10607097811653732, 0.5836652681858798, 0.2900569585575209, -0.009984012898243355, -0.21035336781431457, -0.20055906933592627, -0.1968856478907819, ],
1:[
-0.2572989122500575, 0.6601651066422136, -0.0853370780163012, ],
2:[
0.9951184391975403, -0.3997004527447361, ],
3:[
-0.38714294355782064, ],
4:[
-0.1337568721671356, 0.0559014148510657, 0.2983814191380908, 0.10607097811653732, ],
5:[
-0.2446803174257024, -0.21035336781431457, -0.40803024195205473, -0.3975703222177407, ],
6:[
-0.23156691528841544, -0.0853370780163012, -0.23960231107229069, 0.0559014148510657, -0.16626456179389604, 0.5836652681858798, -0.2446803174257024, -0.24347525733407904, ],
7:[
-0.16626456179389604, ],
8:[
],
9:[
-0.23893439446595433, -0.2634350047467453, ],
10:[
-0.24608768871768394, 0.16312907327266568, ],
11:[
-0.2634350047467453, -0.2572989122500575, -0.1337568721671356, -0.23156691528841544, 0.39207737996953174, ],
12:[
0.9896384477615356, ],
13:[
0.9951184391975403, -0.40317043522521834, ],
14:[
0.20819202867041509, -0.18471053333849397, ],
15:[
-0.20055906933592627, -0.2317150895041411, -0.18471053333849397, -0.38909668832989874, ],
16:[
0.9896384477615356, ],
17:[
0.39207737996953174, 0.2983814191380908, 0.2900569585575209, 0.13653854926340803, ],
18:[
0.6601651066422136, -0.23960231107229069, ],
19:[
-0.17112000867592259, 0.16312907327266568, ],
20:[
-0.24347525733407904, -0.40803024195205473, -0.39647024364969763, -0.38714294355782064, ],
21:[
-0.23893439446595433, ],
22:[
-0.17112000867592259, -0.24608768871768394, ],
23:[
0.13653854926340803, -0.009984012898243355, 0.20819202867041509, -0.2317150895041411, ],
24:[
-0.1968856478907819, -0.3975703222177407, -0.39647024364969763, -0.38909668832989874, ],
25:[
-0.40317043522521834, -0.3997004527447361, ],
}

we4={
0:[
0.8696592148884608, 0.8734780356064313, -0.26270838691016984, 0.03987650895681527, -0.19387212269638376, 0.0614229212294628, -0.19112317198054307, ],
1:[
-0.3915521883566948, 0.9979463219642639, -0.1790893609503188, ],
2:[
-0.321969558428695, -0.3862594915992117, ],
3:[
-0.07300546654505502, ],
4:[
-0.36252324406499614, 0.743924530956515, -0.35392893848652474, 0.8696592148884608, ],
5:[
-0.10576660801795512, -0.19387212269638376, -0.1938716024706415, -0.17037131521072446, ],
6:[
-0.18947176847703462, -0.1790893609503188, -0.18073447040570834, 0.743924530956515, -0.0980290162556603, 0.8734780356064313, -0.10576660801795512, 0.8734788580627172, ],
7:[
-0.0980290162556603, ],
8:[
],
9:[
0.9952900409698486, -0.4013793521719198, ],
10:[
0.024658785292094865, -0.17409908350390335, ],
11:[
-0.4013793521719198, -0.3915521883566948, -0.36252324406499614, -0.18947176847703462, 0.9881431460380554, ],
12:[
-0.20514202230245493, ],
13:[
-0.321969558428695, -0.31313870619263406, ],
14:[
0.6819292875637413, -0.10523646961939348, ],
15:[
0.0614229212294628, 0.07936557362210417, -0.10523646961939348, -0.004017586649421201, ],
16:[
-0.20514202230245493, ],
17:[
0.9881431460380554, -0.35392893848652474, -0.26270838691016984, -0.2557964844706646, ],
18:[
0.9979463219642639, -0.18073447040570834, ],
19:[
-0.21172217633738888, -0.17409908350390335, ],
20:[
0.8734788580627172, -0.1938716024706415, -0.19112264762207865, -0.07300546654505502, ],
21:[
0.9952900409698486, ],
22:[
-0.21172217633738888, 0.024658785292094865, ],
23:[
-0.2557964844706646, 0.03987650895681527, 0.6819292875637413, 0.07936557362210417, ],
24:[
-0.19112317198054307, -0.17037131521072446, -0.19112264762207865, -0.004017586649421201, ],
25:[
-0.31313870619263406, -0.3862594915992117, ],
}


we0={0: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 1: [1.0, 1.0, 1.0],
 2: [1.0, 1.0],
 3: [1.0],
 4: [1.0, 1.0, 1.0, 1.0],
 5: [1.0, 1.0, 1.0, 1.0],
 6: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 7: [1.0],
 8: [],
 9: [1.0, 1.0],
 10: [1.0, 1.0],
 11: [1.0, 1.0, 1.0, 1.0, 1.0],
 12: [1.0],
 13: [1.0, 1.0],
 14: [1.0, 1.0],
 15: [1.0, 1.0, 1.0, 1.0],
 16: [1.0],
 17: [1.0, 1.0, 1.0, 1.0],
 18: [1.0, 1.0],
 19: [1.0, 1.0],
 20: [1.0, 1.0, 1.0, 1.0],
 21: [1.0],
 22: [1.0, 1.0],
 23: [1.0, 1.0, 1.0, 1.0],
 24: [1.0, 1.0, 1.0, 1.0],
 25: [1.0, 1.0]}

we5={
0:[
1.2666572211718752, 1.2499784376974723, 2.4030000388137083, 2.3949982013918873, 2.366018682160653, 2.3995989158100626, 1.0320935249328613, ],
1:[
1.1876624369308701, 2.402001406286463, 1.251662854925922, ],
2:[
2.160072662682414, 2.168359611317015, ],
3:[
2.180367314004332, ],
4:[
1.1207386528577183, 1.261434113055626, 2.1363635191949015, 1.2666572211718752, ],
5:[
2.2574888120045555, 2.366018682160653, 2.357256988602594, 2.342913436034, ],
6:[
1.212564689605894, 1.251662854925922, 2.2946521485477884, 1.261434113055626, 1.2489140766147546, 1.2499784376974723, 2.2574888120045555, 2.1432514205032795, ],
7:[
1.2489140766147546, ],
8:[
],
9:[
1.2890422805815596, 1.1470749421994633, ],
10:[
2.2344742379166753, 2.3088141184385345, ],
11:[
1.1470749421994633, 1.1876624369308701, 1.1207386528577183, 1.212564689605894, 2.2339883209592903, ],
12:[
1.1580357139783426, ],
13:[
2.160072662682414, 1.010042667388916, ],
14:[
2.2867987502745777, 1.1469374854964298, ],
15:[
2.3995989158100626, 2.392855033039142, 1.1469374854964298, 2.377057218057921, ],
16:[
1.1580357139783426, ],
17:[
2.2339883209592903, 2.1363635191949015, 2.4030000388137083, 2.396272583286313, ],
18:[
2.402001406286463, 2.2946521485477884, ],
19:[
1.3983583502171166, 2.3088141184385345, ],
20:[
2.1432514205032795, 2.357256988602594, 2.3705476692651763, 2.180367314004332, ],
21:[
1.2890422805815596, ],
22:[
1.3983583502171166, 2.2344742379166753, ],
23:[
2.396272583286313, 2.3949982013918873, 2.2867987502745777, 2.392855033039142, ],
24:[
1.0320935249328613, 2.342913436034, 2.3705476692651763, 2.377057218057921, ],
25:[
1.010042667388916, 2.168359611317015, ],
}


we6={
0:[
2.3813530919406256, 1.395432561365639, 2.3201298880985224, 2.3866952678704205, 1.0077149271965027, 2.398293493162119, 2.1865449955315315, ],
1:[
2.187382918143622, 2.1945892382961527, 1.2356424653466624, ],
2:[
1.2541088831944345, 2.2627053221310067, ],
3:[
2.3121960621404316, ],
4:[
2.382119321240655, 2.2071005725125934, 2.2143413972495383, 2.3813530919406256, ],
5:[
1.4014666009738954, 1.0077149271965027, 2.340914256472704, 2.192753031500067, ],
6:[
2.2241212078288157, 1.2356424653466624, 2.2313548746826823, 2.2071005725125934, 2.077028982118617, 1.395432561365639, 1.4014666009738954, 2.1545851171422328, ],
7:[
2.077028982118617, ],
8:[
],
9:[
2.3217118677516675, 2.3285106338618053, ],
10:[
1.0204824805259705, 2.2954716984570087, ],
11:[
2.3285106338618053, 2.187382918143622, 2.382119321240655, 2.2241212078288157, 1.1677849407460583, ],
12:[
2.3925358543796658, ],
13:[
1.2541088831944345, 2.271150404072812, ],
14:[
2.386831254406914, 2.3984283518597427, ],
15:[
2.398293493162119, 2.3895138029456797, 2.3984283518597427, 1.3074561469691066, ],
16:[
2.3925358543796658, ],
17:[
1.1677849407460583, 2.2143413972495383, 2.3201298880985224, 2.2196756677547858, ],
18:[
2.1945892382961527, 2.2313548746826823, ],
19:[
2.2801030775117335, 2.2954716984570087, ],
20:[
2.1545851171422328, 2.340914256472704, 2.117556180869633, 2.3121960621404316, ],
21:[
2.3217118677516675, ],
22:[
2.2801030775117335, 1.0204824805259705, ],
23:[
2.2196756677547858, 2.3866952678704205, 2.386831254406914, 2.3895138029456797, ],
24:[
2.1865449955315315, 2.192753031500067, 2.117556180869633, 1.3074561469691066, ],
25:[
2.271150404072812, 2.2627053221310067, ],
}


we7={
0:[
1.8939290218834626, 1.4163347318141202, 1.7099430414424792, 2.0099840128982436, 2.2103533678143146, 2.2005590693359265, 2.196885647890782, ],
1:[
2.2572989122500573, 1.3398348933577864, 2.085337078016301, ],
2:[
1.0048815608024597, 2.399700452744736, ],
3:[
2.3871429435578206, ],
4:[
2.133756872167136, 1.9440985851489343, 1.701618580861909, 1.8939290218834626, ],
5:[
2.2446803174257024, 2.2103533678143146, 2.408030241952055, 2.3975703222177405, ],
6:[
2.2315669152884157, 2.085337078016301, 2.2396023110722907, 1.9440985851489343, 2.166264561793896, 1.4163347318141202, 2.2446803174257024, 2.243475257334079, ],
7:[
2.166264561793896, ],
8:[
],
9:[
2.2389343944659545, 2.2634350047467455, ],
10:[
2.246087688717684, 1.8368709267273342, ],
11:[
2.2634350047467455, 2.2572989122500573, 2.133756872167136, 2.2315669152884157, 1.6079226200304682, ],
12:[
1.0103615522384644, ],
13:[
1.0048815608024597, 2.4031704352252183, ],
14:[
1.791807971329585, 2.184710533338494, ],
15:[
2.2005590693359265, 2.2317150895041413, 2.184710533338494, 2.3890966883298987, ],
16:[
1.0103615522384644, ],
17:[
1.6079226200304682, 1.701618580861909, 1.7099430414424792, 1.863461450736592, ],
18:[
1.3398348933577864, 2.2396023110722907, ],
19:[
2.1711200086759224, 1.8368709267273342, ],
20:[
2.243475257334079, 2.408030241952055, 2.3964702436496976, 2.3871429435578206, ],
21:[
2.2389343944659545, ],
22:[
2.1711200086759224, 2.246087688717684, ],
23:[
1.863461450736592, 2.0099840128982436, 1.791807971329585, 2.2317150895041413, ],
24:[
2.196885647890782, 2.3975703222177405, 2.3964702436496976, 2.3890966883298987, ],
25:[
2.4031704352252183, 2.399700452744736, ],
}


we8={
0:[
1.1303407851115392, 1.1265219643935687, 2.2627083869101696, 1.9601234910431846, 2.1938721226963835, 1.9385770787705372, 2.191123171980543, ],
1:[
2.3915521883566946, 1.002053678035736, 2.179089360950319, ],
2:[
2.321969558428695, 2.386259491599212, ],
3:[
2.0730054665450552, ],
4:[
2.362523244064996, 1.256075469043485, 2.3539289384865247, 1.1303407851115392, ],
5:[
2.105766608017955, 2.1938721226963835, 2.1938716024706415, 2.1703713152107245, ],
6:[
2.1894717684770346, 2.179089360950319, 2.1807344704057083, 1.256075469043485, 2.0980290162556603, 1.1265219643935687, 2.105766608017955, 1.126521141937283, ],
7:[
2.0980290162556603, ],
8:[
],
9:[
1.0047099590301514, 2.4013793521719196, ],
10:[
1.975341214707905, 2.1740990835039034, ],
11:[
2.4013793521719196, 2.3915521883566946, 2.362523244064996, 2.1894717684770346, 1.0118568539619446, ],
12:[
2.205142022302455, ],
13:[
2.321969558428695, 2.313138706192634, ],
14:[
1.3180707124362587, 2.1052364696193937, ],
15:[
1.9385770787705372, 1.9206344263778958, 2.1052364696193937, 2.004017586649421, ],
16:[
2.205142022302455, ],
17:[
1.0118568539619446, 2.3539289384865247, 2.2627083869101696, 2.2557964844706646, ],
18:[
1.002053678035736, 2.1807344704057083, ],
19:[
2.211722176337389, 2.1740990835039034, ],
20:[
1.126521141937283, 2.1938716024706415, 2.1911226476220786, 2.0730054665450552, ],
21:[
1.0047099590301514, ],
22:[
2.211722176337389, 1.975341214707905, ],
23:[
2.2557964844706646, 1.9601234910431846, 1.3180707124362587, 1.9206344263778958, ],
24:[
2.191123171980543, 2.1703713152107245, 2.1911226476220786, 2.004017586649421, ],
25:[
2.313138706192634, 2.386259491599212, ],
}




In [ ]:
#wq = libpysal.weights.Queen.from_dataframe(world_join)
wf = fuzzy_contiguity(world_join)
wf[0]
Wmatrix, ids = wf.full()
#Wmatrix
n_neighbors = Wmatrix.sum(axis=1)
#qW.cardinalities[0]
#qW.transform = 'r'
#Wmatrix, ids = qW.full()
#Wmatrix.sum(axis=1)
#world_join.plot()
#Wmatrix[0]=temporal_1_distance1[0]
#qW[0] =Wmatrix[0]
#Wmatrix[0]
n_neighbors[1]
#Wmatrix
print('ne1 = {')
for x in range(0,len(world_join['occr_country'])):
    
    #print('ne_'+str(x)+'=(')
    print(x,':',list(wf[x]), end=', ')
    #for y in list(wf[x]):
        #print('temporal_1_comparison.iloc['+str(x)+']['+str(y)+']', end = ',')
        #printprint(1-temporal_1_comparison.iloc[x][y], end = ',')
    
print('}')
    #print( wf[x])
    #print(wf[x].keys())
    #print(qW[x].values())
#n_neighbors[0]
#wf.cardinalities[0]
#list(wf[0])

In [ ]:
ne1 = {
0 : [4, 6, 17, 23, 5, 15, 24], 1 : [11, 18, 6], 2 : [13, 25], 3 : [20], 4 : [11, 6, 17, 0], 5 : [6, 0, 20, 24], 6 : [11, 1, 18, 4, 7, 0, 5, 20], 7 : [6], 8 : [], 9 : [21, 11], 10 : [22, 19], 11 : [9, 1, 4, 6, 17], 12 : [16], 13 : [2, 25], 14 : [23, 15], 15 : [0, 23, 14, 24], 16 : [12], 17 : [11, 4, 0, 23], 18 : [1, 6], 19 : [22, 10], 20 : [6, 5, 24, 3], 21 : [9], 22 : [19, 10], 23 : [17, 0, 14, 15], 24 : [0, 5, 20, 15], 25 : [13, 2], }

In [ ]:
from libpysal.weights import W

w1 = W(ne1, we1, silence_warnings=False)
w2 = W(ne1, we2, silence_warnings=False)
w3 = W(ne1, we3, silence_warnings=False)
w4 = W(ne1, we4, silence_warnings=False)
w0 = W(ne1, we0, silence_warnings=False)

w5 = W(ne1, we5, silence_warnings=False)
w6 = W(ne1, we6, silence_warnings=False)
w7 = W(ne1, we7, silence_warnings=False)
w8 = W(ne1, we8, silence_warnings=False)

In [ ]:
#w1.weights
#cluster_output_appended_data_0['cluster']
world_join.head()
world_join = world_shp2.merge(cluster_output_appended_data_0, left_on='ISO_2DIGIT', right_on='occr_country', how='inner')
#cluster_output
#usstates_join = pd.concat([usstates_shp, cluster_output], ignore_index=True)
columns = ['ObjectID', 'FIPS_CNTRY', 'GMI_CNTRY', 'ISO_2DIGIT', 'ISO_3DIGIT',
       'ISO_NUM', 'CNTRY_NAME', 'LONG_NAME', 'ISOSHRTNAM', 'UNSHRTNAM',
       'LOCSHRTNAM', 'LOCLNGNAM', 'STATUS', 'POP2007', 'SQKM', 'SQMI',
       'LAND_SQKM', 'COLORMAP']
world_join = world_join.sort_values(by=['occr_country'], ascending=True)
world_join.drop(columns, axis=1, inplace=True)
world_join.head()
world_join.info()

In [ ]:
#this be the oneee
from pysal.explore.esda.moran import Moran
from scipy.cluster.hierarchy import fcluster
import pysal as ps
from pysal.lib.weights import lat2W
#libpysal.weights.fuzzy_contiguity
from pysal.explore.esda.moran import Moran, Moran_Local

from pysal.explore.esda.join_counts import Join_Counts

cols = ['pvalue','zscore', 'moran_i', 'countries','dendo_cluster','dendo_cutoff','temporal','temporal2']
dat01 = pd.DataFrame(columns = cols)
dat05 = pd.DataFrame(columns = cols)
dat1 = pd.DataFrame(columns = cols)
datn = pd.DataFrame(columns = cols)


ww1 = [w1, w2, w3, w4]
ww0 = [w0, w0, w0, w0]
ww2 = [w5, w6, w7, w8]

#list1 = [_1row_clusters_single, _2row_clusters_single, _3row_clusters_single, _4row_clusters_single,
   #     _1row_clusters_complete, _2row_clusters_complete, _4row_clusters_complete, _4row_clusters_complete]

list1 = [_1row_clusters_single, _2row_clusters_single, _3row_clusters_single, _4row_clusters_single,
        _1row_clusters_complete, _2row_clusters_complete, _3row_clusters_complete, _4row_clusters_complete]
list1str = ['temporal_1_single', 'temporal_2_single', 'temporal_3_single', 'temporal_4_single',
        'temporal_1_complete', 'temporal_2_complete', 'temporal_3_complete', 'temporal_4_complete']
#list1 = [lrow_clusters_single, brow_clusters_single, nrow_clusters_single, erow_clusters_single]
list1 = [_1row_clusters_complete, _2row_clusters_complete, _3row_clusters_complete, _4row_clusters_complete]
list1str = ['temporal_1_complete', 'temporal_2_complete', 'temporal_3_complete', 'temporal_4_complete']

spatial_auto_dict ={}
statespatial_auto001 =[]
statespatial_auto05 =[]
statespatial_auto01 =[]
statespatial_auto =[]

statespatial_auto001_dict ={}
statespatial_auto05_dict ={}
statespatial_auto01_dict ={}
statespatial_auto_dict ={}

statespatial_auto001_count = ''
statespatial_auto05_count = ''
statespatial_auto01_count = ''
statespatial_auto_count = ''

morani = []
p_values = []
p_values_ = []
p_values_001 = []
p_values_05 = []
p_values_01 = []
z_score = []
z_score_ = []
z_score_001 = []
z_score_05 = []
z_score_01 = []
spatial_auto_dict ={}
p_values = []
clusters_assignments = []
clusters_dicts = {}
state = []
cutoff = []
list2 = np.linspace(0.0, 2.5, num=51)
#list2 = np.linspace(0.0, 1.2, num=60)
#list2 = np.linspace(0.0, 2.0, num=10)


#list2 = np.linspace(0.0, 0.5, num=7)
#str1 = ''.join(str(e) for e in list1)
#for xx in list1:
for y,z, w in zip(list1,list1str,ww0):
    for cut_off in list2:#new

      #  print(str(z))
        assignments = fcluster(y,cut_off,'distance')  #0.45 e
       # labels=interfe_df.index#######################################################################change this one
        labels=temporal_1_df.index#######################################################################change this one

        cluster_output = pd.DataFrame({'occr_country':labels , 'cluster':assignments})
        #print(cluster_output)
            #clusters_assignments.append(assignments)
        #print('cluster cut off point= ', str(y))############
        #print(set(assignments))###########################################################################################
        clusters_assignments = []
        clusters_assignments.append(assignments)
       # print(clusters_assignments)
        mx = np.max(clusters_assignments)
        mx = mx+1
        #print(mx)
       # print(mx-1, 'clusters')############################################################################################
            #clusters_assignments.append(assignments)
        cutoff.append(y)
    #print('name of cluster =', str(str1))

        for x in range(1,mx):#mx
            column = ('cluster' + str(x))
            cluster_output[column] = (cluster_output['cluster'] ==x).astype(int)
            cluster_output[column]
            #print('culumns=', str(x))
            cb = cluster_output[column]#column
           # w=ps.lat2W(countofcountries,1, rook=False)

            #w=lat2W(countofcountries,1, rook=False)

            #jc=ps.Join_Counts(cb,w,permutations=999)

            #jc=Join_Counts(cb,w,permutations=999)
            #jc=Moran(w)
            jc=Moran(cb,  w,transformation='r', two_tailed=True)
           # jc=Moran_Local(cb,  w,transformation = "r", permutations = 9,geoda_quads=True)
            #print(jc.p_z_sim)
            #if jc.p_norm<=0.09:
            
           #     print(jc.p_norm, 'jc')
                
           # if tc.p_norm<=0.09:
           #     print(tc.p_norm,'tc')
           # else:
           #     pass
            p_values.append(jc.p_rand)
            z_score.append(jc.z_rand)

            morani.append(jc.I)

            #print(p_values)
            cutoff.append(y)
            cluster_output['p_values_'+str('for_cluster_column_')+str(x)] = jc.p_rand
            cluster_output['z_score_'+str('for_cluster_column_')+str(x)] = jc.z_rand

            cluster_output['morani'+str('for_cluster_column_')+str(x)] = jc.I

            #sstate = cluster_output['STATE_ABBR']
            #state.append(sstate)
            #print(state)
            #zero__dot_01_pvalue = cluster_output[cluster_output['p_values_'+str('for_cluster_column_')+str(x)] <0.01]
            #zero__dot_05_pvalue = cluster_output[cluster_output['p_values_'+str('for_cluster_column_')+str(x)] <0.05]
            #zero__dot_1_pvalue = cluster_output[cluster_output['p_values_'+str('for_cluster_column_')+str(x)] <0.1]
            #print(zero__dot_01_pvalue,zero__dot_05_pvalue, zero__dot_1_pvalue)
            #spatial_auto_dict = dict(zip(zero_dot_05_state, p_values))
            #spatial_auto_dict = dict(zip(p_values, cluster_output['STATE_ABBR']))
            if jc.p_rand<0.01:
                #print('culumns=', str(x))
                #print(jc.p_rand, 'pvalue <0.01')
               # print(jc.I)
               # print('very very very very significant__'*50)
                zero_dot_01_state = cluster_output['cluster' + str(x)] == 1
                zero__dot_01_pvalue = cluster_output['p_values_'+str('for_cluster_column_')+str(x)] <0.01
                #print(cluster_output[zero_dot_01_state& zero__dot_01_pvalue]['occr_country'])
                sstate_001 =  cluster_output[zero_dot_01_state& zero__dot_01_pvalue]['occr_country']  
                statespatial_auto001_count = sstate_001.shape[0]
                #print(statespatial_auto001_count)
                dat01 = dat01.append({'pvalue': float(jc.p_rand),'zscore': float(jc.z_rand), 'moran_i': float(jc.I), 'countries':str(cluster_output[zero_dot_01_state& zero__dot_01_pvalue]['occr_country'].str[-2:]), 'dendo_cluster': int(statespatial_auto001_count), 'dendo_cutoff':float(cut_off), 'temporal': str(temporal),'temporal2': str(z)},ignore_index=True)
            elif jc.p_rand<0.05:
                #print('culumns=', str(x))
                #print('very significant__'*10)
                #print(jc.p_sim_bb, 'bb')
                zero_dot_05_state = cluster_output['cluster' + str(x)] == 1
                zero__dot_05_pvalue = cluster_output['p_values_'+str('for_cluster_column_')+str(x)] <0.05
                #print(cluster_output[zero_dot_05_state& zero__dot_05_pvalue]['occr_country'])
                sstate_05 =  cluster_output[zero_dot_05_state& zero__dot_05_pvalue]['occr_country']  
                statespatial_auto05_count = sstate_05.shape[0]
                #print(statespatial_auto05_count)
                dat05 = dat05.append({'pvalue': float(jc.p_rand),'zscore': float(jc.z_rand),'moran_i': float(jc.I), 'countries':str(cluster_output[zero_dot_05_state& zero__dot_05_pvalue]['occr_country'].str[-2:]), 'dendo_cluster': int(statespatial_auto05_count), 'dendo_cutoff':float(cut_off), 'temporal': str(temporal),'temporal2': str(z)},ignore_index=True)

            elif jc.p_rand<0.1:
                #print('culumns=', str(x))
                #print(jc.p_sim_bb, 'bb')
                #print('significant__'*10)
                zero_dot_1_state = cluster_output['cluster' + str(x)] == 1
                zero__dot_1_pvalue = cluster_output['p_values_'+str('for_cluster_column_')+str(x)] <0.1
                #print(cluster_output[zero_dot_1_state& zero__dot_1_pvalue]['occr_country'])
                sstate_1 =  cluster_output[zero_dot_1_state& zero__dot_1_pvalue]['occr_country']
                statespatial_auto1_count = sstate_1.shape[0]
                #print(statespatial_auto1_count)
                dat1 = dat1.append({'pvalue': float(jc.p_rand),'zscore': float(jc.z_rand), 'moran_i': float(jc.I),'countries':str(cluster_output[zero_dot_1_state& zero__dot_1_pvalue]['occr_country'].str[-2:]), 'dendo_cluster': int(statespatial_auto1_count), 'dendo_cutoff':float(cut_off), 'temporal': str(temporal),'temporal2': str(z)},ignore_index=True)

            elif jc.p_rand>=0.1:
                #print('culumns=', str(x))
                #print(jc.p_sim_bb, 'bb')
                #print('significant__'*10)
                zero_dot_n_state = cluster_output['cluster' + str(x)] == 1
                zero__dot_n_pvalue = cluster_output['p_values_'+str('for_cluster_column_')+str(x)] >=0.1
                #print(cluster_output[zero_dot_1_state& zero__dot_1_pvalue]['occr_country'])
                sstate_n =  cluster_output[zero_dot_n_state& zero__dot_n_pvalue]['occr_country']
                statespatial_auton_count = sstate_n.shape[0]
                #print(statespatial_auto1_count)
                datn = datn.append({'pvalue': float(jc.p_rand),'zscore': float(jc.z_rand),'moran_i': float(jc.I), 'countries':str(cluster_output[zero_dot_n_state& zero__dot_n_pvalue]['occr_country'].str[-2:]), 'dendo_cluster': int(statespatial_auton_count), 'dendo_cutoff':float(cut_off), 'temporal': str(temporal),'temporal2': str(z)},ignore_index=True)


            #elif jc.p_sim_bw<0.05:
             #   print('culumns=', str(x))
              #  print('very significant__'*10)
               # print(jc.p_sim_bw, 'bw')
                #print(cluster_output[zero_dot_1_state& zero__dot_1_pvalue]['STATE_ABBR'])

            #elif jc.p_sim_bw<0.1:
             #   print('culumns=', str(x))
              #  print(jc.p_sim_bw, 'bw')
               # print('significant__'*10)
                #print(cluster_output[zero_dot_1_state& zero__dot_1_pvalue]['STATE_ABBR'])

                #print('postive spatial autocorrelation=', jc.p_sim_bb)
    #np.mean(jc.sim_bb)
    #np.max(jc.sim_bb)
    #np.min(jc.sim_bb)

            #len(jc.sim_bw)
                #print('negative spatial autocorrelation=',jc.p_sim_bw)
    #np.mean(jc.sim_bw)
    #np.max(jc.sim_bw)
    #np.min(jc.sim_bw)

                #p_values


            clusters_dicts =dict(zip(p_values, cluster_output['cluster']))
    #row_count_itemset_dict = dict(zip(zero_dot_05_state, zero__dot_05_pvalue))
print('done')

In [ ]:
datn
datn['countries'] = datn['countries'].replace('\n',',', regex=True)
datn['countries'] = datn['countries'].str.replace('\d+', '')
datn['countries'] = datn.countries.str[:-34]
datn['countries'] = datn.countries.str.strip()
datn = datn.drop_duplicates(subset=['countries','dendo_cluster'], keep='first')
datn = datn[(datn.dendo_cutoff>=0.1) & (datn.dendo_cluster>=2)]

len(datn.index)

In [ ]:
dat01
dat01['countries'] = dat01['countries'].replace('\n',',', regex=True)
dat01['countries'] = dat01['countries'].str.replace('\d+', '')
dat01['countries'] = dat01.countries.str[:-34]
dat01['countries'] = dat01.countries.str.strip()
dat01 = dat01.drop_duplicates(subset=['countries','dendo_cluster'], keep='first')

dat01

In [ ]:
dat05
dat05['countries'] = dat05['countries'].replace('\n',',', regex=True)
dat05['countries'] = dat05['countries'].str.replace('\d+', '')

dat05['countries'] = dat05.countries.str[:-34]
dat05['countries'] = dat05.countries.str.strip()
dat05 = dat05.drop_duplicates(subset=['countries','dendo_cluster'], keep='first')
dat05

In [ ]:
dat1
dat1['countries'] = dat1['countries'].replace('\n',',', regex=True)
dat1['countries'] = dat1['countries'].str.replace('\d+', '')

dat1['countries'] = dat1.countries.str[:-34]
dat1['countries'] = dat1.countries.str.strip()
dat1 = dat1.drop_duplicates(subset=['countries','dendo_cluster'], keep='first')

dat1


In [ ]:

#Using complete linkage and positive autocorrelation only

frame = [datn,dat1,dat05,dat01]
dat = pd.concat(frame)
dat.pvalue = dat.pvalue.astype(float)
dat['pvalue']=dat['pvalue'].round(decimals=5)
dat =dat[dat['dendo_cluster']>=2]
dat = dat[dat['pvalue']<=.05]
dat = dat.reset_index(drop=True)

datt=dat
datt

In [ ]:
brand = "xarelto"
drug = "Rivaroxaban"
k =topic

dat['temporal'] = dat['temporal2']
dat['temporal'] = dat['temporal2'].replace({'temporal_1_complete':'2014', 'temporal_2_complete':'2015', 'temporal_3_complete':'2016', 'temporal_4_complete':'2017'})
dat= dat[dat['temporal'].str.contains('2014|2015|2016|2017')]
dat['Topic'] = str(k)+' Topics'
dat['Brand Name'] = brand.title()#'Eliquis'#Pradaxa#Eliquis
dat['Drug Name'] =  drug#'Apixaban'#Dabigatran#Apixaban
dat.rename(columns={'dendo_cluster':'# of countries in cluster', 'temporal':'Temporal', 'moran_i':'Moran_i','pvalue':'Pvalue', 'countries':'Countries','zscore':'Zscore'  }, inplace=True)
dat['Statistically Significant'] = ''
dat['Spatial Pattern'] = ''
dat =dat[['Statistically Significant','Spatial Pattern','Pvalue','Zscore','Moran_i', 'Countries', '# of countries in cluster', 'Temporal','Brand Name','Drug Name', 'Topic']]
dat['Moran_i'] =dat['Moran_i'].apply(lambda x: round(x, 5))
dat['Pvalue'] =dat['Pvalue'].apply(lambda x: round(x, 5))
dat.loc[(dat['Pvalue'] <= 1) & (dat['Pvalue'] > 0.1), ["Statistically Significant"]] = 'Not Significant' 

dat.loc[(dat['Pvalue'] <= 0.1) & (dat['Pvalue'] > 0.05), ["Statistically Significant"]] = 'less than 0.1' 
dat.loc[(dat['Pvalue'] <= 0.05) & (dat['Pvalue'] > 0.01), ["Statistically Significant"]] = 'less than 0.05' 
dat.loc[(dat['Pvalue'] <= 0.01) & (dat['Pvalue'] >= 0), ["Statistically Significant"]] = 'less than 0.01' 

dat.loc[(dat['Zscore'] < -1.65) & (dat['Zscore'] > -10), ['Spatial Pattern']] = 'Disperesed' 
dat.loc[(dat['Zscore'] < 10) & (dat['Zscore'] > 1.65), ['Spatial Pattern']] = 'Clustered' 
dat.loc[(dat['Zscore'] <= 1.65) & (dat['Zscore'] >= -1.65), ['Spatial Pattern']] = 'Random' 
dat

In [ ]:
weights_used= "weight1"


In [ ]:

#Using complete linkage and positive autocorrelation only

#frame = [datn,dat1,dat05,dat01]
#dat = pd.concat(frame)
#dat.pvalue = dat.pvalue.astype(float)
#dat['pvalue']=dat['pvalue'].round(decimals=5)
#dat =dat[dat['dendo_cluster']>2]
#dat[dat['pvalue']<=.05]
dat
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\pvalue_moran")
os.chdir(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Spatial_auto_test\tp"+str(topic)+"\\"+str(drugnamepath))


#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Spatial_auto_test\tp"+str(topic)+"\\"+str(drugnamepath))
#dat.to_csv('spatial_autocorrelation_test.csv')
dat.to_csv('spatial_autocorrelation_test_'+str(drugname)+'_'+str(temporal)+'_topic_'+str(topic)+'_'+str(countofcountries)+'countofcountries_version_moran_pvalue0dot05_clean'+str(weights_used)+'.csv')
datt.to_csv('spatial_autocorrelation_test_'+str(drugname)+'_'+str(temporal)+'_topic_'+str(topic)+'_'+str(countofcountries)+'countofcountries_version_moran_pvalue0dot05_raw'+str(weights_used)+'.csv')

#dat[dat['pvalue']<=.01]
dat

In [ ]:
#dat1 = datt[(datt['pvalue']<=.05) & (datt['temporal2']=='temporal_3_complete')]
#dat1 = dat[(dat['Pvalue']<=.01) & (dat['Temporal']=='2016')]
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Spatial_auto_test\tp"+str(topic)+"\\"+str(drugnamepath))

#datt['pvalue'] =datt['pvalue'].astype(float)
#datt['pvalue'].iloc[1]
#datt = pd.read_csv('spatial_autocorrelation_test_'+str(drugname)+'_'+str(temporal)+'_topic_'+str(topic)+'_'+str(countofcountries)+'countofcountries_version_moran_pvalue0dot05_raw.csv')
datt

In [ ]:
stop

## pdf png Output 

In [ ]:
index=4
p_val_ =datt['pvalue'].iloc[index]
#cutoffval = dat.loc[dat['pvalue'] == 3, 'A']
dat_ =datt[datt.pvalue==p_val_]

cutoffval = dat_.loc[dat_.pvalue==p_val_,'dendo_cutoff']
pvalue = dat_[dat_.pvalue==p_val_]
te_num = dat_.temporal2.iloc[0][9:10]
te_num =int(te_num)
print(te_num)
#dat_ = dat_.loc[0]
#dat_ =dat_.head(1)
dat_

In [ ]:
cutoffval = dat_.loc[dat_.pvalue==p_val_,'dendo_cutoff']
pvalue = dat_[dat_.pvalue==p_val_]
te_num = dat_.temporal2.iloc[0][9:10]
te_num =int(te_num)
print(te_num)
#dat_ = dat_.loc[0]
#dat_ =dat_.tail(1)
temporal = dat_.temporal.iloc[0]

dat_


In [ ]:
#temporal_1_['occr_country']
temporal

In [ ]:
#spatial_list= temporal_1_.spatial.tolist()
#print(len(spatial_list))

In [ ]:
temporal_1_.spatial = temporal_1_.occr_country
temporal_2_.spatial = temporal_2_.occr_country
temporal_3_.spatial = temporal_3_.occr_country
temporal_4_.spatial = temporal_4_.occr_country


In [ ]:
dat_['countries'] = dat_['countries'].str.strip()
dat_['countries'] = dat_['countries'].str.strip()
#Faers_comparision['FaersValue'] = Faers_comparision['FaersValue'].str.lower()
dat_['countries'] = dat_['countries'].str.replace(u"/",",")
dat_['countries'] = dat_['countries'].str.replace(u"\\\\", ",")

dat_['countries'] = dat_['countries'].str.replace(u"' ","'")

dat_['countries'] = dat_['countries'].str.replace('\n',"")
dat_['countries'] = dat_['countries'].str.replace(' \n',"")
dat_['countries'] = dat_['countries'].str.replace('\n ',"")

dat_['countries'] = [''.join(c.split()) for c in dat_['countries'].astype(str)]

dat_['countries'] = dat_['countries'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r"\n",  '', regex=True)
dat_['countries'] = dat_['countries'].str.replace(r"[\"\']", '')
dat_['countries'] = dat_['countries'].str.replace(',,,',',')
dat_['countries'] = dat_['countries'].str.replace(',,',',')
dat_['countries'] = dat_['countries'].str.replace('{','')
dat_['countries'] = dat_['countries'].str.replace('','')

#dat05['countries'] = dat05['countries'].str[1:]
#dat05['countries'] = dat05['countries'].str[:-1]
test = dat_['countries']
spatial_country_check = []
for x in test:
    spatial_country_check.append(x)
spatial_country_check
pattern = [item.replace(",", "|") for item in spatial_country_check]
pattern =pattern[-1]
for x in [te_num]:
    if (x) == 3:
        tempdf = temporal_3_[temporal_3_['occr_country'].str.contains(pattern)]
    if (x) == 2:
        tempdf = temporal_2_[temporal_2_['occr_country'].str.contains(pattern)]
    if (x) == 1:
        tempdf = temporal_1_[temporal_1_['occr_country'].str.contains(pattern)]
    if (x) == 4:
        tempdf = temporal_4_[temporal_4_['occr_country'].str.contains(pattern)]
#tempdf['pvalue']=np.float
#tempdf = temporal_2_[temporal_2_['spatial'].str.contains(pattern)]


#tempdf.head(1)


In [ ]:
pval = pvalue['pvalue'].values
pval = str(pval)
tempdf['pvalue']=pval
cutoffval = pvalue['dendo_cutoff'].values
cutoffval = float(cutoffval)
tempdf['cutoff']=cutoffval
tempdf['cutoff'] =round(tempdf['cutoff'],3)
#tempdf.head(1)
lm = dat_.temporal2.iloc[0][11:]


In [ ]:

#os.chdir(r"\\Fdswp03752\ocmgis_data\Scripts\Situation_Awareness_Natural_hazards\shapefiles\World")
os.chdir(r"D:\shapefiles\World")
world_shp = gpd.read_file(filename='world.shp')
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.strip()#ISOSHRTNAM
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.lower()
world_shp['CNTRY_NAME'] = world_shp['CNTRY_NAME'].str.strip()#ISOSHRTNAM
world_shp['CNTRY_NAME'] = world_shp['CNTRY_NAME'].str.lower()

world_shp.head()
world_shp.ISO_2DIGIT
europe = ['AL', 'AD', 'AM', 'AT', 'BY', 'BE', 'BA', 'BG', 'CH','CY', 'CZ','DE',
        'DK', 'EE', 'ES', 'FO', 'FI', 'FR','GB', 'GE', 'GI', 'GR', 'HU', 'HR',
        'IE', 'IS','IT','LU','LT', 'LV', 'MC', 'MK', 'MT', 'NO', 'NL', 'PL',
        'PT', 'RO', 'SE', 'SI', 'SK', 'SM', 'TR', 'UA', 'VA','LU']


world_shp1 = world_shp[world_shp['ISO_2DIGIT'].str.contains('|'.join(europe),case=False, na=False, regex=True)]

world_shp1['centroid'] = world_shp1['geometry'].centroid
world_shp1['coords'] = world_shp1['geometry'].apply(lambda x: x.representative_point().coords[:])
world_shp1['coords'] = [coords[0] for coords in world_shp1['coords']]
world_shp1['coords']

world_shp1['Lon'] = world_shp1.centroid.x
world_shp1['Lat'] = world_shp1.centroid.y

world_shp1
tempdf = pd.merge(tempdf,
                 world_shp1[['ISO_2DIGIT', 'CNTRY_NAME']],
                  left_on='occr_country',
                  right_on='ISO_2DIGIT',
                  how='left')
#tempdf.head(1)


#tempdf['FaersValuev2'] = (tempdf['FaersValue'].str.split(', ').apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(', '))

In [ ]:
len(tempdf.index)


from collections import Counter
count  = tempdf['FaersValue'].str.split(', ').map(Counter).sum()
c= pd.DataFrame(list(count.items()), columns=['PT', 'Overall_PT_Count_country'])
c.PT=c.PT.str.strip()
#c =c[c.Overall_PT_Count_country>=0]

maptext =c[c.Overall_PT_Count_country>=0]
space = []
clus_count = []
for index, row in maptext.PT.iteritems():
    #print(index,row)
    val = maptext.PT.loc[index]
    space1 = tempdf[tempdf.FaersValue.str.contains(val, na=False, regex=True)]
    space1 = space1.CNTRY_NAME
    space.append(space1.tolist())
maptext['CountryPT_in_Cluster']=space
maptext = maptext.reset_index(drop=True)
for index, row in maptext.CountryPT_in_Cluster.iteritems():
    y = len(maptext['CountryPT_in_Cluster'].iloc[index])
    clus_count.append(y)
maptext['CountryPT_in_Cluster_count']=clus_count
    
maptext['percent_in_cluster']=round(maptext['CountryPT_in_Cluster_count']/maptext.Overall_PT_Count_country,3)*100
c = maptext
maptext =maptext[maptext.CountryPT_in_Cluster_count>=2]

maptext

In [ ]:
#os.chdir(r"G:\Research\FAERS_Public\Results\Dendo\tp10"+str('\\')+str(drugnamepath))
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Dendo_plus_Jaccard\tp"+str(topic)+"\\"+str(drugnamepath))
#os.chdir(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Dendo_plus_Jaccard\tp"+str(topic)+"\\"+str(drugnamepath))
os.chdir(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Dendo_plus_Jaccard\tp"+str(topic)+"\\"+str(drugnamepath))

import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage

linkagemethod =['single','complete']
#complete

#sns.set(font_scale=2, rc={'figure.figsize':(250,250)})

#sns.set(font_scale=2, rc={'figure.figsize':(50,50)})

plt.clf()
#plt.figure(figsize=(30, 15))
#plt.tick_params(axis='both', left='on', top='off', right='on', bottom='on', labelleft='on', labeltop='off', labelright='on', labelbottom='on')
#sns.set(font_scale=2)#, rc={'figure.figsize':(400,400)})

temporallist = [temporal_1,temporal_2,temporal_3,temporal_4]
llist = [temporal_1_distance1,temporal_2_distance1,temporal_3_distance1,temporal_4_distance1]
#plt.tick_params(labelsize=25)
#plt.xticks(fontsize=60, rotation=90) #font size for x ticks
#plt.yticks(fontsize=60) #font size for x ticks


for x in [te_num]:
    if (x) == 3:
        number = temporal_3
        _mat = np.array([[temporal_3_distance1]])
        _mat = _mat.reshape(countofcountries, countofcountries)
        #Z = linkage(_mat, lm)
        #plt.xticks(fontsize=5, rotation=90) #font size for x ticks
       #plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        Z = linkage(_mat, 'complete')

        g =sns.clustermap(temporal_4_distance,  row_linkage=Z, col_linkage=Z,row_cluster=True, col_cluster=True,yticklabels=1,xticklabels=1)# change here

       # plt.title("Hierarchical Clustering (Method = "+str(lm)+", Top 10 frequent items), drug name: "+str(drugname)+", temporal: "+str(y), fontsize=15)#Change here


        #plt.tick_params(labelsize=2)
        plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0, fontsize=20)  # For y axis
        plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90, fontsize=20) # For x axis
        sns.set(font_scale=1.5)
        #g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 15)
        #plt.xticks(fontsize=25, rotation=45) #font size for x ticks
        plt.yticks(fontsize=15) #font size for x ticks
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        #plt.box('off')

        plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+"_"+str(lm)+'index'+str(index)+str(weights_used)+".png", dpi=300, bbox_inches="tight")
       # plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_"+str(lm)+'version'+str(version)+".pdf", dpi=300, bbox_inches="tight")

        #plt.show()
        #plt.clf()
        
    if (x) == 2:
        number = temporal_2
        _mat = np.array([[temporal_2_distance1]])
        _mat = _mat.reshape(countofcountries, countofcountries)
       # Z = linkage(_mat, lm)
        #plt.xticks(fontsize=5, rotation=90) #font size for x ticks
        #plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        Z = linkage(_mat, 'complete')

        g =sns.clustermap(temporal_4_distance,  row_linkage=Z, col_linkage=Z,row_cluster=True, col_cluster=True,yticklabels=1,xticklabels=1)# change here

       # plt.title("Hierarchical Clustering (Method = "+str(lm)+", Top 10 frequent items), drug name: "+str(drugname)+", temporal: "+str(y), fontsize=15)#Change here


        #plt.tick_params(labelsize=2)
        plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0, fontsize=20)  # For y axis
        plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90, fontsize=20) # For x axis
        sns.set(font_scale=1.5)
        #g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 15)
        #plt.xticks(fontsize=25, rotation=45) #font size for x ticks
        plt.yticks(fontsize=15) #font size for x ticks
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        #plt.box('off')

        plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+"_"+str(lm)+'index'+str(index)+str(weights_used)+".png", dpi=300, bbox_inches="tight")
        #plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_"+str(lm)+'version'+str(version)+".pdf", dpi=300, bbox_inches="tight")

        #plt.show()
        #plt.clf()
    if (x) == 1:
        number = temporal_1
        _mat = np.array([[temporal_1_distance1]])
        _mat = _mat.reshape(countofcountries, countofcountries)
       # Z = linkage(_mat, lm)
        #plt.xticks(fontsize=5, rotation=90) #font size for x ticks
       # plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        Z = linkage(_mat, 'complete')
#, color_threshold=0.212)#, color_threshold=0.3)#, color_threshold=0.02)
        g =sns.clustermap(temporal_4_distance,  row_linkage=Z, col_linkage=Z,row_cluster=True, col_cluster=True,yticklabels=1,xticklabels=1)# change here

       # plt.title("Hierarchical Clustering (Method = "+str(lm)+", Top 10 frequent items), drug name: "+str(drugname)+", temporal: "+str(y), fontsize=15)#Change here


        #plt.tick_params(labelsize=2)
        plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0, fontsize=20)  # For y axis
        plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90, fontsize=20) # For x axis
        sns.set(font_scale=1.5)
        #g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 15)
        #plt.xticks(fontsize=25, rotation=45) #font size for x ticks
        plt.yticks(fontsize=15) #font size for x ticks
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        #plt.box('off')

        plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+"_"+str(lm)+'index'+str(index)+str(weights_used)+".png", dpi=300, bbox_inches="tight")
       # plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_"+str(lm)+'version'+str(version)+".pdf", dpi=300, bbox_inches="tight")

        #plt.show()
        #plt.clf()
    if (x) == 4:
        number = temporal_4
        _mat = np.array([[temporal_4_distance1]])
        _mat = _mat.reshape(countofcountries, countofcountries)
        #Z = linkage(_mat, lm)
        #plt.xticks(fontsize=5, rotation=90) #font size for x ticks
        plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        Z = linkage(_mat, 'complete')
#, color_threshold=0.212)#, color_threshold=0.3)#, color_threshold=0.02)
        g =sns.clustermap(temporal_4_distance,  row_linkage=Z, col_linkage=Z,row_cluster=True, col_cluster=True,yticklabels=1,xticklabels=1)# change here

       # plt.title("Hierarchical Clustering (Method = "+str(lm)+", Top 10 frequent items), drug name: "+str(drugname)+", temporal: "+str(y), fontsize=15)#Change here


        #plt.tick_params(labelsize=2)
        plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0, fontsize=20)  # For y axis
        plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90, fontsize=20) # For x axis
        sns.set(font_scale=1.5)
        #g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 15)
        #plt.xticks(fontsize=25, rotation=45) #font size for x ticks
        plt.yticks(fontsize=15) #font size for x ticks
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        #plt.box('off')

        #plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.png", dpi=300)
        plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+"_"+str(lm)+'index'+str(index)+str(weights_used)+".png", dpi=300, bbox_inches="tight")
       # plt.savefig("Dendrogram_Tree_cluster_Jaccard_Similiarity_testing_"+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_"+str(lm)+'version'+str(version)+".pdf", dpi=300, bbox_inches="tight")

        #plt.show()
        #plt.clf()
    #tempdf['pvalue']=np.float
#tempdf = temporal_2_[temporal_2_['spatial'].str.contains(pattern)]


plt.show()
plt.clf()


In [ ]:
te_num
cutoffval


In [ ]:
#os.chdir(r"G:\Research\FAERS_Public\Results\Dendo\tp10"+str('\\')+str(drugnamepath))
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Dendo\tp"+str(topic)+"\\"+str(drugnamepath))
os.chdir(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Dendo\tp"+str(topic)+"\\"+str(drugnamepath))

import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage

linkagemethod =['single','complete']
#complete

plt.figure(figsize=(25, 15))
plt.tick_params(axis='both', left='off', top='off', right='on', bottom='on', labelleft='off', labeltop='off', labelright='off', labelbottom='on')
sns.set(font_scale=3)#, rc={'figure.figsize':(400,400)})


temporallist = [temporal_1,temporal_2,temporal_3,temporal_4]
llist = [temporal_1_distance1,temporal_2_distance1,temporal_3_distance1,temporal_4_distance1]

for x in [te_num]:
    if (x) == 3:
        number = temporal_3
        _mat = np.array([[temporal_3_distance1]])
        _mat = _mat.reshape(countofcountries, countofcountries)
        #Z = linkage(_mat, 'complete')
        plt.xticks(fontsize=5, rotation=90) #font size for x ticks
        plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        dendrogram(Z,
            leaf_rotation=90.,  # rotates the x axis labels
            leaf_font_size=35.,
            labels=temporal_3_df.index, count_sort=False, distance_sort=True,color_threshold=cutoffval )#, color_threshold=0.212)#, color_threshold=0.3)#, color_threshold=0.02)

        plt.yticks([])
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        plt.box('off')
        plt.title("Hierarchical Cluster Analysis (Linkage Method = "+str(lm)+", Topic "+str(topic)+", Pharmaceutical Drug: "+str(drugname)+", Temporal: "+str(temporal), fontsize=30)#Change here

        plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+'index'+str(index)+".png", dpi=300)
        #plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.pdf", dpi=300)

        #plt.show()
        #plt.clf()
        
    if (x) == 2:
        number = temporal_2
        _mat = np.array([[temporal_2_distance1]])
        _mat = _mat.reshape(countofcountries, countofcountries)
        Z = linkage(_mat, 'complete')
        plt.xticks(fontsize=5, rotation=90) #font size for x ticks
        plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        dendrogram(
            Z,
            leaf_rotation=90.,  # rotates the x axis labels
            leaf_font_size=35.,
            labels=temporal_2_df.index, count_sort=False, distance_sort=True,color_threshold=cutoffval )#, color_threshold=0.212)#, color_threshold=0.3)#, color_threshold=0.02)
        plt.yticks([])
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        plt.box('off')
        plt.title("Hierarchical Cluster Analysis (Linkage Method = "+str(lm)+", Topic "+str(topic)+", Pharmaceutical Drug: "+str(drugname)+", Temporal: "+str(temporal), fontsize=30)#Change here

        plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+'index'+str(index)+".png", dpi=300)
       # plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.pdf", dpi=300)

       # plt.show()
       # plt.clf()
    if (x) == 1:
        number = temporal_1
        #_mat = np.array([[temporal_1_distance1]])
        
        _mat = np.array([[temporal_1_distance1]])#lenalid_distance2
        _mat = _mat.reshape(len(var), len(var))
        Z = linkage(_mat, lm)#LBNE_mat_distArray





        plt.xticks(fontsize=5, rotation=90) #font size for x ticks
        plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        dendrogram(
            Z,
            leaf_rotation=90.,  # rotates the x axis labels
            leaf_font_size=35.,
            labels=temporal_1_df.index, count_sort=False, distance_sort=True,color_threshold=cutoffval )#, color_threshold=0.212)#, color_threshold=0.3)#, color_threshold=0.02)


        plt.title("Hierarchical Cluster Analysis (Linkage Method = "+str(lm)+", Topic "+str(topic)+", Pharmaceutical Drug: "+str(drugname)+", Temporal: "+str(temporal), fontsize=30)#Change here
        plt.yticks([])
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        plt.box('off')
        plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+'index'+str(index)+".png", dpi=300)
       # plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.pdf", dpi=300)

        plt.show()
        plt.clf()
    if (x) == 4:
        number = temporal_4
        _mat = np.array([[temporal_4_distance1]])
        _mat = _mat.reshape(countofcountries, countofcountries)
        Z = linkage(_mat, 'complete')
        plt.xticks(fontsize=5, rotation=90) #font size for x ticks
        plt.tight_layout()
        #plt.axhline(y=0.3, c='grey', lw=1, linestyle='dashed')
        dendrogram(Z,
            leaf_rotation=90.,  # rotates the x axis labels
            leaf_font_size=35.,
            labels=temporal_4_df.index, count_sort=False, distance_sort=True,color_threshold=cutoffval )#, color_threshold=0.212)#, color_threshold=0.3)#, color_threshold=0.02)

        plt.title("Hierarchical Cluster Analysis (Linkage Method = "+str(lm)+", Topic "+str(topic)+", Pharmaceutical Drug: "+str(drugname)+", Temporal: "+str(temporal), fontsize=30)#Change here
        plt.yticks([])
        #plt.ylabel('Similiarity between ' +str(countofcountries)+' European Countries using FAERS Post Market Adverse Events', fontsize=20)#Change here
        #plt.ylabel('Similiarity Inverse Distances from Jaccard Coefficients', fontsize=20)
        plt.box('off')

        plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+'index'+str(index)+".png", dpi=300)
        #plt.savefig("Dendrogram_Tree_cluster_linkage_"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(y)+"_version_naive.pdf", dpi=300)

      #  plt.show()
       # plt.clf()
    #tempdf['pvalue']=np.float
#tempdf = temporal_2_[temporal_2_['spatial'].str.contains(pattern)]


plt.show()
plt.clf


In [ ]:
te_num
#tempdf
#c

In [ ]:
prev_result =dat_[['countries']]
prev_result


In [ ]:
prev_result2 = pd.DataFrame(prev_result.countries.str.split(',').tolist())
prev_result2 = prev_result2.T
prev_result2['cluster']=0
prev_result2.columns.values[0] = "occr_country"

prev_result2

In [ ]:
#importing modules
import os 
print('importing "os" operating system')
from io import BytesIO
#from urllib.request import urlopen
from zipfile import ZipFile
import pandas as pd
print('pandas')
import geopandas as gpd
print('geopandas')
from shapely.geometry import Point
print('shapely')
import numpy as np
print('numpy')
from statsmodels.tsa.stattools import grangercausalitytests
print('statsmodels')
import pysal as ps
print('pysal')
import seaborn as sns
print('seaborn')
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
print('matplotlib')


%matplotlib inline
import os
import pandas as pd

In [ ]:
os.chdir(r"D:\shapefiles\World")
world_shp = gpd.read_file(filename='world.shp')
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.strip()#ISOSHRTNAM
world_shp['ISO_2DIGIT'] = world_shp['ISO_2DIGIT'].str.lower()
world_shp.head()
world_shp.ISO_2DIGIT
europe = ['AL', 'AD', 'AM', 'AT', 'BY', 'BE', 'BA', 'BG', 'CH', 'CY', 'CZ', 'DE',
        'DK', 'EE', 'ES', 'FO', 'FI', 'FR', 'GB', 'GE', 'GI', 'GR', 'HU', 'HR',
        'IE', 'IS', 'IT', 'LT', 'LU', 'LV', 'MC', 'MK', 'MT', 'NO', 'NL', 'PL',
        'PT', 'RO', 'SE', 'SI', 'SK', 'SM', 'TR', 'UA', 'VA']
spatial_list =var# ["de","fr","gb","it","be","ch","es","nl","cz","se","gr","at","ie","pt","dk","no","sk","fi","hu"] # took out us, ca, pr, ru
world_shp2 = world_shp[world_shp['ISO_2DIGIT'].str.contains('|'.join(spatial_list),case=False, na=False, regex=True)]
world_shp1 = world_shp[world_shp['ISO_2DIGIT'].str.contains('|'.join(europe),case=False, na=False, regex=True)]
world_shp2['centroid'] = world_shp2['geometry'].centroid
world_shp2['coords'] = world_shp2['geometry'].apply(lambda x: x.representative_point().coords[:])
world_shp2['coords'] = [coords[0] for coords in world_shp2['coords']]
world_shp2['coords']
world_shp1['centroid'] = world_shp1['geometry'].centroid
world_shp1['coords'] = world_shp1['geometry'].apply(lambda x: x.representative_point().coords[:])
world_shp1['coords'] = [coords[0] for coords in world_shp1['coords']]
world_shp1['coords']

world_shp1['Lon'] = world_shp1.centroid.x
world_shp1['Lat'] = world_shp1.centroid.y

alleurope1=world_shp1
someeurope=world_shp2



In [ ]:
alleurope1.ISO_2DIGIT = alleurope1.ISO_2DIGIT.str.strip()
prev_result2.occr_country = prev_result2.occr_country.str.strip()
world_join = alleurope1.merge(prev_result2 , left_on='ISO_2DIGIT', right_on='occr_country', how='outer')
world_join = world_join[~world_join["occr_country"].isnull()]
world_join.info()


Products_list = [maptext.columns.values.tolist()]+ maptext[['PT','CountryPT_in_Cluster_count','CountryPT_in_Cluster','percent_in_cluster']].values.tolist()
#Products_list = maptext[['PT','CountryPT_in_Cluster_count','CountryPT_in_Cluster','percent_in_cluster']].values.tolist()

f = '{:}|{:}|{:}|{:}\n' # formatting
text1_statecount=[]
for i in Products_list:
    text1_statecount.append(f.format(*i,'\n'))
    print(f.format(*i))

In [ ]:
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
#os.chdir(r"G:\Research\FAERS\Drug_Spatial\L_B_N_E\Data\Helper_Datasets\shapefiles\USstates")
#plt.title("Etanercept Hierarchical Clustering, method:single, (min sup=0.2, top (2) frequent items), cluster 4")

someeurope.ISO_2DIGIT = someeurope.ISO_2DIGIT.str.strip()
prev_result2.occr_country = prev_result2.occr_country.str.strip()
world_join = alleurope1.merge(prev_result2 , left_on='ISO_2DIGIT', right_on='occr_country', how='outer')
#world_join = someeurope.merge(prev_result2 , left_on='ISO_2DIGIT', right_on='occr_country', how='outer')

#world_join = world_join[world_join["occr_country"].isnull()]
world_join

##world_join.drop(columns, axis=1, inplace=True)
world_join['cluster'].fillna(1, inplace=True)

world_join['cluster'] = world_join['cluster'].astype(str).replace('\.0', '', regex=True)
world_join['cluster'] = world_join['cluster'].astype(str)
world_join['cluster'] = world_join['cluster'].str.replace('Nan','1')

cb = world_join[world_join['cluster']=='0']

#cb = usstates_join[(usstates_join['cluster3']==1) | (usstates_join['cluster7']==1)]

non_cb = world_join[world_join['cluster']=='1']
testcb = alleurope1[alleurope1['ISO_2DIGIT'].isin(spatial_list)]
#testcb = someeurope[someeurope['ISO_2DIGIT'].isin(spatial_list)]


#non_cb = usstates_join[(usstates_join['cluster3']==0) | (usstates_join['cluster7']==0)]
#usstates_join.to_csv('ttest.csv')
sns.set(font_scale=1, rc={'figure.figsize':(200,200)})

#cb.plot(color='gray', edgecolor='white', legend = True, linewidth= 5)

cb['coords'] = cb['geometry'].apply(lambda x: x.representative_point().coords[:])
cb['coords'] = [coords[0] for coords in cb['coords']]
cb['coords']

base = non_cb.plot(color='gray', edgecolor='white', legend = True, linewidth= 10)

testcb.plot(ax=base, color='orange', edgecolor='white', legend = True, linewidth= 10)
cb.plot(ax=base, color='red', edgecolor='white', legend = True, linewidth= 10)

cb.plot(ax=base, color='red', edgecolor='white', legend = True, linewidth= 10)
for idx, row in alleurope1.iterrows():
    #print(row)
    plt.annotate(s=row['CNTRY_NAME'], xy=row['coords'],horizontalalignment='center', fontsize= 80)
    

#for tt,r in zip(text1_statecount, (range(15,0,-2))):
  #  plt.text(-10, r, tt,fontsize=100, ha='left', rotation=0, wrap=True) 

#plt.title('Spatial Autocorrelation Test Map pvalue '+str(pval)+' Frequent Itemset Mining of Adverse Events for Pharmaceutical Drug: '+str(drugname), fontsize=225,y=1.05)
#plt.xlabel('Hierarchical Clustering, cutoff= '+str(cutoff)+ ' Temporal = '+str(temporal)+' for output of '+ str(count_countries)+' European Countries using FAERS Post Market Adverse Events', fontsize=150)
import matplotlib.patches as mpatches
gray_patch = mpatches.Patch(color='gray', label='Basemap')
red_patch = mpatches.Patch(color='orange', label='Spatial Pattern Not Significant')
orange_patch = mpatches.Patch(color='red', label='Significant Spatial Autocorrelation')# - Pvalue')#+str(pval)[1:-1])

plt.legend(handles=[gray_patch,red_patch,orange_patch], loc='lower left',markerscale=10, fontsize=200)

os.chdir(r"D:\Research\FAERS_Public_LDA\Paper_Ready_Figures\202103\Maps\tp"+str(topic)+"\\"+str(drugnamepath))
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Maps\tp"+str(topic)+"\\"+str(drugnamepath))
#os.chdir(r"D:\Research\FAERS_Public_LDA\Results\Maps\tp10\\"+str(drugnamepath))

plt.savefig("spatial_autocorrelation_map"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+'index'+str(index)+".png", bbox_inches="tight")
plt.savefig("spatial_autocorrelation_map"+str(lm)+'_'+str(countofcountries)+"_countries_drugname_"+str(drugname)+"_temporal_"+str(temporal)+'index'+str(index)+".pdf", dpi=500, bbox_inches="tight")


plt.show()
plt.clf()

In [ ]:
#testcb.info()
#spatial_list
stop

In [ ]:
te_num

In [ ]:
pval = pvalue['pvalue'].values
pval = str(pval)
tempdf['pvalue']=pval
cutoffval = pvalue['dendo_cutoff'].values
cutoffval = float(cutoffval)
tempdf['cutoff']=cutoffval
tempdf['cutoff'] =round(tempdf['cutoff'],3)
#lm = dat_.temporal2.iloc[0][11:]


In [ ]:
#cutoff = tempdf.cutoff[0]
#pval =tempdf.pvalue[0]
#tempor = tempdf.temporal[0]
#count_countries = len(tempdf.index)
print('cutoff=', cutoffval)
print('pval=',pval)
print('tempor=',temporal)
print('count_countries=',len(var))
print('drugname=',drugname)
lm
#lm= dat_.temporal2.iloc[0][11:]

In [ ]:
print('complete')